Import the data from Edge Impulse. You can obtain the URL from the Dashboard, right-click on the download icon next to 'Spectral features data' and 'Spectral features labels', and click **Copy link location**.

In [5]:
import numpy as np
import requests

API_KEY = 'ei_e06c9dca72e4fa1c636225c3dfc4ebb778310c72f94ae8cc1de4f2269f2dcdd4'

X = (requests.get('https://studio.edgeimpulse.com/v1/api/31143/training/5/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/31143/training/5/y', headers={'x-api-key': API_KEY})).content

Store the data in a temporary file, and load it back through Numpy.

In [9]:
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

#np.set_printoptions(Y=sys.maxsize)

#print(Y[:20])

ValueError: ndarray is not C-contiguous

Define our labels and split the data up in a test and training set:

In [3]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "Cutting", "noise" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def set_batch_size(batch_size, train_dataset, validation_dataset):
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return train_dataset, validation_dataset

def get_reshape_function(reshape_to):
    def reshape(image, label):
        return tf.reshape(image, reshape_to), label
    return reshape

callbacks = []


Train the model:

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

# model architecture
model = Sequential()
model.add(Reshape((int(input_length / 32), 32), input_shape=(input_length, )))
model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(classes, activation='softmax', name='y_pred'))

# this controls the learning rate
opt = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999)
# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset, validation_dataset = set_batch_size(BATCH_SIZE, train_dataset, validation_dataset)

# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_dataset, epochs=500, validation_data=validation_dataset, verbose=2, callbacks=callbacks)


Epoch 1/500
169/169 - 1s - loss: 0.6608 - accuracy: 0.6585 - val_loss: 0.6432 - val_accuracy: 0.6868
Epoch 2/500
169/169 - 1s - loss: 0.6431 - accuracy: 0.6775 - val_loss: 0.6352 - val_accuracy: 0.6868
Epoch 3/500
169/169 - 1s - loss: 0.6409 - accuracy: 0.6766 - val_loss: 0.6330 - val_accuracy: 0.6868
Epoch 4/500
169/169 - 1s - loss: 0.6393 - accuracy: 0.6768 - val_loss: 0.6311 - val_accuracy: 0.6868
Epoch 5/500
169/169 - 1s - loss: 0.6387 - accuracy: 0.6768 - val_loss: 0.6291 - val_accuracy: 0.6868
Epoch 6/500
169/169 - 1s - loss: 0.6336 - accuracy: 0.6768 - val_loss: 0.6263 - val_accuracy: 0.6868
Epoch 7/500
169/169 - 1s - loss: 0.6331 - accuracy: 0.6766 - val_loss: 0.6238 - val_accuracy: 0.6868
Epoch 8/500
169/169 - 1s - loss: 0.6288 - accuracy: 0.6768 - val_loss: 0.6209 - val_accuracy: 0.6868
Epoch 9/500
169/169 - 1s - loss: 0.6245 - accuracy: 0.6768 - val_loss: 0.6176 - val_accuracy: 0.6868
Epoch 10/500
169/169 - 1s - loss: 0.6206 - accuracy: 0.6768 - val_loss: 0.6144 - val_accura

Epoch 82/500
169/169 - 1s - loss: 0.4429 - accuracy: 0.7827 - val_loss: 0.4276 - val_accuracy: 0.8125
Epoch 83/500
169/169 - 1s - loss: 0.4428 - accuracy: 0.7825 - val_loss: 0.4253 - val_accuracy: 0.8118
Epoch 84/500
169/169 - 1s - loss: 0.4396 - accuracy: 0.7833 - val_loss: 0.4239 - val_accuracy: 0.8118
Epoch 85/500
169/169 - 1s - loss: 0.4417 - accuracy: 0.7870 - val_loss: 0.4233 - val_accuracy: 0.8155
Epoch 86/500
169/169 - 1s - loss: 0.4355 - accuracy: 0.7861 - val_loss: 0.4203 - val_accuracy: 0.8132
Epoch 87/500
169/169 - 1s - loss: 0.4388 - accuracy: 0.7936 - val_loss: 0.4182 - val_accuracy: 0.8147
Epoch 88/500
169/169 - 1s - loss: 0.4343 - accuracy: 0.7907 - val_loss: 0.4162 - val_accuracy: 0.8147
Epoch 89/500
169/169 - 1s - loss: 0.4327 - accuracy: 0.7929 - val_loss: 0.4151 - val_accuracy: 0.8170
Epoch 90/500
169/169 - 1s - loss: 0.4305 - accuracy: 0.7920 - val_loss: 0.4149 - val_accuracy: 0.8192
Epoch 91/500
169/169 - 1s - loss: 0.4301 - accuracy: 0.7922 - val_loss: 0.4136 - v

Epoch 162/500
169/169 - 1s - loss: 0.3692 - accuracy: 0.8369 - val_loss: 0.3441 - val_accuracy: 0.8594
Epoch 163/500
169/169 - 1s - loss: 0.3630 - accuracy: 0.8401 - val_loss: 0.3430 - val_accuracy: 0.8601
Epoch 164/500
169/169 - 1s - loss: 0.3654 - accuracy: 0.8364 - val_loss: 0.3436 - val_accuracy: 0.8601
Epoch 165/500
169/169 - 1s - loss: 0.3631 - accuracy: 0.8338 - val_loss: 0.3426 - val_accuracy: 0.8609
Epoch 166/500
169/169 - 1s - loss: 0.3641 - accuracy: 0.8369 - val_loss: 0.3397 - val_accuracy: 0.8624
Epoch 167/500
169/169 - 1s - loss: 0.3636 - accuracy: 0.8412 - val_loss: 0.3404 - val_accuracy: 0.8601
Epoch 168/500
169/169 - 1s - loss: 0.3599 - accuracy: 0.8443 - val_loss: 0.3397 - val_accuracy: 0.8616
Epoch 169/500
169/169 - 1s - loss: 0.3632 - accuracy: 0.8401 - val_loss: 0.3373 - val_accuracy: 0.8624
Epoch 170/500
169/169 - 1s - loss: 0.3619 - accuracy: 0.8373 - val_loss: 0.3420 - val_accuracy: 0.8609
Epoch 171/500
169/169 - 1s - loss: 0.3645 - accuracy: 0.8308 - val_loss: 

Epoch 242/500
169/169 - 1s - loss: 0.3200 - accuracy: 0.8591 - val_loss: 0.2962 - val_accuracy: 0.8810
Epoch 243/500
169/169 - 1s - loss: 0.3203 - accuracy: 0.8608 - val_loss: 0.2979 - val_accuracy: 0.8787
Epoch 244/500
169/169 - 1s - loss: 0.3219 - accuracy: 0.8587 - val_loss: 0.2964 - val_accuracy: 0.8810
Epoch 245/500
169/169 - 1s - loss: 0.3236 - accuracy: 0.8585 - val_loss: 0.2954 - val_accuracy: 0.8817
Epoch 246/500
169/169 - 1s - loss: 0.3236 - accuracy: 0.8569 - val_loss: 0.2945 - val_accuracy: 0.8824
Epoch 247/500
169/169 - 1s - loss: 0.3244 - accuracy: 0.8589 - val_loss: 0.2952 - val_accuracy: 0.8817
Epoch 248/500
169/169 - 1s - loss: 0.3116 - accuracy: 0.8608 - val_loss: 0.2969 - val_accuracy: 0.8839
Epoch 249/500
169/169 - 1s - loss: 0.3237 - accuracy: 0.8561 - val_loss: 0.2924 - val_accuracy: 0.8854
Epoch 250/500
169/169 - 1s - loss: 0.3192 - accuracy: 0.8593 - val_loss: 0.2914 - val_accuracy: 0.8824
Epoch 251/500
169/169 - 1s - loss: 0.3187 - accuracy: 0.8634 - val_loss: 

Epoch 322/500
169/169 - 1s - loss: 0.2816 - accuracy: 0.8778 - val_loss: 0.2575 - val_accuracy: 0.8958
Epoch 323/500
169/169 - 1s - loss: 0.2838 - accuracy: 0.8756 - val_loss: 0.2571 - val_accuracy: 0.8936
Epoch 324/500
169/169 - 1s - loss: 0.2858 - accuracy: 0.8811 - val_loss: 0.2555 - val_accuracy: 0.8966
Epoch 325/500
169/169 - 1s - loss: 0.2822 - accuracy: 0.8818 - val_loss: 0.2579 - val_accuracy: 0.8951
Epoch 326/500
169/169 - 1s - loss: 0.2785 - accuracy: 0.8781 - val_loss: 0.2566 - val_accuracy: 0.8929
Epoch 327/500
169/169 - 1s - loss: 0.2854 - accuracy: 0.8780 - val_loss: 0.2562 - val_accuracy: 0.8936
Epoch 328/500
169/169 - 1s - loss: 0.2799 - accuracy: 0.8800 - val_loss: 0.2516 - val_accuracy: 0.8996
Epoch 329/500
169/169 - 1s - loss: 0.2761 - accuracy: 0.8826 - val_loss: 0.2518 - val_accuracy: 0.8973
Epoch 330/500
169/169 - 1s - loss: 0.2805 - accuracy: 0.8800 - val_loss: 0.2520 - val_accuracy: 0.8981
Epoch 331/500
169/169 - 1s - loss: 0.2840 - accuracy: 0.8761 - val_loss: 

Epoch 402/500
169/169 - 1s - loss: 0.2588 - accuracy: 0.8905 - val_loss: 0.2241 - val_accuracy: 0.9159
Epoch 403/500
169/169 - 1s - loss: 0.2563 - accuracy: 0.8940 - val_loss: 0.2265 - val_accuracy: 0.9129
Epoch 404/500
169/169 - 1s - loss: 0.2559 - accuracy: 0.8952 - val_loss: 0.2232 - val_accuracy: 0.9167
Epoch 405/500
169/169 - 1s - loss: 0.2557 - accuracy: 0.8933 - val_loss: 0.2214 - val_accuracy: 0.9152
Epoch 406/500
169/169 - 1s - loss: 0.2552 - accuracy: 0.8926 - val_loss: 0.2204 - val_accuracy: 0.9159
Epoch 407/500
169/169 - 1s - loss: 0.2571 - accuracy: 0.8900 - val_loss: 0.2212 - val_accuracy: 0.9159
Epoch 408/500
169/169 - 1s - loss: 0.2528 - accuracy: 0.8981 - val_loss: 0.2206 - val_accuracy: 0.9174
Epoch 409/500
169/169 - 1s - loss: 0.2553 - accuracy: 0.8939 - val_loss: 0.2235 - val_accuracy: 0.9144
Epoch 410/500
169/169 - 1s - loss: 0.2521 - accuracy: 0.8983 - val_loss: 0.2180 - val_accuracy: 0.9144
Epoch 411/500
169/169 - 1s - loss: 0.2540 - accuracy: 0.8953 - val_loss: 

Epoch 482/500
169/169 - 1s - loss: 0.2310 - accuracy: 0.9027 - val_loss: 0.1971 - val_accuracy: 0.9196
Epoch 483/500
169/169 - 1s - loss: 0.2339 - accuracy: 0.9068 - val_loss: 0.2013 - val_accuracy: 0.9196
Epoch 484/500
169/169 - 1s - loss: 0.2294 - accuracy: 0.9122 - val_loss: 0.2005 - val_accuracy: 0.9211
Epoch 485/500
169/169 - 1s - loss: 0.2293 - accuracy: 0.9050 - val_loss: 0.1998 - val_accuracy: 0.9234
Epoch 486/500
169/169 - 1s - loss: 0.2350 - accuracy: 0.9009 - val_loss: 0.2017 - val_accuracy: 0.9189
Epoch 487/500
169/169 - 1s - loss: 0.2326 - accuracy: 0.9018 - val_loss: 0.1986 - val_accuracy: 0.9241
Epoch 488/500
169/169 - 1s - loss: 0.2354 - accuracy: 0.9016 - val_loss: 0.1970 - val_accuracy: 0.9234
Epoch 489/500
169/169 - 1s - loss: 0.2252 - accuracy: 0.9075 - val_loss: 0.1982 - val_accuracy: 0.9241
Epoch 490/500
169/169 - 1s - loss: 0.2297 - accuracy: 0.9029 - val_loss: 0.2001 - val_accuracy: 0.9196
Epoch 491/500
169/169 - 1s - loss: 0.2314 - accuracy: 0.9033 - val_loss: 

In [7]:
# Save the model to disk
model.save('saved_model')

In [29]:
#выдать predict 
from tensorflow.keras.models import load_model
#from 'saved_model' import *
model = load_model(r"C:\Users\Alexe\Audioclassify\saved_model")
#датасету
results = model.evaluate(X_test)
#отдельному звуку
#results = model.predict(file)

43/43 [==============================] - 0s 789us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [30]:
import tensorflow as tf
from tensorflow.keras.models import save_model, Sequential
import os
#os.chdir(r"C:\Users\Alexe\Audioclassify\saved_model")

model_path = r"C:\Users\Alexe\Audioclassify\saved_model"

model = tf.keras.models.load_model(model_path)

save_model(model,model_path + r"\new_model.h5", save_format='h5')

In [41]:
from scipy.io.wavfile import read
import os

path = r"C:\Users\Alexe\OneDrive\Рабочий стол\44.1K"
assert os.path.isfile(path)
with open(path, "r") as f:
    a = read(path)
    numpy.array(a[0],dtype=float)
    pass


AssertionError: 

AssertionError: 

In [45]:
from numpy import asarray
from numpy import save
# define data
data = asarray([[78, 118, 141, 141, 126, 112, 107, 108, 104, 89, 67, 48, 39, 38, 38, 31, 22, 16, 16, 21, 27, 30, 33, 39, 50, 63, 73, 82, 92, 105, 123, 139, 152, 159, 162, 161, 156, 148, 138, 129, 123, 116, 107, 96, 83, 72, 63, 54, 44, 33, 24, 19, 18, 18, 15, 13, 13, 19, 27, 35, 40, 42, 45, 50, 59, 69, 79, 88, 96, 103, 110, 116, 120, 121, 120, 117, 112, 106, 99, 90, 78, 65, 49, 33, 17, 5, -3, -8, -11, -13, -15, -19, -23, -24, -22, -17, -11, -6, -3, 0, 4, 14, 27, 43, 58, 71, 78, 83, 86, 91, 97, 102, 102, 94, 82, 69, 60, 54, 47, 36, 19, 1, -13, -24, -33, -39, -45, -49, -52, -55, -60, -64, -63, -54, -40, -26, -16, -11, -7, 0, 12, 26, 38, 46, 51, 56, 62, 67, 70, 70, 66, 60, 50, 37, 24, 13, 4, -4, -15, -28, -38, -43, -41, -36, -33, -35, -39, -41, -40, -36, -31, -25, -18, -7, 3, 13, 21, 28, 35, 43, 49, 50, 45, 40, 38, 42, 47, 51, 49, 43, 36, 30, 23, 14, 1, -14, -30, -43, -51, -55, -57, -59, -60, -61, -60, -59, -56, -53, -48, -43, -35, -27, -19, -11, -5, 2, 11, 21, 32, 41, 47, 50, 50, 46, 41, 36, 32, 31, 30, 27, 19, 6, -8, -22, -33, -40, -45, -49, -52, -53, -51, -50, -48, -48, -48, -47, -42, -33, -21, -9, -1, 2, 4, 8, 19, 34, 48, 58, 60, 59, 57, 56, 53, 46, 37, 31, 29, 29, 27, 19, 5, -7, -16, -20, -23, -27, -32, -33, -29, -23, -17, -14, -11, -8, -3, 0, 3, 6, 14, 28, 46, 61, 70, 74, 75, 77, 80, 80, 78, 75, 73, 71, 69, 63, 56, 51, 49, 47, 42, 31, 19, 11, 7, 6, 4, 0, -5, -9, -9, -7, -3, 1, 8, 15, 21, 22, 20, 20, 24, 33, 42, 48, 52, 55, 62, 69, 73, 71, 66, 60, 58, 57, 56, 52, 45, 38, 31, 25, 17, 8, -1, -12, -23, -33, -39, -40, -37, -31, -27, -26, -27, -28, -26, -23, -21, -21, -21, -18, -12, -7, -4, -5, -6, -4, 0, 2, 1, -4, -12, -21, -28, -31, -33, -35, -36, -41, -48, -58, -66, -70, -70, -69, -71, -77, -83, -86, -84, -79, -72, -67, -64, -61, -58, -56, -52, -47, -40, -32, -26, -23, -23, -24, -23, -23, -25, -32, -42, -50, -54, -55, -56, -60, -66, -73, -80, -85, -89, -92, -94, -93, -92, -92, -92, -93, -91, -88, -83, -78, -74, -69, -62, -55, -49, -43, -39, -34, -29, -25, -23, -24, -26, -27, -26, -27, -29, -34, -38, -42, -44, -48, -53, -59, -66, -69, -69, -67, -64, -63, -64, -67, -70, -71, -71, -69, -65, -59, -52, -43, -32, -22, -15, -13, -15, -17, -17, -16, -14, -13, -15, -16, -16, -15, -15, -18, -24, -30, -35, -37, -38, -39, -42, -47, -53, -58, -59, -55, -49, -44, -41, -40, -37, -30, -19, -7, 0, 1, -2, -4, 0, 6, 15, 20, 21, 21, 22, 24, 23, 18, 8, -1, -8, -11, -12, -13, -17, -22, -26, -28, -29, -28, -29, -31, -31, -30, -25, -20, -16, -15, -15, -15, -11, -3, 6, 13, 17, 18, 18, 18, 19, 21, 24, 25, 26, 24, 22, 19, 20, 25, 31, 35, 35, 30, 24, 20, 20, 23, 26, 28, 28, 27, 27, 25, 24, 23, 24, 27, 32, 36, 40, 44, 48, 53, 57, 57, 56, 55, 55, 57, 57, 56, 52, 47, 44, 44, 44, 43, 39, 35, 32, 32, 34, 34, 32, 28, 23, 19, 17, 16, 17, 20, 25, 30, 35, 38, 39, 38, 39, 39, 40, 39, 38, 38, 37, 37, 36, 35, 34, 35, 40, 44, 46, 43, 38, 34, 33, 35, 34, 28, 19, 12, 12, 17, 22, 25, 23, 20, 21, 24, 28, 30, 28, 27, 27, 31, 35, 36, 34, 31, 27, 26, 27, 28, 30, 31, 32, 32, 33, 33, 34, 32, 29, 23, 18, 13, 11, 9, 8, 7, 6, 6, 5, 2, -1, -5, -7, -6, -3, -2, -2, -1, 2, 8, 13, 14, 10, 2, -4, -9, -13, -15, -17, -17, -14, -11, -11, -12, -15, -17, -16, -15, -16, -20, -24, -27, -28, -30, -33, -37, -41, -44, -45, -46, -45, -42, -37, -31, -28, -30, -34, -37, -35, -30, -25, -24, -25, -26, -25, -22, -22, -25, -30, -35, -38, -40, -43, -48, -52, -53, -52, -52, -55, -60, -65, -68, -67, -64, -63, -62, -62, -59, -55, -49, -43, -39, -36, -35, -34, -33, -31, -30, -29, -31, -33, -35, -35, -34, -35, -37, -38, -38, -35, -33, -34, -40, -48, -56, -59, -58, -56, -56, -55, -54, -50, -45, -42, -40, -41, -40, -37, -35, -34, -35, -36, -34, -28, -19, -12, -8, -9, -13, -18, -21, -19, -14, -8, -4, -2, -4, -6, -8, -9, -9, -9, -10, -12, -13, -16, -20, -25, -29, -29, -26, -24, -24, -28, -32, -32, -27, -19, -11, -7, -6, -5, -2, 4, 10, 13, 13, 11, 10, 13, 17, 21, 23, 21, 20, 20, 23, 27, 29, 29, 28, 28, 29, 30, 29, 27, 24, 24, 25, 27, 30, 32, 36, 40, 45, 50, 54, 57, 58, 56, 53, 51, 51, 56, 64, 71, 74, 72, 69, 66, 64, 62, 58, 54, 50, 51, 53, 56, 56, 55, 54, 54, 55, 52, 48, 44, 45, 52, 63, 72, 77, 77, 75, 74, 74, 74, 76, 78, 82, 88, 92, 95, 96, 94, 92, 90, 89, 88, 89, 90, 89, 87, 84, 82, 79, 77, 74, 72, 71, 75, 83, 92, 97, 97, 93, 89, 87, 89, 94, 98, 100, 101, 99, 98, 99, 102, 107, 111, 111, 106, 99, 94, 95, 99, 103, 105, 104, 102, 99, 96, 90, 83, 77, 76, 78, 82, 85, 85, 85, 86, 89, 90, 90, 88, 86, 85, 86, 88, 89, 90, 91, 92, 90, 86, 80, 76, 74, 75, 75, 75, 72, 68, 64, 61, 59, 58, 59, 59, 59, 57, 54, 52, 53, 56, 59, 59, 57, 53, 50, 49, 48, 50, 52, 55, 58, 60, 58, 53, 46, 40, 36, 32, 29, 29, 31, 36, 41, 46, 48, 48, 45, 39, 31, 23, 18, 18, 23, 30, 33, 31, 26, 20, 17, 18, 21, 25, 30, 34, 36, 36, 32, 27, 22, 19, 18, 19, 20, 22, 23, 22, 17, 10, 2, -2, -1, 2, 6, 8, 8, 7, 7, 6, 7, 8, 10, 12, 13, 13, 9, 6, 3, 2, 4, 8, 13, 19, 26, 30, 33, 33, 31, 30, 29, 28, 27, 27, 27, 29, 33, 38, 41, 44, 45, 45, 44, 43, 42, 42, 42, 42, 42, 40, 38, 36, 32, 29, 26, 25, 27, 31, 33, 33, 31, 29, 29, 31, 33, 34, 34, 36, 40, 45, 48, 49, 47, 47, 50, 54, 57, 56, 54, 53, 55, 60, 64, 67, 66, 64, 63, 63, 64, 63, 61, 60, 62, 66, 71, 75, 76, 74, 70, 67, 65, 64, 66, 69, 73, 75, 73, 68, 62, 57, 57, 61, 64, 64, 61, 58, 58, 62, 68, 74, 76, 75, 74, 72, 69, 65, 59, 57, 58, 63, 67, 68, 66, 61, 56, 52, 47, 43, 43, 47, 54, 61, 65, 66, 67, 70, 73, 72, 67, 60, 56, 59, 66, 69, 65, 56, 49, 45, 46, 45, 41, 34, 31, 33, 40, 46, 47, 43, 36, 32, 31, 32, 34, 36, 38, 39, 39, 38, 38, 40, 42, 43, 43, 40, 38, 36, 37, 38, 37, 35, 33, 32, 31, 28, 22, 14, 8, 6, 10, 15, 19, 19, 15, 10, 7, 7, 11, 15, 17, 17, 15, 13, 10, 8, 6, 3, 1, 2, 4, 5, 5, 4, 2, 1, 1, 1, 0, -2, -7, -13, -17, -20, -22, -24, -25, -26, -27, -27, -29, -32, -36, -41, -43, -42, -39, -36, -35, -37, -38, -38, -35, -31, -28, -29, -33, -37, -41, -43, -45, -48, -50, -52, -53, -51, -48, -45, -44, -45, -48, -52, -56, -60, -61, -61, -59, -58, -57, -57, -56, -55, -55, -55, -54, -53, -52, -54, -56, -59, -60, -59, -57, -58, -60, -63, -63, -57, -50, -44, -42, -43, -45, -44, -41, -38, -38, -40, -45, -49, -52, -53, -53, -52, -51, -51, -53, -55, -58, -61, -63, -63, -64, -65, -68, -70, -72, -71, -69, -65, -62, -61, -61, -63, -63, -62, -61, -61, -63, -64, -63, -59, -55, -53, -55, -58, -62, -65, -67, -71, -74, -76, -74, -67, -59, -56, -59, -68, -79, -87, -89, -87, -83, -79, -78, -80, -84, -89, -91, -93, -93, -92, -91, -87, -82, -78, -77, -82, -90, -96, -99, -97, -95, -95, -98, -102, -103, -104, -105, -109, -114, -117, -117, -115, -115, -118, -122, -125, -123, -118, -114, -113, -115, -116, -115, -113, -110, -110, -109, -106, -100, -91, -85, -82, -84, -88, -92, -93, -93, -92, -90, -90, -92, -94, -94, -93, -89, -84, -82, -83, -89, -95, -98, -98, -95, -92, -90, -89, -85, -81, -77, -76, -77, -79, -80, -78, -76, -76, -77, -77, -75, -70, -65, -63, -63, -65, -67, -67, -69, -73, -76, -79, -78, -77, -77, -79, -82, -84, -83, -84, -86, -91, -95, -97, -95, -92, -88, -86, -83, -80, -77, -78, -83, -86, -87, -84, -81, -81, -86, -91, -93, -89, -86, -85, -87, -90, -90, -87, -84, -84, -87, -88, -88, -87, -90, -98, -107, -113, -112, -107, -102, -101, -102, -104, -104, -105, -106, -108, -110, -108, -103, -99, -97, -96, -94, -90, -87, -88, -94, -101, -105, -104, -99, -92, -87, -84, -83, -84, -87, -94, -101, -106, -107, -105, -103, -102, -104, -105, -105, -103, -101, -99, -101, -103, -105, -103, -97, -89, -84, -82, -83, -85, -85, -84, -82, -82, -82, -83, -81, -79, -78, -79, -84, -89, -92, -95, -98, -100, -99, -95, -88, -84, -84, -91, -99, -104, -103, -98, -95, -96, -99, -102, -104, -103, -103, -102, -101, -99, -96, -94, -94, -93, -92, -91, -90, -93, -99, -104, -107, -105, -101, -97, -94, -94, -95, -97, -99, -101, -104, -109, -113, -117, -117, -114, -107, -101, -98, -99, -103, -105, -105, -102, -99, -96, -94, -89, -83, -76, -73, -75, -79, -83, -83, -81, -79, -78, -80, -81, -81, -80, -78, -78, -79, -81, -82, -81, -78, -74, -71, -69, -68, -67, -68, -69, -70, -69, -68, -66, -63, -59, -55, -50, -45, -41, -38, -36, -34, -33, -32, -32, -33, -32, -31, -28, -25, -23, -23, -23, -24, -24, -23, -20, -15, -11, -6, 0, 4, 8, 9, 8, 5, 1, -2, -4, -2, 3, 14, 26, 35, 39, 36, 31, 26, 24, 23, 22, 20, 19, 20, 24, 30, 33, 35, 34, 34, 33, 32, 31, 29, 27, 28, 31, 37, 43, 49, 53, 56, 55, 52, 48, 45, 44, 45, 50, 56, 62, 67, 68, 65, 59, 54, 49, 47, 47, 48, 49, 52, 56, 61, 65, 66, 64, 61, 59, 60, 62, 64, 65, 65, 66, 69, 73, 75, 74, 69, 63, 58, 57, 60, 64, 66, 65, 61, 58, 58, 61, 64, 67, 66, 64, 61, 60, 58, 56, 54, 54, 56, 58, 59, 58, 54, 49, 45, 42, 40, 39, 39, 42, 48, 55, 60, 63, 63, 63, 65, 68, 70, 69, 65, 61, 60, 62, 67, 70, 70, 67, 64, 63, 64, 65, 65, 63, 61, 62, 66, 70, 73, 72, 68, 63, 58, 54, 52, 53, 55, 58, 60, 61, 62, 64, 68, 70, 70, 65, 59, 57, 60, 69, 78, 83, 85, 86, 87, 86, 83, 77, 72, 69, 70, 73, 76, 80, 87, 96, 104, 107, 101, 89, 77, 70, 67, 67, 68, 70, 74, 78, 81, 80, 78, 79, 83, 90, 93, 90, 81, 74, 75, 83, 94, 100, 101, 97, 93, 91, 90, 91, 93, 96, 98, 97, 95, 90, 87, 87, 88, 89, 89, 88, 90, 93, 96, 98, 99, 100, 102, 105, 107, 107, 105, 104, 106, 109, 111, 108, 101, 95, 92, 96, 104, 113, 120, 121, 116, 108, 99, 94, 93, 95, 97, 96, 92, 88, 86, 86, 88, 89, 88, 87, 84, 80, 75, 71, 67, 63, 60, 57, 55, 56, 58, 60, 62, 64, 67, 71, 76, 78, 75, 70, 67, 69, 75, 81, 86, 89, 91, 93, 95, 92, 85, 77, 72, 71, 74, 77, 75, 71, 67, 65, 64, 64, 63, 62, 62, 63, 63, 61, 58, 56, 56, 57, 57, 55, 51, 49, 50, 52, 53, 52, 49, 48, 50, 54, 60, 63, 63, 61, 58, 55, 55, 56, 59, 63, 65, 66, 65, 63, 59, 54, 48, 45, 46, 52, 57, 58, 55, 50, 49, 53, 61, 65, 65, 59, 53, 50, 53, 58, 63, 64, 63, 60, 56, 52, 47, 43, 41, 40, 39, 36, 34, 32, 34, 37, 39, 38, 34, 28, 23, 22, 25, 28, 29, 28, 24, 21, 19, 21, 23, 23, 21, 17, 14, 13, 15, 16, 13, 7, -1, -8, -12, -12, -13, -17, -22, -26, -26, -22, -18, -17, -19, -22, -24, -22, -22, -25, -30, -35, -35, -29, -23, -20, -22, -26, -27, -26, -24, -24, -27, -31, -32, -30, -28, -28, -33, -38, -42, -44, -44, -47, -50, -53, -52, -51, -51, -54, -59, -63, -65, -63, -61, -59, -58, -57, -55, -55, -57, -59, -60, -58, -52, -47, -43, -42, -42, -43, -43, -45, -47, -48, -48, -46, -45, -44, -45, -45, -43, -40, -39, -40, -42, -41, -37, -31, -28, -29, -31, -31, -27, -21, -16, -16, -21, -26, -30, -30, -28, -24, -21, -19, -19, -21, -24, -26, -24, -18, -12, -8, -8, -10, -13, -13, -13, -14, -15, -17, -18, -18, -20, -24, -30, -35, -35, -31, -25, -23, -24, -28, -32, -35, -35, -35, -35, -35, -35, -33, -29, -27, -26, -27, -31, -36, -39, -41, -40, -37, -34, -32, -31, -31, -29, -26, -23, -22, -23, -27, -31, -34, -35, -34, -33, -32, -28, -24, -20, -18, -20, -23, -26, -27, -28, -28, -29, -31, -32, -31, -29, -26, -25, -28, -31, -35, -37, -37, -37, -36, -36, -36, -36, -35, -35, -35, -36, -38, -41, -44, -46, -47, -45, -41, -36, -33, -35, -41, -48, -51, -49, -42, -35, -34, -38, -45, -49, -50, -49, -51, -56, -62, -64, -61, -56, -53, -53, -55, -55, -52, -48, -49, -56, -66, -73, -75, -73, -71, -72, -74, -74, -70, -64, -58, -56, -56, -57, -58, -59, -61, -63, -64, -63, -59, -56, -53, -52, -50, -48, -46, -47, -52, -57, -61, -61, -58, -53, -51, -51, -52, -52, -51, -50, -51, -54, -57, -58, -56, -52, -47, -44, -45, -48, -52, -55, -55, -52, -47, -42, -39, -38, -39, -41, -40, -39, -39, -41, -46, -50, -52, -51, -48, -46, -47, -49, -50, -47, -41, -34, -31, -33, -38, -44, -47, -47, -45, -43, -43, -46, -50, -55, -58, -57, -54, -52, -52, -54, -54, -50, -44, -38, -36, -38, -43, -48, -52, -55, -58, -61, -64, -64, -61, -57, -53, -52, -54, -58, -63, -66, -66, -65, -63, -61, -60, -61, -61, -62, -62, -62, -64, -65, -65, -64, -62, -61, -60, -59, -56, -52, -48, -48, -51, -56, -59, -61, -62, -64, -66, -66, -62, -57, -52, -50, -51, -53, -54, -55, -55, -57, -58, -58, -57, -55, -55, -57, -59, -59, -58, -55, -53, -51, -48, -45, -43, -43, -45, -46, -44, -40, -36, -35, -36, -37, -34, -29, -24, -22, -22, -22, -19, -17, -16, -18, -20, -19, -15, -11, -10, -12, -17, -19, -17, -14, -12, -14, -17, -19, -18, -14, -11, -9, -7, -2, 5, 13, 21, 23, 23, 21, 22, 27, 36, 46, 54, 57, 57, 55, 52, 49, 47, 44, 44, 45, 47, 48, 47, 44, 40, 40, 44, 50, 56, 61, 62, 60, 57, 55, 55, 57, 61, 63, 64, 62, 62, 65, 72, 79, 81, 77, 71, 67, 67, 70, 71, 70, 68, 68, 71, 75, 77, 77, 75, 75, 76, 78, 80, 80, 81, 83, 85, 85, 84, 82, 84, 87, 90, 90, 85, 81, 81, 85, 91, 95, 97, 99, 103, 109, 113, 113, 111, 109, 108, 109, 108, 105, 101, 99, 99, 100, 99, 96, 93, 90, 89, 88, 87, 87, 89, 95, 102, 106, 108, 106, 105, 106, 109, 112, 118, 124, 131, 137, 138, 136, 133, 131, 132, 135, 136, 135, 132, 131, 131, 132, 131, 128, 123, 118, 114, 112, 111, 112, 114, 116, 118, 119, 117, 114, 112, 111, 112, 113, 115, 116, 116, 113, 110, 107, 107, 110, 113, 113, 110, 105, 103, 103, 105, 106, 103, 100, 98, 100, 102, 100, 95, 87, 80, 76, 76, 76, 76, 76, 76, 76, 75, 71, 67, 64, 64, 66, 68, 67, 64, 60, 57, 53, 50, 48, 48, 52, 58, 63, 65, 63, 59, 55, 52, 51, 53, 57, 62, 64, 63, 58, 52, 49, 51, 55, 56, 52, 45, 41, 41, 45, 50, 49, 45, 39, 36, 36, 37, 36, 34, 34, 37, 42, 46, 45, 43, 43, 47, 53, 57, 57, 55, 54, 55, 59, 60, 58, 53, 51, 52, 55, 56, 53, 50, 49, 52, 58, 63, 65, 64, 64, 66, 69, 71, 69, 65, 62, 62, 63, 65, 63, 59, 55, 53, 53, 54, 53, 50, 46, 42, 39, 39, 40, 41, 42, 41, 38, 33, 28, 24, 22, 22, 23, 27, 32, 37, 41, 42, 40, 37, 34, 33, 33, 33, 31, 28, 26, 24, 23, 23, 23, 25, 26, 26, 24, 20, 15, 11, 9, 10, 10, 8, 5, 1, 0, 0, 4, 10, 17, 25, 30, 31, 29, 26, 25, 25, 25, 22, 18, 16, 17, 23, 29, 32, 32, 31, 32, 35, 39, 41, 41, 41, 43, 47, 51, 53, 53, 52, 52, 52, 51, 47, 43, 41, 42, 45, 47, 47, 45, 41, 39, 38, 39, 40, 41, 44, 47, 49, 49, 49, 48, 47, 46, 46, 45, 46, 48, 51, 55, 57, 57, 57, 57, 55, 54, 52, 50, 49, 48, 47, 47, 47, 47, 46, 43, 39, 36, 36, 37, 38, 38, 38, 38, 41, 44, 46, 43, 40, 38, 42, 48, 51, 49, 42, 35, 32, 35, 40, 43, 44, 45, 49, 55, 59, 58, 52, 45, 40, 40, 43, 48, 51, 52, 50, 46, 40, 34, 29, 28, 29, 32, 36, 39, 39, 37, 30, 22, 15, 11, 11, 13, 14, 13, 10, 8, 8, 9, 11, 11, 7, 0, -7, -11, -11, -8, -5, -5, -11, -19, -25, -26, -23, -17, -13, -11, -11, -11, -12, -12, -15, -18, -21, -23, -25, -25, -26, -25, -25, -24, -24, -24, -23, -20, -17, -14, -15, -17, -21, -25, -30, -35, -40, -41, -36, -28, -22, -21, -27, -35, -40, -41, -39, -38, -40, -42, -43, -41, -38, -38, -40, -42, -44, -43, -41, -40, -41, -41, -42, -43, -45, -46, -47, -44, -40, -36, -35, -37, -41, -45, -47, -47, -46, -44, -44, -49, -58, -68, -73, -71, -65, -60, -59, -62, -65, -64, -60, -58, -61, -69, -77, -80, -78, -73, -71, -71, -70, -67, -62, -59, -61, -66, -71, -74, -75, -74, -74, -76, -78, -80, -79, -78, -78, -80, -83, -87, -91, -92, -92, -91, -90, -89, -88, -87, -86, -87, -91, -96, -99, -97, -91, -84, -80, -78, -76, -74, -72, -73, -80, -89, -95, -96, -92, -89, -89, -92, -94, -92, -86, -82, -81, -85, -89, -91, -90, -87, -86, -86, -87, -88, -90, -91, -93, -93, -92, -90, -90, -91, -94, -96, -98, -97, -96, -93, -90, -86, -84, -85, -89, -93, -95, -94, -91, -88, -86, -86, -86, -85, -82, -78, -73, -70, -69, -71, -75, -78, -78, -76, -73, -71, -70, -71, -71, -72, -75, -79, -82, -85, -87, -89, -93, -98, -100, -97, -91, -85, -83, -85, -88, -88, -86, -83, -82, -85, -87, -86, -83, -79, -78, -81, -85, -88, -88, -88, -89, -91, -92, -91, -88, -87, -90, -95, -100, -103, -101, -98, -97, -98, -102, -105, -106, -103, -98, -94, -92, -93, -94, -96, -97, -99, -102, -106, -110, -112, -111, -107, -102, -97, -97, -101, -108, -116, -120, -121, -120, -117, -115, -112, -108, -105, -104, -108, -115, -119, -119, -113, -108, -107, -112, -117, -119, -117, -114, -114, -119, -126, -128, -125, -119, -113, -111, -113, -115, -114, -111, -109, -109, -111, -113, -112, -110, -107, -105, -105, -106, -108, -109, -111, -112, -114, -114, -114, -115, -116, -119, -120, -120, -117, -114, -112, -112, -113, -115, -114, -112, -111, -113, -117, -123, -127, -127, -124, -120, -117, -114, -110, -107, -105, -107, -111, -115, -117, -117, -116, -118, -121, -123, -123, -121, -119, -119, -119, -116, -110, -105, -103, -106, -110, -112, -111, -109, -109, -111, -113, -111, -105, -100, -97, -100, -105, -110, -113, -114, -114, -113, -110, -107, -107, -109, -111, -112, -109, -106, -106, -109, -112, -112, -107, -100, -94, -93, -94, -94, -91, -88, -88, -93, -100, -102, -98, -90, -81, -75, -73, -74, -75, -78, -81, -84, -85, -84, -82, -81, -80, -79, -77, -76, -76, -77, -79, -80, -77, -73, -71, -69, -68, -65, -60, -54, -49, -48, -49, -50, -49, -46, -44, -43, -43, -43, -40, -37, -34, -33, -33, -34, -33, -30, -27, -23, -20, -17, -15, -14, -14, -16, -19, -21, -20, -14, -6, 1, 6, 6, 4, 2, 1, 2, 4, 7, 10, 13, 15, 14, 11, 7, 6, 9, 15, 20, 23, 24, 25, 29, 35, 40, 43, 44, 45, 48, 53, 59, 63, 65, 65, 65, 66, 68, 69, 70, 69, 67, 64, 62, 60, 57, 53, 51, 52, 56, 61, 64, 63, 59, 55, 56, 62, 69, 72, 69, 61, 56, 56, 61, 67, 71, 70, 67, 65, 67, 70, 72, 72, 70, 69, 70, 72, 72, 71, 68, 65, 63, 62, 60, 58, 57, 57, 59, 61, 62, 60, 59, 60, 63, 67, 69, 66, 62, 59, 58, 60, 60, 58, 55, 53, 56, 62, 69, 73, 74, 73, 74, 76, 77, 76, 73, 70, 69, 72, 76, 80, 81, 78, 73, 67, 63, 61, 62, 65, 67, 69, 70, 73, 76, 79, 78, 74, 67, 63, 63, 67, 71, 70, 64, 57, 52, 54, 59, 65, 67, 65, 61, 59, 60, 64, 69, 73, 73, 72, 71, 71, 73, 75, 76, 75, 73, 72, 74, 78, 82, 83, 80, 72, 63, 56, 54, 56, 60, 65, 68, 71, 73, 75, 76, 75, 74, 74, 76, 79, 80, 80, 78, 75, 75, 77, 82, 84, 83, 79, 74, 72, 72, 74, 76, 75, 74, 74, 75, 76, 77, 75, 74, 73, 75, 77, 78, 79, 80, 83, 87, 90, 91, 89, 87, 86, 87, 87, 86, 84, 83, 85, 88, 90, 88, 84, 82, 82, 83, 83, 80, 78, 78, 81, 87, 91, 91, 88, 83, 81, 81, 85, 89, 91, 91, 90, 88, 88, 91, 96, 101, 103, 101, 97, 93, 91, 91, 91, 89, 88, 88, 90, 93, 93, 89, 82, 77, 75, 77, 80, 83, 84, 83, 83, 82, 82, 80, 79, 79, 82, 85, 89, 93, 94, 93, 91, 88, 85, 83, 83, 84, 85, 86, 87, 86, 86, 84, 82, 80, 79, 78, 78, 77, 75, 75, 75, 77, 77, 77, 75, 75, 76, 78, 79, 77, 76, 76, 79, 83, 84, 81, 75, 68, 66, 66, 69, 73, 77, 82, 87, 90, 89, 86, 82, 80, 79, 77, 75, 72, 71, 72, 74, 75, 74, 70, 65, 61, 58, 55, 53, 53, 55, 57, 57, 56, 52, 50, 50, 52, 55, 58, 60, 62, 63, 63, 61, 58, 57, 57, 58, 60, 59, 58, 58, 61, 66, 72, 74, 73, 67, 62, 57, 55, 56, 58, 60, 64, 67, 70, 72, 72, 70, 66, 63, 61, 60, 61, 62, 62, 62, 61, 62, 63, 65, 63, 60, 54, 48, 43, 43, 45, 47, 49, 47, 43, 38, 33, 30, 28, 27, 26, 27, 29, 32, 34, 35, 34, 32, 31, 29, 27, 23, 18, 13, 11, 9, 9, 8, 7, 6, 5, 4, 3, 3, 3, 4, 4, 3, 2, 3, 8, 13, 16, 13, 4, -5, -11, -11, -8, -6, -8, -13, -17, -19, -17, -16, -17, -21, -25, -28, -29, -31, -32, -33, -31, -28, -25, -23, -24, -28, -33, -38, -41, -41, -39, -35, -31, -29, -30, -32, -37, -41, -45, -47, -47, -45, -42, -39, -39, -38, -39, -39, -41, -45, -50, -56, -58, -58, -55, -54, -54, -55, -55, -53, -52, -51, -52, -54, -54, -54, -52, -50, -48, -48, -48, -47, -47, -45, -43, -41, -39, -37, -35, -33, -30, -29, -31, -36, -43, -49, -51, -50, -48, -48, -50, -52, -53, -53, -52, -50, -46, -41, -36, -34, -35, -39, -42, -42, -41, -41, -44, -48, -51, -50, -46, -43, -43, -45, -49, -52, -53, -52, -50, -48, -46, -45, -47, -49, -50, -49, -48, -49, -52, -57, -60, -57, -51, -45, -44, -47, -53, -56, -55, -52, -49, -48, -50, -52, -52, -50, -47, -43, -41, -41, -43, -45, -46, -44, -42, -42, -44, -45, -45, -43, -41, -41, -44, -48, -49, -47, -46, -46, -49, -51, -47, -40, -35, -34, -39, -44, -47, -46, -44, -45, -49, -53, -54, -52, -49, -48, -50, -52, -53, -53, -54, -55, -54, -51, -45, -41, -42, -46, -52, -55, -54, -51, -49, -50, -50, -49, -46, -42, -40, -39, -37, -32, -25, -17, -13, -15, -20, -26, -29, -27, -22, -14, -7, -3, -1, -2, -3, -4, -5, -8, -12, -15, -17, -14, -9, -2, 2, 4, 1, -4, -11, -18, -20, -18, -11, -3, 1, 2, 1, -1, -3, -2, 0, 2, 5, 7, 8, 8, 7, 7, 7, 8, 9, 10, 9, 7, 5, 4, 4, 6, 8, 11, 13, 11, 8, 4, 2, 4, 7, 10, 10, 4, -2, -8, -10, -10, -9, -9, -9, -9, -8, -6, -5, -6, -9, -12, -13, -10, -6, -4, -4, -7, -9, -9, -7, -7, -10, -16, -21, -22, -17, -9, -3, -2, -5, -6, -4, 0, 4, 5, 3, 0, -1, -1, -1, -3, -7, -12, -15, -17, -18, -18, -18, -17, -17, -17, -16, -16, -15, -15, -17, -20, -24, -26, -26, -24, -22, -24, -27, -31, -31, -29, -27, -25, -24, -22, -20, -17, -16, -18, -21, -21, -19, -16, -14, -15, -16, -14, -9, -2, 0, 0, -4, -9, -11, -11, -10, -9, -9, -8, -6, -5, -7, -9, -12, -11, -8, -4, -2, -3, -7, -10, -11, -12, -11, -11, -10, -11, -11, -12, -12, -11, -9, -7, -7, -7, -8, -7, -4, 0, 2, 3, 3, 3, 5, 9, 12, 13, 11, 10, 11, 14, 17, 18, 16, 13, 12, 14, 16, 16, 14, 12, 13, 19, 27, 33, 35, 33, 28, 23, 19, 16, 15, 18, 23, 30, 35, 37, 36, 36, 39, 45, 51, 53, 50, 47, 45, 47, 50, 52, 54, 55, 57, 58, 59, 58, 58, 60, 63, 65, 64, 60, 56, 54, 55, 56, 55, 52, 50, 50, 52, 54, 54, 52, 49, 49, 49, 49, 47, 43, 39, 36, 35, 33, 30, 28, 28, 32, 38, 43, 45, 43, 39, 35, 34, 35, 38, 43, 48, 52, 54, 55, 54, 54, 54, 54, 53, 51, 48, 48, 49, 51, 53, 53, 52, 49, 47, 44, 43, 42, 42, 43, 43, 41, 36, 30, 25, 22, 23, 26, 30, 32, 34, 35, 36, 37, 36, 35, 34, 34, 34, 34, 32, 29, 29, 34, 43, 51, 54, 49, 40, 34, 35, 43, 51, 54, 51, 44, 39, 36, 35, 32, 28, 26, 26, 30, 33, 32, 26, 19, 14, 12, 15, 19, 22, 22, 19, 15, 10, 8, 10, 14, 17, 19, 19, 18, 19, 21, 20, 18, 13, 10, 11, 13, 14, 12, 9, 7, 9, 12, 14, 13, 8, 3, 3, 7, 12, 15, 12, 8, 4, 3, 5, 8, 9, 9, 8, 7, 8, 11, 14, 18, 19, 18, 15, 12, 12, 13, 17, 21, 23, 24, 23, 23, 24, 25, 27, 28, 27, 27, 27, 29, 32, 35, 36, 34, 31, 29, 31, 36, 41, 43, 41, 37, 33, 31, 29, 28, 27, 26, 27, 28, 29, 30, 30, 29, 28, 27, 25, 23, 20, 19, 20, 24, 28, 31, 32, 31, 27, 21, 14, 8, 3, 3, 5, 10, 15, 18, 20, 19, 17, 13, 8, 3, 0, -2, -1, 2, 8, 13, 14, 12, 7, 3, 1, 2, 4, 5, 3, 1, 0, 0, 0, -2, -5, -8, -10, -11, -13, -15, -19, -21, -22, -21, -20, -22, -25, -26, -26, -24, -22, -21, -23, -25, -26, -25, -23, -21, -22, -25, -31, -37, -41, -41, -36, -29, -21, -17, -17, -21, -27, -31, -33, -34, -35, -37, -40, -40, -36, -27, -18, -12, -12, -18, -25, -30, -33, -32, -31, -30, -29, -27, -22, -17, -14, -16, -20, -27, -33, -38, -42, -44, -45, -45, -45, -46, -48, -50, -51, -50, -49, -47, -46, -45, -43, -40, -35, -31, -27, -24, -22, -21, -21, -21, -21, -19, -19, -20, -23, -26, -27, -25, -21, -17, -17, -21, -26, -30, -31, -31, -32, -34, -36, -37, -35, -32, -30, -31, -37, -45, -50, -50, -45, -39, -36, -38, -43, -45, -44, -41, -39, -42, -49, -55, -58, -55, -51, -47, -46, -47, -47, -47, -47, -48, -49, -50, -50, -48, -47, -46, -46, -44, -41, -36, -31, -27, -24, -24, -26, -30, -34, -37, -38, -38, -37, -34, -31, -28, -28, -31, -38, -45, -50, -52, -53, -54, -57, -59, -60, -58, -57, -58, -61, -63, -60, -55, -50, -48, -50, -52, -51, -48, -46, -48, -53, -56, -53, -46, -39, -37, -41, -48, -53, -56, -55, -55, -56, -58, -58, -57, -55, -54, -55, -56, -57, -58, -58, -58, -59, -61, -61, -60, -57, -54, -53, -55, -59, -63, -64, -64, -63, -65, -70, -75, -79, -79, -76, -72, -71, -72, -75, -76, -76, -76, -76, -79, -84, -89, -92, -94, -93, -92, -90, -89, -87, -86, -85, -83, -81, -78, -74, -70, -68, -69, -72, -77, -80, -79, -73, -66, -61, -60, -62, -65, -67, -67, -65, -60, -55, -50, -49, -50, -53, -56, -55, -51, -46, -43, -43, -43, -43, -41, -38, -38, -39, -41, -42, -41, -38, -38, -40, -43, -45, -44, -41, -37, -36, -37, -38, -35, -30, -24, -19, -16, -16, -18, -21, -26, -32, -39, -44, -43, -37, -29, -20, -15, -15, -16, -19, -20, -20, -19, -18, -17, -18, -18, -16, -10, -2, 2, 2, -2, -7, -10, -9, -6, -7, -12, -17, -19, -16, -12, -12, -16, -22, -23, -18, -11, -7, -8, -11, -12, -10, -6, -5, -7, -12, -16, -18, -19, -21, -23, -24, -23, -19, -16, -14, -12, -11, -11, -12, -14, -15, -15, -12, -8, -6, -7, -9, -11, -13, -13, -14, -14, -15, -17, -20, -23, -26, -26, -23, -21, -22, -28, -34, -34, -28, -18, -11, -10, -13, -17, -20, -21, -23, -27, -29, -28, -25, -23, -25, -30, -36, -37, -36, -34, -34, -37, -40, -40, -38, -37, -37, -38, -40, -40, -41, -41, -41, -40, -38, -36, -37, -41, -46, -50, -51, -48, -45, -43, -44, -45, -46, -44, -38, -29, -21, -17, -20, -26, -31, -32, -29, -24, -21, -20, -21, -21, -22, -23, -25, -27, -27, -26, -24, -23, -23, -24, -24, -24, -24, -24, -22, -19, -14, -10, -9, -11, -14, -15, -14, -13, -13, -16, -20, -22, -22, -18, -12, -7, -4, -3, -3, -2, 1, 6, 12, 13, 11, 6, 2, 2, 4, 7, 9, 10, 11, 12, 12, 11, 8, 5, 4, 2, 0, -2, -3, -2, 0, 0, -3, -9, -12, -7, 1, 7, 8, 2, -2, -3, 1, 7, 11, 13, 15, 19, 25, 28, 26, 22, 18, 15, 13, 11, 11, 13, 16, 20, 21, 21, 20, 20, 20, 18, 14, 11, 10, 13, 18, 21, 21, 20, 20, 23, 27, 28, 26, 26, 27, 32, 36, 38, 38, 37, 35, 32, 28, 27, 29, 34, 38, 37, 33, 30, 32, 38, 41, 39, 33, 29, 30, 35, 39, 38, 36, 37, 44, 53, 58, 58, 56, 55, 56, 55, 49, 42, 40, 45, 56, 65, 68, 65, 61, 58, 54, 50, 49, 55, 67, 76, 74, 64, 54, 54, 64, 73, 70, 58, 51, 58, 73, 83, 76, 62, 59, 74, 94, 96, 76, 52, 52, 82, 111, 98, 36, -34, -50, 18, 143, 252, 287, 247, 177, 125, 104, 93, 66, 26, 0, 9, 43, 70, 66, 37, 13, 13, 33, 47, 42, 30, 35, 66, 105, 122, 102, 64, 42, 57, 100, 141, 156, 142, 119, 104, 98, 94, 83, 68, 56, 54, 56, 54, 45, 31, 20, 18, 24, 36, 52, 68, 78, 77, 65, 49, 42, 45, 53, 51, 34, 7, -10, -7, 12, 34, 43, 34, 19, 11, 16, 28, 40, 49, 62, 83, 110, 128, 125, 99, 65, 41, 38, 51, 65, 68, 61, 56, 61, 80, 100, 111, 109, 93, 69, 39, 6, -19, -26, -5, 32, 64, 67, 41, 12, 11, 49, 104, 136, 122, 68, 11, -17, -7, 21, 48, 62, 68, 73, 75, 65, 42, 14, 0, 5, 24, 41, 47, 42, 34, 30, 27, 22, 18, 19, 26, 33, 30, 13, -10, -29, -37, -37, -33, -27, -15, 0, 10, 6, -9, -26, -31, -25, -22, -38, -67, -86, -71, -22, 34, 63, 48, 1, -47, -77, -86, -84, -79, -69, -50, -29, -17, -19, -28, -29, -13, 7, 17, 3, -27, -52, -56, -37, -11, 9, 17, 19, 21, 26, 30, 33, 32, 24, 11, -5, -16, -10, 13, 48, 73, 75, 55, 30, 15, 17, 31, 46, 62, 81, 108, 136, 154, 150, 127, 100, 83, 85, 99, 114, 123, 125, 126, 125, 123, 115, 101, 87, 79, 81, 90, 99, 105, 108, 115, 131, 155, 181, 195, 191, 169, 140, 120, 115, 124, 138, 144, 139, 124, 109, 103, 105, 111, 114, 105, 84, 58, 35, 28, 41, 71, 106, 133, 140, 127, 106, 93, 99, 120, 144, 156, 147, 120, 86, 58, 46, 55, 83, 123, 162, 185, 188, 174, 152, 133, 119, 106, 93, 89, 107, 153, 216, 274, 299, 280, 221, 140, 60, 1, -24, -16, 21, 75, 129, 171, 199, 210, 202, 171, 116, 55, 11, 4, 34, 80, 120, 144, 161, 181, 199, 200, 170, 114, 56, 22, 18, 30, 42, 48, 55, 71, 92, 103, 93, 67, 41, 26, 23, 23, 19, 15, 18, 33, 56, 77, 89, 88, 74, 46, 9, -25, -44, -41, -21, 1, 15, 20, 24, 32, 43, 49, 46, 37, 32, 32, 35, 34, 28, 20, 14, 10, 4, 0, 2, 16, 37, 46, 32, -4, -44, -66, -61, -37, -9, 12, 26, 32, 31, 21, 5, -5, -6, 2, 10, 7, -7, -24, -33, -30, -18, -2, 12, 22, 23, 12, -9, -34, -54, -63, -64, -62, -57, -50, -38, -26, -21, -24, -33, -40, -44, -49, -57, -61, -55, -37, -24, -35, -72, -115, -134, -113, -68, -31, -27, -53, -89, -112, -116, -112, -110, -115, -122, -127, -129, -126, -115, -99, -84, -75, -72, -67, -58, -53, -70, -113, -165, -191, -177, -142, -125, -143, -172, -166, -110, -45, -37, -112, -218, -267, -210, -89, -6, -31, -143, -253, -279, -216, -130, -88, -106, -143, -157, -141, -122, -122, -132, -130, -107, -83, -86, -122, -170, -201, -207, -194, -174, -153, -131, -119, -132, -170, -210, -219, -188, -147, -142, -194, -281, -347, -350, -286, -181, -69, 18, 60, 36, -51, -162, -233, -222, -144, -76, -87, -185, -302, -345, -278, -139, -6, 61, 56, 6, -59, -122, -172, -192, -180, -158, -161, -200, -236, -207, -87, 77, 191, 184, 68, -80, -185, -230, -244, -249, -232, -166, -64, 11, 2, -86, -178, -192, -119, -34, -19, -85, -160, -163, -79, 19, 49, -9, -101, -153, -138, -94, -69, -71, -72, -48, -19, -28, -96, -191, -249, -230, -149, -62, -18, -25, -57, -87, -104, -108, -101, -79, -46, -15, -7, -24, -50, -66, -63, -54, -54, -65, -78, -83, -85, -98, -129, -162, -176, -156, -112, -64, -26, 2, 30, 57, 68, 48, -1, -60, -99, -101, -67, -17, 30, 62, 73, 65, 43, 14, -17, -49, -80, -105, -115, -98, -51, 12, 69, 95, 77, 24, -40, -90, -103, -73, -13, 53, 97, 106, 85, 53, 31, 22, 15, -2, -29, -53, -57, -39, -11, 12, 26, 33, 41, 46, 41, 21, -6, -30, -39, -37, -35, -40, -47, -43, -20, 15, 50, 68, 61, 36, 6, -17, -28, -25, -13, 0, 6, 6, 0, -2, 0, 9, 18, 22, 20, 15, 12, 12, 12, 9, 2, -7, -17, -26, -30, -27, -17, -2, 11, 17, 13, 2, -8, -15, -16, -16, -19, -21, -19, -8, 8, 27, 39, 45, 46, 47, 44, 31, 3, -34, -71, -91, -91, -78, -65, -59, -55, -46, -26, -4, 4, -8, -35, -59, -67, -57, -40, -30, -30, -37, -45, -54, -67, -83, -95, -95, -80, -59, -42, -33, -29, -23, -18, -22, -39, -64, -81, -78, -54, -26, -10, -10, -18, -21, -13, 0, 7, 4, -5, -17, -28, -41, -56, -67, -66, -49, -19, 11, 33, 42, 40, 29, 11, -14, -45, -72, -81, -68, -35, 5, 39, 56, 54, 37, 11, -14, -32, -39, -30, -8, 22, 52, 69, 67, 45, 8, -28, -51, -55, -43, -23, -7, 0, 0, -4, -8, -12, -17, -22, -27, -31, -32, -28, -20, -9, -1, 0, -6, -17, -26, -23, -9, 10, 26, 32, 27, 18, 9, 2, -2, -7, -13, -14, -6, 7, 22, 31, 29, 16, -1, -17, -28, -31, -29, -24, -17, -7, 2, 9, 10, 3, -7, -13, -8, 9, 33, 54, 64, 63, 54, 45, 39, 39, 43, 51, 58, 60, 56, 44, 28, 14, 5, 2, 5, 15, 31, 54, 76, 88, 82, 62, 37, 18, 9, 7, 6, 5, 7, 15, 27, 33, 28, 13, 0, -1, 9, 29, 48, 62, 68, 69, 66, 60, 56, 56, 61, 65, 61, 46, 27, 14, 13, 23, 36, 46, 53, 60, 71, 84, 94, 96, 89, 74, 53, 32, 18, 20, 36, 57, 70, 71, 66, 69, 86, 109, 121, 114, 96, 84, 85, 95, 102, 101, 101, 113, 133, 141, 120, 76, 38, 36, 72, 115, 129, 100, 54, 31, 51, 100, 142, 152, 134, 109, 98, 101, 106, 106, 100, 98, 100, 102, 95, 82, 69, 66, 73, 82, 86, 82, 73, 61, 46, 31, 20, 22, 34, 48, 53, 48, 43, 56, 90, 127, 140, 111, 49, -12, -39, -18, 35, 92, 122, 120, 98, 77, 71, 77, 79, 63, 27, -12, -33, -21, 20, 67, 89, 71, 21, -32, -63, -56, -19, 22, 48, 48, 28, 6, -4, -2, 5, 6, -3, -15, -18, -4, 24, 53, 69, 67, 49, 26, 3, -16, -32, -41, -38, -21, 3, 28, 45, 50, 45, 34, 17, -4, -25, -34, -24, 4, 38, 59, 55, 27, -12, -51, -81, -98, -102, -93, -72, -45, -19, -3, 0, -5, -12, -18, -25, -33, -41, -43, -35, -20, -4, 3, 1, -5, -11, -12, -12, -15, -23, -32, -36, -32, -19, -1, 16, 32, 41, 41, 28, 4, -25, -52, -70, -76, -69, -50, -24, 1, 17, 16, -3, -31, -56, -68, -65, -53, -39, -27, -17, -11, -11, -19, -33, -45, -49, -47, -42, -41, -44, -47, -46, -43, -40, -37, -33, -25, -17, -17, -29, -45, -52, -40, -15, 5, 10, -1, -17, -25, -28, -35, -51, -70, -76, -66, -48, -36, -37, -44, -43, -33, -25, -32, -53, -71, -70, -48, -21, -8, -9, -13, -7, 5, 10, 0, -20, -28, -14, 12, 28, 22, 0, -16, -18, -14, -19, -35, -49, -42, -15, 10, 15, -5, -30, -36, -21, -3, 0, -12, -20, -7, 21, 42, 38, 13, -6, -3, 19, 40, 39, 18, -2, -4, 12, 32, 42, 41, 35, 33, 29, 19, 1, -12, -16, -11, -7, -10, -13, -5, 19, 48, 61, 50, 25, 11, 20, 43, 55, 42, 8, -20, -26, -11, 4, 4, -11, -25, -20, 4, 35, 52, 47, 26, 4, -7, -8, 0, 13, 26, 36, 36, 24, 3, -18, -28, -24, -10, 1, 6, 2, -3, -5, -1, 2, 2, -1, -8, -12, -15, -17, -18, -14, -3, 13, 27, 26, 7, -18, -38, -41, -31, -19, -15, -16, -10, 7, 32, 46, 34, -1, -41, -65, -67, -54, -41, -36, -37, -37, -32, -26, -22, -22, -25, -30, -39, -55, -71, -77, -64, -35, -4, 11, 8, -6, -19, -26, -29, -33, -37, -33, -13, 14, 36, 38, 17, -12, -35, -47, -50, -56, -65, -71, -67, -51, -31, -15, -7, -5, -8, -15, -27, -41, -48, -39, -16, 10, 23, 16, -4, -23, -28, -22, -14, -11, -14, -17, -18, -21, -34, -52, -64, -55, -26, 6, 19, 5, -26, -49, -48, -27, -4, 4, 1, -1, 1, 5, -1, -20, -38, -38, -16, 9, 14, -11, -54, -87, -96, -85, -74, -76, -85, -82, -55, -8, 38, 65, 62, 34, -2, -34, -49, -46, -33, -26, -32, -48, -59, -49, -22, 1, 2, -21, -47, -51, -28, 1, 12, 0, -19, -22, -6, 7, 1, -20, -32, -13, 27, 55, 40, -11, -62, -73, -39, 7, 30, 17, -12, -33, -36, -34, -41, -53, -57, -44, -22, -7, -7, -13, -9, 5, 18, 16, 0, -15, -18, -7, 0, -3, -13, -16, -3, 14, 18, 2, -20, -30, -20, -1, 8, 5, 0, 9, 34, 60, 71, 58, 30, 0, -24, -42, -52, -49, -32, -8, 7, 1, -21, -45, -50, -32, -2, 24, 40, 45, 42, 30, 10, -11, -22, -12, 11, 33, 38, 24, 4, -6, -6, -2, -2, -5, -2, 10, 25, 32, 22, 5, -8, -12, -12, -18, -26, -25, -2, 37, 69, 69, 32, -20, -60, -67, -48, -21, -6, -4, -5, 0, 6, 5, -9, -33, -51, -52, -35, -14, 0, -4, -22, -45, -62, -65, -55, -39, -25, -19, -25, -42, -62, -77, -79, -67, -44, -22, -11, -15, -32, -50, -61, -63, -63, -69, -80, -85, -73, -42, -6, 16, 14, -5, -25, -27, -13, 3, 7, -6, -28, -46, -50, -42, -28, -11, 7, 26, 41, 43, 28, -1, -32, -55, -61, -53, -37, -21, -8, 0, 6, 11, 16, 16, 6, -13, -41, -65, -74, -66, -46, -26, -15, -10, -5, 2, 9, 2, -25, -68, -108, -127, -119, -93, -67, -54, -52, -50, -37, -13, 7, 11, -5, -32, -50, -48, -30, -13, -15, -38, -66, -79, -68, -44, -27, -28, -41, -47, -34, -10, 3, -11, -51, -93, -109, -88, -43, 0, 24, 23, 8, -7, -20, -33, -47, -60, -65, -59, -46, -39, -45, -63, -81, -87, -79, -66, -61, -64, -67, -58, -33, -5, 7, 0, -21, -37, -39, -30, -25, -32, -47, -54, -46, -30, -21, -27, -44, -55, -51, -39, -30, -35, -51, -63, -61, -44, -22, -4, 3, 2, -4, -13, -21, -26, -28, -28, -31, -34, -34, -26, -12, 0, 1, -6, -15, -16, -8, 2, 7, 3, -4, -9, -8, -4, -2, 0, 0, 0, 1, 5, 13, 25, 30, 15, -14, -34, -17, 40, 104, 117, 50, -63, -148, -140, -41, 83, 162, 174, 164, 188, 255, 310, 284, 160, -8, -143, -202, -208, -210, -225, -219, -146, -3, 146, 218, 174, 52, -55, -76, 0, 120, 211, 228, 177, 97, 33, 9, 29, 73, 113, 125, 100, 47, -6, -36, -31, -2, 35, 65, 86, 102, 118, 134, 144, 140, 118, 81, 45, 25, 33, 64, 97, 107, 82, 36, -1, -1, 39, 100, 151, 169, 157, 131, 111, 102, 98, 92, 78, 60, 41, 26, 15, 9, 11, 21, 38, 55, 65, 64, 57, 54, 64, 86, 109, 118, 103, 73, 47, 44, 67, 103, 130, 132, 111, 80, 55, 39, 28, 14, -1, -11, -5, 20, 59, 98, 124, 129, 110, 74, 33, 2, -7, 6, 39, 78, 111, 133, 139, 126, 95, 51, 7, -16, -9, 26, 70, 99, 95, 62, 19, -10, -11, 12, 52, 90, 115, 120, 106, 79, 48, 24, 10, 4, 2, 2, 8, 25, 54, 87, 109, 109, 89, 63, 46, 40, 39, 31, 11, -10, -19, -7, 23, 57, 75, 71, 49, 22, 7, 12, 38, 72, 98, 102, 80, 42, 4, -15, -13, 4, 20, 20, 2, -19, -26, -10, 23, 53, 59, 33, -9, -43, -47, -22, 11, 29, 19, -6, -26, -25, -5, 16, 28, 26, 21, 22, 32, 41, 42, 34, 23, 15, 10, 6, 1, 1, 12, 35, 59, 69, 53, 15, -26, -53, -54, -36, -14, -3, -2, -3, 3, 20, 36, 42, 35, 20, 7, 2, 3, 7, 17, 34, 57, 76, 81, 69, 53, 45, 50, 54, 43, 18, -2, 3, 40, 85, 109, 100, 70, 42, 29, 22, 10, -1, 1, 34, 83, 118, 115, 80, 44, 30, 36, 40, 27, 12, 24, 71, 123, 137, 95, 27, -10, 11, 73, 128, 143, 130, 130, 166, 217, 234, 187, 92, 0, -50, -54, -40, -34, -33, -15, 35, 108, 165, 176, 136, 73, 27, 21, 55, 109, 154, 172, 157, 121, 83, 61, 66, 92, 122, 138, 129, 101, 69, 53, 65, 100, 140, 161, 155, 129, 108, 111, 140, 178, 199, 191, 159, 120, 93, 85, 93, 108, 124, 132, 132, 126, 122, 129, 147, 163, 161, 136, 101, 75, 72, 85, 97, 98, 97, 110, 140, 173, 186, 171, 146, 137, 154, 181, 190, 168, 128, 102, 103, 119, 125, 106, 75, 62, 82, 125, 161, 168, 148, 119, 99, 92, 87, 77, 64, 60, 73, 102, 136, 162, 177, 180, 172, 153, 123, 92, 70, 66, 79, 96, 108, 111, 111, 114, 118, 114, 95, 64, 33, 20, 29, 54, 76, 81, 70, 55, 48, 54, 65, 70, 67, 61, 64, 79, 99, 109, 98, 67, 33, 12, 13, 30, 46, 50, 40, 28, 24, 33, 47, 57, 59, 55, 50, 43, 29, 7, -17, -33, -32, -22, -16, -23, -37, -38, -11, 36, 78, 86, 55, 6, -30, -40, -31, -26, -36, -52, -49, -15, 38, 80, 86, 55, 10, -19, -24, -17, -18, -37, -63, -71, -46, 4, 52, 67, 45, 5, -23, -19, 7, 35, 39, 14, -24, -55, -66, -60, -45, -33, -27, -25, -25, -27, -30, -31, -32, -33, -40, -52, -63, -69, -67, -61, -58, -61, -63, -61, -53, -48, -55, -72, -90, -98, -88, -65, -39, -18, -6, -6, -18, -41, -66, -83, -86, -80, -78, -88, -109, -129, -135, -126, -110, -95, -86, -80, -77, -79, -90, -107, -119, -117, -104, -93, -93, -105, -115, -114, -105, -100, -109, -125, -132, -117, -86, -58, -54, -74, -102, -121, -124, -115, -103, -89, -69, -43, -25, -29, -59, -99, -123, -118, -94, -75, -78, -99, -118, -119, -104, -92, -96, -112, -119, -102, -68, -37, -28, -38, -53, -60, -62, -73, -98, -122, -123, -94, -56, -43, -67, -108, -130, -109, -63, -31, -40, -85, -129, -143, -124, -96, -78, -71, -61, -39, -15, -10, -33, -69, -95, -102, -99, -99, -108, -110, -93, -59, -30, -26, -50, -81, -97, -91, -75, -65, -68, -78, -86, -86, -82, -75, -62, -40, -14, 2, 2, -15, -39, -55, -60, -60, -62, -67, -71, -70, -67, -66, -68, -71, -67, -54, -38, -25, -20, -22, -27, -31, -33, -31, -27, -23, -25, -33, -43, -51, -48, -38, -26, -20, -22, -30, -36, -36, -33, -33, -41, -58, -78, -93, -91, -71, -40, -16, -14, -32, -58, -74, -73, -66, -72, -95, -120, -125, -99, -55, -21, -16, -36, -58, -63, -50, -35, -32, -48, -73, -98, -118, -134, -146, -149, -140, -118, -94, -76, -69, -72, -82, -96, -113, -126, -131, -125, -115, -109, -110, -116, -119, -115, -108, -102, -100, -101, -98, -95, -95, -102, -111, -115, -110, -99, -93, -97, -105, -107, -100, -90, -88, -97, -109, -116, -113, -108, -108, -114, -117, -111, -95, -83, -85, -98, -112, -114, -103, -91, -90, -104, -120, -123, -108, -82, -61, -57, -66, -79, -87, -88, -87, -93, -106, -119, -123, -113, -92, -72, -64, -71, -89, -106, -110, -98, -74, -51, -42, -55, -82, -111, -123, -112, -86, -62, -56, -72, -95, -108, -101, -77, -53, -44, -54, -72, -84, -81, -68, -54, -50, -60, -79, -97, -105, -101, -86, -66, -49, -41, -47, -62, -77, -82, -72, -55, -46, -57, -85, -114, -124, -108, -75, -46, -35, -43, -60, -73, -79, -81, -82, -81, -78, -72, -70, -73, -76, -72, -60, -50, -56, -78, -98, -95, -64, -21, 2, -6, -39, -74, -97, -111, -128, -145, -141, -100, -30, 31, 47, 8, -54, -95, -91, -55, -23, -18, -34, -49, -43, -15, 18, 44, 52, 41, 14, -25, -69, -102, -112, -99, -74, -52, -40, -37, -36, -33, -29, -24, -17, -14, -22, -47, -80, -100, -87, -43, 8, 35, 21, -18, -54, -62, -44, -17, 0, 3, -1, -6, -7, -3, 9, 32, 55, 63, 43, 1, -41, -62, -58, -39, -25, -20, -14, 2, 28, 49, 56, 53, 52, 57, 60, 48, 19, -6, -12, 6, 34, 51, 49, 40, 41, 57, 76, 84, 74, 53, 33, 23, 23, 30, 42, 59, 76, 89, 92, 87, 80, 73, 65, 57, 49, 47, 49, 51, 45, 30, 18, 22, 43, 71, 89, 91, 84, 76, 73, 70, 60, 43, 27, 19, 20, 23, 25, 30, 40, 54, 62, 57, 42, 29, 25, 30, 34, 30, 23, 21, 26, 30, 23, 5, -10, -6, 19, 54, 74, 67, 39, 8, -7, -4, 12, 29, 37, 32, 17, 0, -10, -8, 3, 16, 20, 13, 2, -3, 1, 16, 32, 43, 44, 34, 15, -5, -20, -17, 1, 27, 44, 41, 23, 3, -3, 4, 19, 27, 23, 5, -16, -33, -37, -24, 1, 32, 55, 65, 65, 59, 49, 31, 4, -18, -20, 6, 44, 63, 43, 2, -16, 19, 98, 164, 165, 93, -1, -57, -42, 17, 70, 81, 55, 26, 19, 33, 51, 59, 59, 63, 73, 80, 76, 61, 48, 44, 49, 55, 58, 61, 66, 73, 74, 67, 57, 53, 58, 64, 61, 45, 28, 23, 34, 52, 64, 62, 55, 49, 49, 52, 52, 46, 37, 27, 20, 20, 30, 47, 64, 68, 55, 35, 22, 25, 36, 39, 27, 13, 15, 36, 57, 54, 25, -5, -9, 16, 45, 45, 11, -22, -21, 12, 37, 9, -67, -132, -115, 2, 158, 253, 224, 93, -55, -138, -132, -77, -31, -22, -32, -28, 3, 44, 60, 34, -17, -64, -81, -67, -41, -18, -5, 5, 19, 32, 32, 17, -6, -21, -15, 10, 38, 51, 42, 22, 5, 0, 4, 8, 9, 9, 10, 13, 15, 12, 5, 0, 0, 2, 3, 0, -2, 1, 11, 21, 25, 18, 5, -6, -14, -18, -19, -17, -12, -7, -6, -10, -16, -17, -10, 1, 5, -2, -17, -28, -27, -19, -15, -20, -31, -37, -33, -22, -14, -14, -17, -15, -8, -3, -8, -20, -27, -19, 2, 22, 25, 9, -14, -32, -35, -30, -23, -17, -12, -5, 0, -2, -14, -31, -44, -47, -39, -26, -12, -3, 0, -5, -16, -28, -33, -31, -26, -26, -35, -45, -47, -37, -27, -28, -42, -58, -58, -40, -16, -9, -28, -63, -90, -94, -80, -66, -64, -71, -75, -66, -51, -45, -57, -80, -97, -94, -73, -47, -31, -30, -40, -53, -66, -77, -83, -81, -72, -63, -64, -77, -96, -108, -107, -97, -90, -93, -102, -105, -97, -83, -80, -95, -124, -148, -150, -128, -96, -75, -78, -102, -133, -155, -160, -151, -134, -117, -103, -94, -89, -87, -86, -89, -97, -110, -127, -143, -154, -155, -146, -133, -121, -114, -112, -112, -110, -105, -100, -103, -114, -131, -147, -153, -149, -138, -130, -127, -127, -123, -113, -101, -98, -107, -125, -142, -147, -139, -126, -119, -121, -131, -140, -142, -137, -129, -122, -119, -118, -118, -117, -113, -107, -103, -105, -113, -124, -133, -135, -131, -125, -121, -119, -116, -111, -107, -110, -120, -134, -142, -137, -121, -101, -87, -84, -91, -106, -126, -146, -160, -160, -144, -119, -97, -89, -94, -105, -113, -113, -111, -112, -115, -114, -106, -91, -80, -78, -86, -94, -93, -83, -69, -63, -67, -77, -85, -89, -91, -94, -99, -102, -97, -82, -62, -45, -39, -44, -55, -62, -59, -50, -42, -40, -46, -55, -58, -54, -45, -39, -40, -47, -53, -53, -45, -34, -24, -16, -11, -8, -7, -11, -18, -26, -33, -36, -36, -33, -27, -17, -4, 7, 12, 10, 5, 1, 1, 0, -4, -14, -26, -33, -31, -22, -10, 1, 10, 15, 15, 9, 1, -3, -2, 4, 11, 12, 6, 0, -1, 4, 13, 16, 11, 3, 0, 2, 8, 10, 7, 3, 4, 13, 26, 33, 29, 18, 8, 5, 12, 25, 35, 39, 38, 36, 36, 37, 37, 35, 30, 27, 29, 35, 41, 45, 44, 42, 40, 40, 40, 42, 46, 55, 66, 73, 71, 57, 39, 28, 30, 42, 55, 63, 66, 69, 76, 84, 87, 81, 72, 65, 66, 73, 78, 78, 73, 66, 63, 64, 68, 73, 79, 82, 80, 73, 62, 56, 56, 63, 71, 74, 73, 71, 69, 67, 61, 55, 54, 64, 81, 93, 90, 76, 64, 65, 80, 90, 84, 61, 41, 45, 74, 108, 123, 105, 68, 36, 27, 40, 59, 71, 72, 69, 69, 70, 70, 65, 59, 58, 63, 72, 74, 68, 55, 46, 47, 63, 87, 107, 112, 99, 72, 44, 29, 30, 41, 52, 54, 51, 49, 51, 56, 56, 49, 39, 32, 35, 44, 54, 63, 70, 76, 80, 78, 68, 53, 41, 40, 49, 61, 69, 69, 66, 64, 66, 70, 71, 69, 65, 63, 64, 65, 63, 57, 51, 47, 48, 55, 64, 73, 80, 84, 85, 83, 80, 76, 72, 67, 62, 57, 51, 46, 45, 49, 59, 74, 88, 96, 95, 87, 75, 63, 54, 51, 55, 62, 71, 76, 76, 72, 70, 75, 84, 92, 92, 83, 70, 63, 67, 79, 93, 99, 97, 90, 83, 83, 89, 98, 108, 112, 111, 105, 98, 94, 95, 100, 105, 105, 99, 91, 85, 85, 90, 99, 107, 112, 113, 110, 105, 101, 99, 100, 103, 104, 105, 105, 108, 113, 118, 119, 117, 111, 106, 106, 108, 108, 105, 98, 93, 95, 103, 116, 128, 136, 139, 140, 141, 141, 138, 131, 122, 115, 116, 122, 131, 137, 139, 141, 144, 150, 155, 155, 148, 139, 131, 130, 133, 139, 143, 145, 145, 142, 135, 125, 115, 109, 112, 123, 138, 152, 159, 158, 151, 143, 138, 138, 141, 143, 139, 131, 123, 122, 129, 140, 148, 149, 145, 141, 142, 144, 142, 132, 117, 107, 106, 115, 129, 142, 151, 156, 156, 151, 142, 130, 122, 120, 124, 132, 139, 145, 147, 143, 131, 115, 104, 106, 124, 152, 173, 179, 171, 157, 148, 144, 139, 130, 116, 107, 109, 122, 137, 147, 149, 147, 148, 152, 157, 159, 159, 158, 155, 149, 139, 128, 120, 119, 126, 134, 138, 135, 128, 119, 110, 103, 98, 98, 103, 110, 117, 119, 117, 113, 109, 104, 98, 89, 80, 78, 84, 96, 107, 112, 111, 107, 107, 110, 111, 105, 91, 74, 63, 63, 71, 80, 83, 81, 77, 75, 76, 76, 74, 70, 68, 72, 77, 77, 68, 52, 38, 32, 32, 32, 28, 19, 15, 21, 37, 55, 67, 69, 66, 62, 58, 52, 45, 37, 34, 36, 37, 31, 18, 4, -4, -4, 0, 1, -1, -6, -6, -1, 4, 8, 7, 4, 2, 0, -3, -11, -16, -14, -3, 10, 17, 11, -3, -17, -22, -16, -4, 10, 23, 32, 35, 28, 12, -6, -20, -20, -9, 2, 7, 4, -2, -5, -1, 6, 12, 13, 10, 7, 7, 9, 10, 7, 1, -4, -9, -9, -2, 8, 21, 32, 39, 39, 36, 33, 32, 32, 32, 31, 30, 31, 34, 36, 36, 29, 18, 6, 0, 1, 8, 15, 17, 15, 13, 17, 26, 38, 44, 40, 28, 17, 14, 20, 29, 33, 29, 21, 14, 14, 17, 21, 23, 25, 29, 36, 42, 42, 35, 25, 15, 7, 0, -6, -11, -10, -4, 5, 14, 17, 17, 16, 17, 18, 16, 10, 6, 8, 14, 21, 22, 15, 4, -3, -4, -1, 0, -4, -12, -18, -16, -5, 12, 29, 39, 41, 32, 18, 2, -9, -12, -8, 0, 8, 14, 14, 10, 0, -10, -18, -20, -13, -5, 0, 0, -2, -3, -1, 0, -6, -19, -30, -32, -22, -7, 1, 0, -10, -18, -19, -13, -9, -9, -11, -10, -6, -4, -9, -22, -36, -42, -37, -26, -18, -18, -27, -36, -40, -38, -34, -31, -32, -36, -43, -48, -49, -47, -43, -39, -37, -36, -35, -33, -34, -40, -52, -63, -67, -60, -46, -33, -31, -39, -50, -56, -55, -48, -41, -39, -43, -50, -58, -65, -69, -69, -65, -64, -66, -72, -76, -75, -67, -60, -61, -70, -83, -91, -91, -86, -85, -90, -98, -100, -95, -86, -83, -90, -104, -115, -118, -113, -107, -107, -110, -112, -108, -100, -93, -92, -98, -107, -113, -114, -113, -114, -118, -122, -124, -122, -117, -113, -117, -127, -141, -152, -156, -153, -145, -141, -143, -153, -163, -168, -164, -154, -145, -142, -145, -150, -149, -142, -134, -130, -133, -139, -141, -136, -130, -128, -134, -145, -152, -152, -146, -141, -142, -147, -153, -153, -149, -144, -143, -147, -152, -156, -158, -161, -164, -166, -166, -163, -160, -156, -150, -143, -134, -131, -139, -156, -175, -183, -171, -144, -116, -101, -105, -122, -140, -148, -147, -141, -139, -146, -159, -170, -170, -157, -136, -121, -121, -136, -157, -171, -173, -165, -157, -154, -155, -152, -142, -130, -125, -130, -141, -149, -148, -142, -136, -133, -130, -121, -110, -105, -114, -130, -143, -138, -121, -104, -103, -120, -140, -146, -137, -125, -126, -142, -159, -159, -136, -108, -95, -107, -133, -154, -154, -140, -125, -120, -126, -131, -128, -121, -119, -126, -137, -146, -146, -141, -139, -140, -141, -135, -121, -107, -103, -112, -126, -137, -138, -134, -132, -135, -140, -142, -136, -127, -122, -125, -133, -140, -142, -137, -132, -130, -132, -136, -138, -137, -132, -127, -123, -119, -115, -111, -106, -101, -96, -92, -90, -92, -99, -110, -119, -118, -104, -80, -59, -53, -67, -95, -122, -137, -134, -119, -105, -96, -94, -95, -96, -96, -98, -102, -104, -102, -93, -79, -66, -53, -41, -32, -27, -28, -37, -48, -55, -53, -43, -30, -19, -14, -11, -10, -10, -12, -18, -27, -35, -39, -37, -28, -17, -5, 4, 10, 10, 3, -5, -15, -21, -24, -26, -27, -28, -25, -18, -10, -6, -6, -11, -15, -15, -10, -4, 0, 0, 0, 0, 1, 1, 0, -1, 0, 3, 8, 11, 11, 8, 5, 5, 8, 12, 15, 18, 18, 18, 15, 9, 0, -8, -15, -19, -20, -18, -14, -9, -2, 2, 6, 4, 0, -8, -16, -21, -20, -15, -7, 0, 5, 8, 10, 10, 9, 7, 3, 0, 0, 2, 4, 3, -3, -15, -25, -29, -27, -23, -20, -19, -19, -18, -17, -18, -22, -24, -21, -11, 0, 6, 2, -7, -14, -11, -3, 3, 2, -4, -10, -10, -2, 4, 6, 1, -8, -17, -22, -21, -15, -5, 4, 13, 17, 16, 12, 10, 11, 14, 15, 15, 16, 20, 28, 36, 39, 37, 31, 26, 25, 28, 34, 40, 44, 47, 48, 50, 53, 57, 57, 50, 39, 28, 26, 34, 47, 56, 56, 48, 42, 41, 46, 48, 43, 34, 29, 34, 46, 58, 63, 60, 55, 52, 54, 59, 64, 67, 67, 65, 62, 58, 54, 52, 51, 50, 47, 44, 45, 52, 63, 71, 68, 54, 33, 16, 8, 12, 23, 36, 43, 44, 41, 37, 36, 37, 37, 32, 23, 14, 10, 14, 22, 28, 27, 21, 16, 17, 22, 30, 34, 36, 37, 39, 41, 41, 40, 40, 41, 44, 45, 43, 41, 40, 42, 41, 36, 26, 19, 19, 30, 46, 60, 65, 64, 59, 55, 53, 52, 52, 54, 56, 57, 57, 58, 62, 70, 79, 87, 92, 97, 105, 119, 131, 136, 126, 105, 83, 68, 65, 69, 75, 80, 83, 89, 98, 109, 117, 120, 116, 108, 101, 99, 104, 113, 120, 120, 113, 103, 97, 101, 113, 128, 138, 137, 125, 105, 86, 75, 78, 92, 109, 117, 112, 100, 95, 103, 123, 138, 136, 116, 92, 80, 87, 104, 115, 110, 91, 73, 68, 77, 94, 107, 112, 110, 103, 94, 86, 83, 84, 83, 71, 46, 15, 0, 9, 42, 75, 82, 61, 31, 18, 31, 55, 63, 45, 16, 6, 29, 73, 106, 101, 59, 5, -30, -31, -1, 37, 63, 62, 30, -18, -61, -73, -48, -1, 42, 62, 62, 59, 66, 77, 75, 49, 12, -15, -21, -11, -5, -11, -19, -15, 1, 14, 10, -5, -9, 13, 48, 56, 16, -51, -95, -77, -4, 67, 87, 47, -12, -44, -31, 3, 30, 36, 37, 50, 75, 92, 85, 58, 27, 5, -4, -5, 1, 23, 58, 90, 101, 85, 59, 45, 51, 59, 51, 22, -5, -8, 16, 46, 51, 24, -14, -37, -29, 0, 28, 41, 39, 30, 24, 27, 37, 52, 65, 68, 57, 38, 25, 24, 29, 27, 12, -6, -12, 0, 17, 21, 4, -16, -16, 10, 50, 74, 68, 41, 16, 5, 6, 7, 4, 0, 4, 13, 20, 21, 19, 17, 12, -3, -32, -61, -76, -68, -47, -28, -19, -15, -5, 10, 22, 21, 11, 5, 9, 14, 9, -7, -23, -22, -5, 8, 5, -12, -24, -17, 1, 13, 7, -3, -4, 5, 7, -10, -34, -31, 11, 61, 59, -22, -137, -196, -136, 18, 174, 235, 182, 70, -24, -68, -77, -82, -87, -76, -46, -26, -48, -113, -170, -161, -72, 50, 136, 149, 109, 66, 49, 53, 50, 28, 1, -9, 0, 15, 20, 11, -4, -23, -46, -71, -86, -75, -32, 18, 42, 23, -24, -63, -63, -23, 31, 73, 90, 83, 60, 25, -15, -47, -57, -41, -11, 15, 29, 34, 41, 51, 58, 50, 31, 10, -2, -5, -6, -12, -19, -19, -11, -1, 4, 2, 0, -1, 0, -4, -15, -30, -41, -45, -44, -39, -27, -8, 12, 21, 7, -21, -44, -42, -19, 3, 1, -28, -66, -84, -74, -51, -38, -46, -67, -85, -89, -81, -67, -48, -24, 0, 15, 13, -2, -14, -11, 6, 20, 13, -14, -48, -67, -66, -55, -49, -55, -65, -70, -63, -48, -33, -23, -20, -23, -36, -55, -74, -84, -79, -62, -43, -34, -36, -44, -53, -59, -67, -77, -89, -96, -95, -88, -81, -78, -78, -74, -63, -48, -36, -34, -42, -56, -71, -84, -91, -91, -82, -68, -57, -62, -87, -119, -140, -133, -99, -58, -32, -33, -54, -75, -85, -83, -78, -77, -81, -85, -87, -89, -95, -104, -111, -110, -101, -88, -78, -73, -73, -81, -97, -116, -128, -124, -104, -80, -69, -79, -103, -124, -133, -132, -130, -129, -124, -106, -77, -52, -44, -58, -80, -98, -105, -105, -103, -98, -89, -75, -67, -71, -87, -102, -105, -96, -88, -90, -101, -107, -98, -76, -58, -56, -73, -97, -115, -121, -115, -99, -76, -54, -47, -62, -95, -127, -137, -121, -96, -86, -100, -126, -140, -131, -110, -97, -105, -127, -143, -140, -120, -95, -79, -76, -85, -103, -126, -147, -159, -157, -145, -132, -127, -129, -131, -128, -120, -114, -114, -121, -129, -134, -136, -136, -133, -129, -127, -134, -150, -169, -173, -153, -115, -77, -56, -59, -72, -84, -91, -97, -109, -121, -125, -113, -91, -71, -62, -60, -60, -60, -64, -79, -100, -116, -113, -89, -55, -25, -8, -6, -15, -32, -52, -69, -77, -74, -64, -56, -56, -63, -66, -61, -47, -32, -20, -11, -5, -2, -5, -17, -32, -44, -48, -48, -53, -65, -79, -84, -70, -43, -16, 0, 2, -2, -5, -2, 4, 8, 5, -5, -20, -34, -39, -34, -22, -10, -3, -1, 1, 8, 20, 31, 36, 33, 24, 13, 2, -10, -22, -28, -25, -13, -1, 4, 0, -7, -15, -19, -20, -22, -21, -12, 4, 22, 31, 26, 9, -13, -32, -47, -58, -65, -65, -55, -39, -27, -25, -32, -39, -38, -28, -15, -8, -10, -19, -30, -42, -54, -66, -72, -66, -48, -24, -8, -7, -19, -35, -46, -46, -39, -29, -23, -22, -21, -10, 10, 35, 44, 20, -36, -106, -153, -151, -102, -39, 0, -1, -29, -52, -50, -27, -5, -4, -26, -55, -76, -80, -70, -49, -16, 23, 56, 63, 36, -9, -44, -46, -15, 22, 39, 29, 7, -3, 7, 32, 52, 58, 50, 38, 30, 30, 38, 51, 66, 75, 67, 44, 17, 3, 13, 41, 71, 89, 92, 86, 74, 59, 40, 26, 26, 46, 74, 94, 93, 80, 69, 74, 87, 91, 77, 55, 47, 64, 100, 133, 141, 122, 91, 63, 49, 46, 51, 58, 65, 70, 71, 72, 78, 91, 104, 109, 103, 92, 89, 97, 109, 109, 92, 69, 58, 69, 94, 115, 119, 106, 90, 83, 86, 95, 101, 100, 94, 84, 73, 64, 61, 66, 76, 85, 90, 94, 99, 106, 110, 106, 92, 74, 62, 60, 69, 83, 99, 113, 121, 121, 114, 107, 107, 114, 123, 124, 118, 113, 118, 130, 142, 146, 148, 155, 167, 173, 163, 141, 128, 140, 171, 191, 176, 134, 100, 108, 150, 182, 166, 106, 54, 64, 142, 243, 303, 298, 249, 202, 175, 153, 110, 43, -23, -55, -36, 28, 123, 234, 346, 426, 437, 357, 212, 61, -31, -45, -6, 41, 78, 113, 167, 243, 311, 333, 296, 218, 135, 79, 58, 66, 95, 136, 181, 219, 239, 237, 223, 210, 205, 206, 200, 180, 149, 122, 115, 134, 167, 190, 186, 158, 128, 124, 154, 201, 233, 228, 191, 146, 119, 112, 114, 112, 110, 119, 143, 170, 180, 162, 130, 102, 91, 92, 94, 93, 95, 112, 141, 168, 180, 176, 164, 154, 144, 127, 102, 82, 80, 102, 134, 159, 165, 159, 151, 150, 154, 159, 161, 162, 161, 158, 151, 143, 139, 142, 149, 154, 152, 147, 141, 137, 131, 122, 115, 117, 126, 134, 128, 109, 94, 104, 146, 205, 255, 276, 267, 240, 209, 178, 148, 121, 104, 100, 106, 116, 127, 142, 161, 180, 188, 178, 157, 139, 136, 145, 153, 150, 137, 127, 126, 134, 142, 147, 154, 167, 185, 200, 204, 200, 194, 192, 192, 185, 172, 161, 160, 170, 181, 181, 172, 166, 174, 196, 220, 230, 222, 205, 192, 191, 199, 210, 222, 231, 238, 238, 231, 219, 205, 190, 175, 161, 151, 149, 158, 170, 177, 175, 168, 165, 168, 171, 166, 153, 140, 134, 136, 136, 127, 112, 101, 106, 123, 142, 150, 146, 137, 130, 127, 123, 113, 97, 84, 78, 79, 84, 91, 98, 104, 108, 109, 106, 99, 90, 78, 62, 47, 38, 41, 54, 68, 76, 76, 74, 73, 72, 66, 51, 31, 15, 8, 8, 8, 4, 0, 2, 12, 29, 44, 52, 56, 55, 49, 37, 21, 7, 6, 17, 35, 48, 53, 52, 53, 56, 56, 47, 31, 19, 19, 30, 44, 50, 49, 46, 48, 56, 64, 67, 62, 53, 47, 45, 48, 52, 57, 63, 66, 65, 58, 49, 39, 33, 31, 32, 35, 39, 42, 42, 41, 41, 48, 62, 76, 81, 72, 55, 40, 38, 46, 56, 61, 62, 67, 78, 94, 104, 101, 86, 65, 49, 39, 37, 38, 43, 49, 54, 54, 50, 45, 46, 54, 60, 56, 39, 14, -3, -10, -5, 3, 11, 19, 29, 37, 37, 25, 8, -4, -6, 0, 3, 0, -9, -14, -9, 0, 7, 6, 3, 4, 12, 21, 23, 18, 13, 14, 18, 16, 4, -11, -20, -17, -9, -9, -22, -41, -49, -42, -26, -14, -9, -7, 0, 10, 15, 6, -13, -29, -34, -30, -28, -33, -41, -41, -31, -21, -23, -41, -64, -77, -76, -67, -59, -54, -49, -39, -32, -36, -50, -65, -70, -66, -61, -65, -76, -81, -69, -45, -27, -28, -47, -68, -79, -79, -75, -75, -76, -74, -67, -61, -60, -64, -65, -62, -65, -80, -105, -123, -121, -102, -84, -86, -110, -140, -155, -147, -124, -95, -62, -30, -8, -13, -57, -125, -181, -195, -161, -106, -66, -58, -71, -86, -91, -91, -96, -107, -119, -126, -129, -129, -126, -113, -93, -76, -79, -104, -137, -157, -156, -149, -154, -174, -193, -191, -166, -138, -132, -151, -176, -185, -173, -151, -136, -136, -146, -160, -178, -197, -210, -204, -177, -146, -132, -142, -160, -160, -136, -108, -104, -131, -170, -194, -190, -174, -169, -177, -183, -170, -138, -104, -82, -70, -60, -56, -79, -146, -239, -311, -317, -254, -166, -104, -93, -113, -129, -127, -123, -143, -187, -232, -249, -230, -190, -152, -122, -100, -86, -92, -125, -177, -220, -227, -195, -144, -105, -88, -87, -89, -90, -101, -129, -163, -184, -175, -139, -95, -63, -54, -66, -89, -113, -133, -146, -151, -148, -141, -131, -119, -103, -86, -77, -81, -96, -115, -126, -127, -123, -121, -122, -122, -115, -104, -95, -94, -100, -106, -107, -102, -97, -96, -99, -102, -101, -97, -95, -98, -108, -119, -127, -127, -117, -100, -84, -78, -86, -102, -120, -130, -132, -131, -132, -135, -135, -133, -131, -134, -142, -148, -147, -138, -129, -127, -135, -145, -150, -150, -149, -153, -160, -164, -161, -154, -151, -155, -165, -172, -173, -168, -164, -162, -161, -154, -138, -118, -103, -101, -113, -131, -145, -149, -145, -142, -149, -162, -173, -169, -149, -126, -116, -126, -151, -174, -181, -171, -153, -138, -130, -129, -131, -136, -143, -149, -149, -143, -138, -140, -149, -156, -151, -132, -113, -107, -115, -124, -121, -109, -103, -118, -149, -174, -173, -144, -112, -99, -112, -133, -142, -132, -118, -115, -129, -147, -155, -147, -129, -110, -95, -84, -79, -82, -94, -108, -115, -108, -95, -88, -94, -105, -105, -87, -58, -37, -37, -59, -85, -101, -101, -91, -82, -80, -82, -84, -82, -78, -76, -80, -89, -98, -103, -100, -92, -82, -76, -76, -82, -92, -104, -112, -111, -100, -81, -64, -58, -67, -83, -94, -93, -85, -80, -86, -97, -101, -90, -69, -51, -45, -50, -61, -73, -88, -106, -120, -118, -97, -66, -45, -48, -69, -89, -93, -85, -78, -80, -87, -87, -75, -61, -57, -66, -79, -82, -73, -63, -61, -63, -59, -44, -30, -31, -51, -72, -75, -55, -30, -25, -44, -68, -72, -47, -12, 4, -6, -33, -56, -64, -60, -53, -44, -31, -13, 1, 2, -9, -21, -18, -4, 7, 2, -16, -31, -26, -2, 23, 33, 26, 15, 12, 15, 15, 6, -2, 0, 16, 35, 39, 25, 6, 0, 13, 36, 52, 52, 44, 37, 39, 44, 43, 34, 21, 12, 10, 10, 11, 13, 19, 30, 46, 61, 74, 83, 90, 93, 88, 77, 64, 54, 51, 56, 64, 73, 79, 80, 75, 64, 51, 45, 52, 71, 89, 93, 79, 55, 38, 41, 62, 85, 97, 93, 82, 72, 67, 63, 54, 43, 38, 49, 71, 93, 102, 94, 76, 61, 55, 55, 55, 55, 60, 74, 95, 114, 121, 113, 95, 76, 61, 53, 52, 58, 68, 78, 82, 78, 71, 70, 81, 101, 120, 128, 123, 108, 92, 79, 70, 64, 60, 58, 59, 60, 62, 62, 59, 49, 34, 20, 18, 33, 62, 89, 98, 87, 66, 54, 57, 69, 73, 64, 46, 34, 34, 42, 48, 46, 41, 40, 46, 55, 62, 66, 69, 72, 70, 59, 41, 24, 15, 14, 14, 11, 8, 13, 33, 59, 78, 79, 64, 46, 36, 36, 40, 46, 54, 66, 80, 89, 85, 71, 55, 45, 43, 45, 47, 50, 54, 56, 52, 39, 27, 27, 40, 56, 59, 44, 24, 16, 31, 56, 71, 67, 55, 53, 73, 103, 118, 105, 72, 41, 30, 38, 52, 59, 57, 52, 50, 50, 50, 48, 50, 56, 60, 51, 26, -2, -19, -10, 22, 61, 86, 88, 72, 48, 27, 11, 1, -2, 0, 7, 15, 18, 17, 17, 20, 26, 26, 14, -7, -24, -27, -16, -3, -4, -18, -31, -29, -7, 17, 27, 15, -7, -21, -19, -11, -16, -39, -67, -76, -53, -8, 34, 54, 45, 20, -4, -20, -27, -29, -31, -33, -35, -35, -28, -13, 8, 25, 27, 10, -15, -37, -46, -46, -48, -55, -57, -41, -8, 24, 35, 17, -13, -32, -26, -4, 17, 26, 26, 25, 24, 16, -5, -33, -52, -47, -23, 1, 15, 16, 16, 23, 34, 37, 27, 9, -2, -1, 8, 19, 27, 35, 48, 60, 61, 47, 24, 7, 7, 21, 37, 40, 32, 22, 18, 24, 32, 36, 33, 28, 25, 24, 26, 27, 25, 17, 3, -15, -31, -37, -32, -18, -3, 8, 16, 22, 28, 30, 28, 22, 18, 22, 33, 46, 53, 47, 28, 0, -30, -52, -57, -46, -25, -8, -5, -15, -28, -31, -20, -1, 13, 16, 10, 1, -2, -3, -4, -11, -24, -37, -44, -42, -29, -13, 0, 4, -1, -14, -25, -28, -22, -12, -5, -2, 0, 0, 0, -5, -15, -25, -28, -23, -13, -4, 1, 8, 16, 24, 22, 7, -17, -39, -45, -34, -13, 4, 10, 3, -10, -25, -37, -41, -38, -26, -12, -1, 1, -5, -16, -27, -34, -36, -32, -21, -6, 4, 3, -9, -28, -38, -28, -4, 15, 15, -10, -49, -76, -74, -47, -10, 14, 19, 11, 3, 1, 0, -12, -39, -71, -90, -87, -66, -45, -39, -48, -60, -61, -52, -44, -49, -63, -70, -55, -17, 23, 42, 28, -9, -49, -70, -68, -53, -44, -50, -71, -96, -110, -104, -80, -53, -38, -42, -59, -74, -73, -62, -56, -69, -99, -126, -129, -102, -60, -29, -28, -54, -89, -113, -119, -111, -105, -108, -124, -146, -162, -162, -143, -116, -93, -87, -96, -108, -109, -95, -81, -81, -101, -126, -135, -117, -83, -54, -48, -65, -89, -102, -101, -93, -89, -92, -97, -100, -100, -104, -120, -145, -169, -183, -182, -168, -150, -136, -128, -124, -121, -114, -103, -93, -88, -92, -104, -120, -134, -141, -141, -135, -123, -108, -89, -71, -61, -63, -80, -104, -126, -136, -126, -100, -67, -39, -26, -33, -55, -86, -114, -135, -144, -140, -123, -95, -62, -31, -15, -20, -44, -76, -103, -117, -115, -104, -92, -83, -74, -66, -59, -57, -61, -70, -79, -82, -82, -81, -83, -85, -83, -76, -68, -67, -80, -102, -125, -140, -145, -141, -130, -112, -86, -55, -31, -26, -42, -71, -94, -100, -89, -73, -60, -55, -54, -52, -51, -53, -62, -75, -86, -90, -86, -79, -70, -58, -44, -31, -24, -31, -51, -80, -105, -119, -115, -98, -75, -57, -47, -42, -35, -20, 0, 11, 3, -24, -60, -89, -97, -85, -64, -44, -29, -19, -11, -7, -9, -14, -19, -21, -21, -24, -30, -37, -42, -42, -43, -46, -46, -36, -10, 22, 47, 50, 32, 10, -1, 0, 0, -8, -26, -38, -30, 0, 39, 67, 72, 57, 30, 1, -25, -42, -43, -22, 20, 70, 112, 131, 127, 106, 76, 46, 23, 12, 14, 25, 37, 44, 46, 52, 63, 77, 86, 84, 79, 80, 91, 101, 95, 67, 27, -2, -6, 13, 43, 68, 85, 97, 107, 109, 96, 69, 35, 5, -14, -26, -26, -9, 31, 87, 137, 153, 128, 78, 33, 13, 16, 27, 36, 50, 79, 123, 162, 172, 146, 99, 56, 32, 26, 29, 36, 52, 75, 95, 99, 85, 64, 54, 57, 57, 41, 12, -2, 18, 69, 120, 139, 122, 94, 82, 90, 94, 73, 30, -6, -13, 7, 33, 42, 37, 38, 59, 94, 119, 125, 117, 109, 103, 86, 49, 2, -25, -12, 38, 97, 137, 147, 139, 124, 103, 71, 32, 5, 7, 34, 66, 79, 74, 72, 89, 121, 142, 132, 93, 52, 33, 36, 43, 38, 19, 0, -2, 11, 37, 65, 91, 108, 110, 91, 56, 19, -5, -12, -3, 17, 49, 93, 138, 165, 157, 114, 59, 12, -16, -38, -59, -72, -59, -15, 42, 84, 95, 83, 72, 77, 90, 97, 92, 86, 87, 88, 70, 26, -25, -50, -37, -2, 19, 16, 13, 43, 111, 183, 209, 173, 105, 55, 49, 70, 86, 83, 76, 90, 122, 147, 136, 93, 46, 21, 21, 24, 15, 4, 12, 49, 95, 117, 97, 52, 15, 10, 31, 55, 61, 45, 21, 2, -1, 7, 23, 39, 46, 43, 34, 34, 55, 97, 144, 172, 166, 133, 97, 82, 99, 137, 167, 165, 127, 75, 48, 78, 158, 243, 276, 225, 112, 0, -51, -18, 68, 159, 220, 242, 241, 230, 215, 196, 176, 158, 145, 134, 126, 128, 149, 182, 203, 187, 132, 63, 20, 24, 62, 103, 126, 130, 125, 117, 99, 71, 50, 62, 115, 187, 237, 238, 196, 146, 114, 105, 103, 101, 108, 140, 192, 237, 242, 196, 119, 46, 3, -2, 20, 64, 121, 179, 223, 242, 238, 223, 208, 194, 177, 156, 144, 148, 165, 177, 170, 148, 131, 139, 168, 197, 204, 187, 160, 136, 116, 95, 72, 57, 61, 80, 96, 97, 85, 78, 87, 104, 114, 105, 88, 81, 95, 120, 141, 150, 150, 146, 138, 120, 98, 89, 108, 148, 183, 183, 148, 104, 82, 85, 93, 82, 53, 30, 37, 72, 112, 127, 113, 81, 51, 30, 20, 26, 56, 111, 177, 224, 232, 201, 153, 106, 69, 39, 19, 18, 46, 98, 150, 175, 162, 118, 61, 7, -27, -22, 37, 146, 269, 348, 346, 269, 162, 79, 45, 50, 68, 85, 101, 120, 135, 135, 122, 113, 130, 171, 212, 222, 192, 141, 97, 72, 56, 28, -13, -55, -74, -60, -24, 14, 36, 34, 12, -19, -39, -27, 27, 115, 203, 255, 252, 206, 149, 113, 108, 119, 117, 82, 18, -51, -99, -105, -75, -32, -2, 7, 8, 21, 50, 79, 82, 50, 6, -15, 3, 51, 99, 128, 144, 166, 198, 224, 219, 176, 112, 52, 0, -54, -116, -164, -164, -97, 9, 100, 135, 116, 85, 78, 93, 95, 57, -10, -71, -92, -70, -29, 8, 43, 84, 132, 175, 200, 206, 206, 204, 183, 120, 19, -79, -119, -80, 7, 81, 97, 66, 34, 32, 49, 42, -16, -111, -188, -200, -144, -56, 17, 54, 55, 39, 18, -2, -15, -14, 4, 28, 37, 17, -21, -52, -48, -5, 52, 87, 71, 1, -100, -199, -264, -278, -246, -184, -117, -63, -25, -1, 13, 17, 3, -22, -45, -42, -1, 63, 121, 140, 110, 48, -9, -39, -33, -5, 26, 48, 58, 60, 53, 37, 10, -22, -46, -44, -5, 64, 141, 196, 210, 186, 144, 109, 92, 85, 76, 62, 55, 66, 94, 120, 122, 91, 45, 5, -17, -30, -46, -65, -70, -50, -9, 21, 15, -30, -93, -138, -150, -139, -115, -78, -14, 79, 176, 232, 218, 149, 78, 50, 73, 111, 124, 104, 72, 57, 58, 46, -7, -99, -187, -225, -195, -114, -18, 60, 107, 118, 91, 33, -33, -72, -58, 2, 71, 105, 90, 48, 20, 27, 52, 57, 17, -55, -125, -154, -131, -79, -34, -26, -60, -116, -164, -178, -152, -97, -41, -2, 15, 22, 32, 45, 50, 34, -5, -58, -107, -143, -161, -162, -142, -103, -59, -34, -46, -88, -128, -131, -91, -35, -8, -34, -95, -146, -149, -101, -34, 11, 18, -6, -43, -75, -97, -106, -99, -71, -23, 30, 77, 104, 108, 86, 40, -23, -85, -118, -104, -45, 30, 87, 106, 88, 53, 14, -23, -59, -85, -85, -50, 11, 73, 105, 95, 50, -11, -74, -124, -154, -153, -120, -66, -16, 5, -10, -52, -101, -142, -170, -180, -161, -106, -23, 55, 90, 60, -19, -103, -153, -156, -130, -101, -84, -77, -76, -82, -101, -126, -137, -113, -55, 7, 41, 29, -9, -42, -50, -43, -51, -89, -143, -178, -169, -119, -60, -27, -40, -90, -153, -205, -234, -242, -240, -240, -247, -253, -248, -222, -182, -149, -145, -177, -222, -246, -219, -144, -54, 9, 21, -16, -78, -135, -169, -172, -149, -111, -72, -42, -30, -33, -43, -51, -57, -64, -74, -83, -82, -68, -47, -34, -39, -57, -74, -76, -65, -52, -50, -56, -58, -47, -30, -24, -41, -74, -97, -93, -65, -36, -29, -46, -68, -72, -53, -25, -10, -15, -30, -40, -44, -54, -80, -112, -128, -110, -65, -18, 4, -2, -17, -22, -9, 3, 2, -16, -36, -41, -33, -31, -52, -91, -129, -143, -132, -114, -114, -137, -162, -162, -122, -53, 11, 46, 42, 10, -25, -48, -52, -43, -32, -31, -42, -60, -69, -61, -33, 0, 17, 8, -23, -61, -82, -79, -60, -40, -29, -24, -18, -10, -9, -25, -58, -93, -111, -104, -80, -59, -50, -51, -51, -49, -55, -83, -127, -167, -175, -146, -99, -65, -60, -76, -92, -90, -73, -54, -38, -20, 6, 39, 60, 53, 20, -20, -48, -55, -45, -32, -18, 0, 19, 31, 21, -11, -49, -67, -55, -22, 5, 12, -1, -18, -27, -29, -38, -60, -89, -107, -100, -68, -28, -1, -3, -29, -64, -85, -76, -36, 19, 65, 85, 73, 33, -19, -71, -108, -117, -91, -38, 15, 41, 21, -31, -84, -104, -85, -44, -6, 13, 22, 32, 44, 49, 34, 2, -32, -52, -52, -40, -27, -23, -30, -49, -79, -112, -130, -119, -81, -41, -32, -69, -130, -172, -164, -112, -56, -40, -72, -123, -152, -139, -102, -77, -90, -132, -169, -174, -146, -110, -91, -95, -101, -83, -36, 18, 50, 39, -6, -59, -93, -99, -86, -69, -58, -57, -68, -88, -113, -125, -112, -76, -34, -13, -22, -45, -54, -38, -15, -15, -51, -101, -129, -115, -80, -65, -94, -151, -191, -179, -123, -55, -7, 17, 34, 54, 67, 53, 12, -34, -56, -43, -12, 7, 1, -23, -48, -65, -76, -82, -77, -47, 7, 67, 105, 104, 72, 30, 1, -6, 2, 26, 58, 90, 105, 92, 52, 3, -36, -59, -71, -79, -78, -55, -7, 46, 76, 64, 20, -24, -43, -34, -16, -5, -2, 7, 35, 73, 100, 101, 75, 39, 14, 11, 30, 61, 85, 87, 57, 8, -33, -39, -6, 39, 58, 30, -27, -74, -77, -41, 0, 9, -23, -80, -130, -151, -138, -97, -40, 16, 52, 52, 22, -10, -15, 18, 63, 80, 46, -15, -57, -43, 14, 68, 69, 9, -75, -131, -127, -70, 13, 94, 158, 197, 204, 172, 106, 27, -36, -65, -62, -45, -27, -9, 12, 37, 53, 43, 3, -50, -93, -106, -87, -49, -6, 35, 66, 78, 59, 9, -54, -96, -91, -34, 44, 108, 131, 117, 92, 81, 89, 104, 109, 96, 71, 47, 34, 30, 28, 20, 5, -16, -39, -59, -66, -49, -2, 70, 146, 195, 198, 158, 102, 64, 62, 82, 89, 57, -11, -83, -115, -85, -6, 78, 130, 129, 85, 20, -40, -81, -96, -85, -55, -20, 6, 17, 15, 6, -10, -44, -104, -181, -244, -255, -191, -69, 69, 174, 216, 198, 142, 80, 31, 7, 9, 28, 50, 60, 51, 26, -4, -34, -59, -84, -106, -111, -87, -33, 33, 85, 99, 76, 32, -5, -20, -13, 9, 39, 73, 112, 151, 180, 186, 161, 113, 60, 24, 15, 30, 50, 57, 45, 20, 0, -7, -1, 6, 4, -9, -26, -34, -23, 4, 37, 62, 68, 55, 30, 7, -3, 1, 19, 39, 52, 51, 37, 14, -11, -29, -33, -18, 3, 16, 9, -16, -44, -54, -41, -19, -10, -28, -65, -103, -126, -131, -124, -108, -80, -35, 22, 77, 113, 121, 107, 82, 57, 31, 5, -18, -33, -37, -35, -33, -37, -44, -49, -51, -52, -58, -70, -81, -87, -89, -90, -91, -87, -72, -48, -26, -23, -44, -78, -104, -108, -87, -55, -27, -12, -6, 0, 14, 34, 48, 49, 38, 21, 10, 5, 2, -9, -32, -55, -64, -51, -23, 4, 15, 8, -6, -16, -15, -10, -8, -11, -17, -21, -22, -18, -6, 16, 45, 63, 48, -9, -96, -181, -232, -235, -202, -154, -109, -73, -46, -32, -38, -64, -96, -112, -90, -29, 53, 128, 172, 172, 138, 91, 55, 42, 47, 53, 44, 16, -14, -28, -13, 17, 41, 42, 23, 6, 12, 46, 92, 126, 127, 95, 44, -7, -44, -61, -67, -75, -93, -114, -122, -103, -58, -9, 18, 9, -28, -70, -96, -101, -95, -91, -91, -91, -81, -63, -41, -25, -19, -24, -34, -43, -40, -20, 16, 60, 96, 117, 124, 123, 115, 92, 50, -6, -59, -92, -104, -106, -116, -140, -169, -187, -188, -177, -161, -145, -126, -101, -76, -58, -51, -51, -53, -52, -51, -51, -46, -29, -3, 17, 16, -7, -40, -61, -58, -40, -22, -13, -9, -1, 9, 14, 2, -23, -48, -60, -58, -53, -57, -72, -97, -126, -155, -178, -180, -151, -91, -19, 33, 47, 22, -22, -63, -88, -94, -83, -56, -19, 17, 38, 37, 19, -4, -24, -44, -72, -108, -139, -145, -120, -74, -29, 0, 13, 18, 22, 24, 19, 8, 0, 0, 13, 30, 44, 47, 36, 11, -24, -65, -100, -121, -127, -124, -118, -113, -109, -106, -106, -110, -116, -113, -92, -56, -19, 0, -5, -30, -60, -83, -95, -95, -88, -76, -69, -73, -86, -94, -82, -49, -12, 1, -19, -60, -90, -86, -54, -21, -6, -6, -1, 22, 55, 72, 56, 11, -38, -71, -80, -78, -78, -84, -90, -89, -78, -63, -49, -43, -46, -56, -68, -71, -60, -37, -15, -10, -24, -51, -75, -89, -93, -95, -99, -104, -109, -111, -111, -103, -82, -49, -15, 0, -12, -43, -70, -78, -69, -60, -62, -71, -73, -61, -46, -41, -52, -63, -60, -39, -17, -12, -26, -41, -39, -21, -5, -7, -28, -52, -66, -72, -81, -97, -112, -107, -79, -45, -27, -39, -72, -103, -112, -95, -59, -18, 13, 27, 19, 0, -9, 4, 39, 73, 82, 64, 41, 43, 80, 129, 154, 134, 83, 32, 8, 11, 29, 44, 54, 60, 63, 58, 41, 18, -4, -26, -50, -80, -108, -116, -93, -45, 2, 29, 30, 18, 9, 5, 2, -1, 1, 25, 71, 125, 163, 168, 141, 101, 68, 51, 49, 50, 37, 2, -48, -94, -110, -85, -38, -3, -9, -54, -105, -126, -102, -47, 10, 55, 83, 95, 84, 49, 3, -21, 2, 69, 140, 167, 138, 83, 47, 49, 66, 57, 6, -64, -112, -108, -57, 18, 97, 171, 232, 262, 246, 190, 126, 96, 112, 151, 171, 148, 98, 56, 42, 45, 32, -13, -81, -140, -169, -163, -134, -93, -42, 14, 63, 86, 75, 38, 1, -15, -5, 20, 46, 58, 52, 29, -2, -30, -43, -39, -31, -35, -61, -96, -112, -88, -28, 43, 94, 105, 81, 45, 25, 42, 104, 200, 301, 366, 362, 289, 183, 103, 92, 151, 240, 306, 316, 273, 207, 151, 124, 122, 129, 129, 114, 90, 75, 90, 140, 207, 258, 263, 222, 164, 130, 145, 201, 266, 306, 305, 269, 215, 160, 119, 100, 104, 119, 121, 89, 20, -57, -106, -100, -44, 29, 90, 124, 136, 137, 133, 121, 104, 92, 95, 115, 138, 154, 162, 178, 208, 235, 224, 152, 32, -82, -135, -104, -16, 74, 128, 139, 129, 120, 117, 112, 100, 90, 93, 109, 126, 133, 132, 136, 152, 168, 156, 101, 17, -57, -88, -68, -23, 13, 27, 25, 28, 45, 68, 77, 59, 16, -39, -89, -117, -112, -73, -8, 59, 105, 114, 91, 62, 56, 90, 154, 220, 261, 260, 222, 165, 111, 68, 37, 10, -13, -26, -15, 25, 86, 142, 168, 158, 130, 117, 142, 209, 296, 373, 417, 420, 386, 320, 230, 122, 12, -80, -136, -141, -95, -10, 93, 190, 258, 280, 251, 180, 93, 18, -18, -9, 38, 99, 150, 174, 165, 126, 66, -3, -66, -107, -112, -78, -11, 75, 168, 251, 304, 304, 237, 115, -23, -126, -157, -118, -50, -1, 4, -20, -45, -48, -28, -2, 16, 29, 47, 76, 111, 137, 144, 125, 84, 31, -17, -38, -9, 69, 174, 263, 299, 272, 203, 121, 40, -38, -125, -219, -301, -348, -344, -286, -186, -63, 58, 152, 194, 177, 117, 49, 10, 20, 74, 151, 224, 275, 290, 268, 214, 139, 58, -16, -78, -119, -129, -97, -23, 77, 178, 251, 283, 276, 242, 192, 135, 84, 56, 56, 71, 67, 15, -87, -206, -291, -308, -260, -173, -79, 8, 92, 171, 233, 262, 247, 195, 128, 72, 47, 60, 103, 153, 177, 146, 55, -67, -173, -219, -195, -131, -72, -49, -62, -89, -107, -106, -96, -90, -96, -104, -98, -63, 7, 103, 200, 272, 296, 270, 210, 144, 92, 63, 51, 42, 29, 12, -4, -12, -11, -1, 8, 14, 13, 7, 0, -11, -26, -45, -63, -67, -47, -7, 36, 68, 80, 81, 83, 92, 100, 93, 70, 41, 24, 26, 40, 50, 44, 24, -1, -28, -51, -70, -77, -69, -50, -34, -36, -58, -82, -84, -52, 3, 54, 80, 74, 49, 20, -3, -17, -23, -13, 13, 53, 91, 109, 95, 58, 14, -16, -24, -8, 24, 70, 124, 177, 215, 220, 184, 118, 50, 13, 20, 57, 93, 102, 82, 50, 23, 5, -16, -49, -84, -101, -89, -54, -20, -4, 1, 12, 38, 62, 56, 7, -69, -136, -169, -173, -177, -201, -238, -256, -227, -156, -74, -23, -19, -51, -85, -91, -58, 0, 57, 95, 112, 125, 156, 210, 263, 279, 232, 129, 6, -91, -143, -158, -161, -162, -156, -126, -66, 10, 82, 133, 159, 163, 152, 131, 107, 86, 70, 58, 46, 36, 38, 54, 72, 65, 15, -69, -154, -201, -200, -165, -119, -69, -8, 69, 147, 198, 202, 170, 131, 108, 100, 91, 71, 45, 25, 10, -10, -49, -99, -146, -179, -200, -219, -230, -214, -155, -64, 17, 53, 38, 2, -18, -18, -18, -37, -68, -83, -69, -42, -39, -75, -128, -156, -134, -78, -25, 1, 11, 22, 38, 41, 11, -44, -101, -134, -137, -127, -118, -115, -116, -128, -164, -224, -281, -300, -264, -194, -132, -99, -84, -58, -8, 47, 83, 93, 97, 113, 128, 112, 53, -22, -62, -42, 10, 39, 15, -35, -62, -37, 7, 20, -23, -87, -116, -85, -23, 11, -9, -65, -112, -124, -111, -97, -93, -92, -83, -77, -89, -124, -161, -167, -134, -84, -52, -56, -87, -117, -134, -143, -163, -199, -243, -278, -294, -293, -284, -274, -267, -264, -259, -242, -201, -135, -58, 10, 60, 91, 113, 128, 127, 104, 66, 39, 44, 74, 98, 84, 26, -47, -99, -107, -82, -53, -44, -62, -95, -129, -148, -144, -117, -71, -24, 10, 26, 29, 30, 33, 32, 19, -8, -43, -79, -111, -139, -156, -153, -123, -74, -23, 18, 48, 75, 98, 106, 85, 39, -8, -30, -15, 21, 53, 68, 69, 64, 49, 14, -40, -93, -110, -79, -23, 13, 1, -56, -130, -190, -226, -243, -243, -220, -170, -102, -41, -10, -13, -34, -53, -60, -55, -40, -13, 23, 59, 77, 65, 29, -13, -44, -61, -72, -90, -117, -149, -176, -186, -175, -144, -110, -96, -121, -178, -234, -252, -219, -157, -105, -86, -95, -106, -106, -97, -92, -92, -85, -64, -42, -39, -62, -93, -105, -98, -98, -135, -206, -264, -260, -184, -84, -26, -44, -119, -199, -241, -229, -170, -73, 46, 162, 237, 247, 207, 158, 140, 150, 152, 112, 44, -4, 4, 65, 125, 134, 84, 4, -66, -115, -145, -160, -150, -108, -44, 4, 14, -10, -37, -37, -13, 10, 12, 0, -5, 11, 38, 52, 40, 12, -16, -45, -83, -130, -167, -160, -102, -19, 38, 39, -6, -61, -90, -86, -68, -57, -60, -68, -76, -82, -86, -87, -76, -57, -40, -37, -48, -61, -60, -40, -16, -11, -39, -96, -162, -209, -218, -187, -131, -79, -55, -58, -68, -57, -18, 29, 52, 38, 3, -18, -7, 24, 44, 29, -10, -47, -61, -52, -39, -31, -22, 2, 45, 90, 114, 106, 74, 37, 7, -12, -25, -32, -30, -20, -7, 2, 6, 7, 8, 7, -2, -24, -55, -87, -112, -123, -119, -101, -80, -63, -58, -59, -51, -25, 11, 42, 47, 26, -5, -33, -51, -61, -67, -61, -36, 4, 41, 55, 52, 56, 88, 133, 153, 117, 38, -33, -55, -25, 11, 12, -24, -58, -48, 5, 55, 54, -5, -84, -131, -124, -80, -34, -3, 14, 27, 36, 37, 34, 39, 64, 100, 126, 126, 101, 70, 51, 48, 52, 55, 57, 61, 59, 38, -7, -69, -123, -153, -158, -151, -141, -123, -86, -33, 16, 43, 46, 39, 39, 48, 52, 36, 6, -18, -20, 1, 28, 42, 40, 34, 37, 52, 71, 84, 83, 68, 40, 0, -39, -65, -62, -31, 13, 52, 71, 75, 77, 86, 99, 103, 88, 56, 20, -1, 0, 18, 44, 63, 75, 84, 101, 123, 135, 117, 58, -29, -113, -156, -142, -85, -14, 37, 56, 46, 20, -8, -32, -44, -42, -30, -17, -9, -7, 1, 28, 71, 115, 137, 126, 89, 50, 26, 22, 25, 20, 5, -10, -12, 8, 47, 90, 119, 121, 97, 55, 8, -29, -55, -68, -72, -64, -38, 9, 71, 132, 169, 170, 136, 86, 38, 7, -4, -1, 13, 39, 75, 117, 152, 172, 173, 160, 144, 132, 122, 108, 84, 51, 15, -15, -34, -37, -22, 5, 41, 74, 92, 95, 88, 79, 70, 56, 31, -1, -27, -29, -6, 29, 56, 62, 54, 49, 56, 68, 74, 69, 63, 74, 104, 135, 142, 113, 64, 26, 24, 54, 95, 126, 144, 160, 187, 218, 233, 212, 154, 75, 6, -32, -35, -10, 29, 65, 86, 85, 65, 39, 20, 18, 36, 64, 95, 122, 140, 142, 120, 73, 11, -41, -62, -41, 8, 63, 101, 114, 107, 87, 59, 26, -5, -20, -3, 47, 114, 167, 180, 153, 107, 72, 65, 78, 90, 79, 43, 2, -17, 1, 49, 101, 129, 120, 83, 38, 3, -18, -31, -44, -55, -60, -53, -33, -5, 24, 53, 75, 86, 79, 57, 29, 8, -2, -7, -16, -29, -38, -29, 0, 39, 69, 82, 81, 79, 84, 91, 92, 82, 63, 47, 39, 42, 53, 65, 68, 51, 10, -48, -100, -123, -103, -52, 8, 61, 105, 145, 182, 203, 199, 171, 137, 114, 107, 106, 98, 83, 68, 61, 56, 45, 28, 17, 22, 41, 53, 42, 14, -9, -10, 7, 26, 35, 48, 82, 138, 180, 170, 101, 13, -33, -10, 54, 105, 103, 61, 24, 24, 56, 85, 86, 60, 31, 15, 13, 11, 3, -1, 2, 13, 20, 15, 8, 13, 37, 69, 89, 86, 68, 53, 53, 66, 79, 80, 62, 31, -1, -22, -21, 5, 50, 96, 124, 123, 91, 37, -22, -69, -91, -83, -57, -36, -33, -44, -47, -24, 19, 52, 40, -22, -105, -167, -180, -153, -113, -82, -65, -60, -64, -78, -94, -94, -67, -23, 11, 18, 11, 25, 79, 146, 170, 111, -6, -111, -139, -87, -11, 26, 9, -31, -51, -35, -2, 23, 41, 60, 79, 71, 16, -75, -159, -181, -130, -39, 39, 78, 85, 80, 74, 58, 28, -7, -29, -32, -25, -25, -32, -31, -7, 29, 54, 48, 19, -7, -18, -22, -39, -74, -106, -106, -65, -8, 27, 24, -7, -36, -40, -16, 21, 56, 71, 55, 0, -78, -150, -179, -154, -101, -64, -69, -99, -112, -78, -14, 34, 30, -19, -78, -108, -108, -105, -124, -161, -189, -187, -162, -142, -149, -178, -202, -200, -179, -167, -185, -225, -255, -239, -171, -81, -14, -2, -41, -101, -142, -146, -126, -108, -116, -154, -204, -242, -253, -239, -212, -184, -163, -153, -155, -170, -197, -230, -260, -276, -278, -271, -261, -252, -241, -227, -211, -202, -209, -232, -260, -271, -251, -198, -127, -57, -4, 29, 49, 57, 56, 45, 30, 17, 7, -4, -27, -59, -84, -81, -46, -3, 10, -28, -109, -188, -224, -202, -142, -80, -41, -23, -13, -1, 8, 10, 1, -13, -30, -49, -69, -85, -84, -64, -38, -34, -68, -130, -180, -186, -147, -96, -74, -94, -132, -149, -125, -71, -21, -5, -27, -74, -121, -151, -154, -129, -82, -31, 0, 0, -27, -57, -66, -51, -30, -24, -44, -78, -109, -127, -133, -135, -135, -134, -135, -145, -166, -190, -199, -183, -150, -121, -112, -114, -107, -74, -26, 8, 8, -22, -62, -93, -116, -142, -175, -205, -215, -206, -198, -206, -223, -222, -185, -124, -77, -79, -124, -176, -202, -197, -186, -192, -213, -226, -210, -170, -126, -99, -87, -82, -82, -94, -117, -137, -132, -97, -55, -40, -68, -119, -158, -162, -142, -126, -132, -156, -182, -203, -219, -232, -228, -195, -137, -80, -51, -56, -72, -68, -37, 0, 22, 20, 11, 9, 11, 7, -4, -7, 11, 44, 57, 25, -39, -96, -108, -80, -50, -52, -82, -100, -72, -3, 66, 98, 84, 46, 11, -7, -16, -22, -30, -43, -68, -110, -157, -187, -183, -154, -123, -110, -113, -112, -95, -68, -47, -33, -10, 31, 78, 97, 66, 7, -35, -29, 7, 33, 18, -19, -39, -22, 9, 14, -24, -80, -109, -93, -54, -31, -37, -60, -83, -108, -147, -194, -219, -187, -100, -1, 56, 56, 24, -1, -3, 2, -1, -15, -25, -24, -24, -43, -80, -107, -100, -62, -20, 0, -2, -6, -1, 5, 1, -13, -23, -17, -7, -16, -52, -90, -100, -77, -48, -44, -61, -59, -8, 74, 129, 106, 13, -90, -141, -124, -67, -6, 40, 70, 78, 51, -8, -70, -91, -55, 4, 37, 21, -20, -50, -61, -77, -116, -160, -166, -117, -55, -37, -79, -121, -89, 29, 163, 219, 171, 86, 51, 89, 143, 142, 82, 28, 37, 93, 125, 89, 22, 6, 85, 208, 272, 214, 62, -100, -213, -271, -298, -293, -235, -119, 3, 58, 11, -96, -178, -178, -103, -7, 66, 105, 106, 59, -40, -155, -207, -129, 59, 258, 349, 295, 157, 35, -19, -27, -35, -51, -45, 4, 68, 90, 49, -13, -32, 14, 81, 96, 33, -67, -140, -154, -126, -90, -49, 9, 90, 155, 160, 100, 20, -18, 4, 51, 70, 40, -11, -40, -20, 43, 133, 231, 318, 367, 351, 268, 152, 50, -6, -26, -28, -16, 23, 97, 173, 204, 173, 121, 113, 178, 273, 323, 286, 191, 106, 70, 70, 65, 45, 38, 71, 128, 157, 125, 66, 54, 131, 258, 338, 303, 171, 34, -15, 44, 163, 258, 278, 217, 103, -20, -108, -121, -49, 81, 208, 273, 256, 184, 104, 48, 19, 9, 13, 34, 69, 103, 119, 119, 125, 155, 203, 236, 222, 155, 62, -18, -60, -54, -3, 79, 170, 241, 266, 243, 197, 164, 159, 175, 187, 187, 185, 193, 205, 194, 136, 47, -25, -33, 32, 130, 192, 166, 50, -100, -209, -215, -106, 69, 230, 310, 289, 209, 135, 111, 138, 180, 203, 194, 166, 133, 102, 79, 71, 85, 110, 120, 87, 15, -57, -78, -34, 40, 88, 74, 8, -62, -102, -106, -89, -65, -28, 29, 101, 159, 173, 135, 62, -14, -66, -73, -32, 45, 133, 199, 223, 208, 174, 137, 96, 41, -16, -39, 9, 126, 252, 309, 262, 142, 20, -58, -99, -139, -190, -225, -205, -127, -41, -4, -30, -84, -119, -124, -125, -149, -182, -184, -133, -59, -20, -48, -109, -133, -80, 20, 97, 99, 41, -24, -62, -82, -119, -182, -233, -220, -132, -17, 58, 63, 24, -2, 7, 33, 28, -29, -113, -167, -152, -74, 24, 96, 123, 114, 98, 96, 114, 140, 154, 140, 93, 25, -34, -58, -33, 32, 112, 176, 201, 181, 127, 64, 21, 14, 39, 68, 67, 23, -43, -96, -108, -86, -59, -48, -43, -10, 75, 194, 284, 282, 180, 33, -69, -73, 10, 114, 160, 119, 20, -72, -109, -75, 6, 102, 184, 235, 241, 190, 84, -49, -159, -194, -141, -32, 70, 123, 125, 108, 106, 122, 131, 109, 65, 33, 41, 81, 118, 121, 97, 86, 117, 177, 214, 179, 73, -51, -131, -128, -55, 47, 141, 205, 230, 210, 147, 63, -6, -24, 15, 91, 158, 183, 158, 100, 30, -31, -72, -81, -55, 0, 66, 122, 145, 121, 52, -41, -124, -155, -115, -22, 71, 121, 124, 120, 162, 264, 386, 463, 450, 358, 239, 142, 91, 81, 91, 102, 93, 53, -11, -68, -75, -17, 79, 156, 181, 165, 154, 169, 190, 169, 94, 7, -26, 17, 102, 167, 182, 174, 182, 210, 222, 185, 111, 52, 44, 71, 91, 85, 85, 131, 208, 245, 182, 40, -79, -80, 37, 183, 254, 229, 171, 147, 158, 149, 83, -9, -63, -51, -17, -21, -62, -70, 20, 185, 312, 303, 164, 10, -42, 22, 125, 176, 166, 157, 208, 299, 352, 312, 202, 98, 58, 76, 103, 101, 72, 36, 3, -38, -89, -129, -132, -89, -28, 20, 43, 50, 48, 27, -19, -76, -102, -73, -8, 40, 41, 16, 16, 64, 124, 134, 73, -14, -62, -52, -26, -42, -105, -162, -157, -95, -30, -10, -21, -14, 39, 106, 126, 75, -6, -67, -92, -122, -189, -266, -285, -204, -71, 12, -23, -150, -269, -299, -245, -177, -156, -182, -207, -190, -130, -55, 17, 87, 150, 188, 178, 127, 70, 42, 45, 49, 32, 9, 14, 57, 103, 101, 36, -45, -84, -60, -18, -24, -103, -219, -305, -318, -258, -151, -32, 63, 99, 48, -71, -190, -222, -132, 24, 140, 136, 26, -101, -160, -131, -60, -13, -19, -62, -112, -152, -179, -193, -187, -157, -108, -51, 4, 49, 67, 37, -41, -135, -189, -169, -104, -68, -115, -225, -313, -299, -181, -36, 45, 30, -44, -121, -172, -207, -235, -245, -212, -137, -54, -4, 4, -1, 1, 8, -8, -61, -121, -142, -111, -61, -42, -65, -90, -64, 22, 129, 195, 193, 141, 71, 4, -47, -60, 3, 156, 353, 496, 493, 325, 64, -173, -305, -316, -246, -150, -66, -12, 8, 7, 8, 37, 96, 153, 163, 98, -29, -171, -280, -326, -306, -226, -97, 56, 197, 284, 301, 263, 203, 144, 89, 33, -15, -41, -38, -24, -32, -73, -121, -127, -58, 74, 222, 326, 348, 282, 149, -9, -139, -193, -156, -61, 25, 49, 1, -74, -118, -94, -9, 96, 178, 199, 140, 15, -132, -236, -248, -174, -78, -26, -39, -72, -61, 10, 97, 137, 117, 88, 115, 204, 292, 298, 195, 32, -110, -189, -212, -214, -209, -186, -136, -74, -29, -9, 10, 57, 120, 152, 106, -12, -138, -200, -176, -109, -65, -71, -95, -77, 19, 169, 303, 350, 284, 135, -24, -123, -122, -35, 75, 147, 144, 81, 6, -37, -35, 2, 52, 95, 115, 101, 47, -30, -102, -134, -116, -68, -25, -12, -30, -67, -110, -150, -170, -150, -82, 9, 75, 68, -12, -122, -205, -232, -209, -155, -75, 34, 153, 226, 191, 41, -156, -293, -302, -205, -90, -45, -95, -194, -274, -287, -225, -109, 20, 114, 123, 29, -135, -293, -367, -331, -227, -123, -60, -36, -26, -21, -35, -79, -133, -158, -137, -92, -67, -79, -93, -61, 25, 113, 125, 26, -131, -248, -249, -134, 32, 170, 237, 234, 189, 133, 90, 72, 70, 53, -19, -158, -324, -440, -441, -331, -178, -75, -69, -140, -223, -258, -228, -152, -62, 14, 60, 63, 26, -26, -58, -42, 19, 93, 141, 146, 120, 80, 31, -37, -130, -229, -298, -313, -285, -245, -222, -211, -182, -113, -16, 63, 81, 25, -77, -177, -229, -212, -133, -17, 95, 162, 155, 76, -37, -129, -151, -93, 10, 103, 141, 119, 64, 17, -1, -1, -8, -38, -81, -101, -73, 0, 76, 107, 76, 12, -29, -10, 61, 134, 155, 111, 38, -12, -9, 36, 89, 113, 93, 42, -17, -61, -64, -12, 83, 186, 241, 212, 111, 0, -46, 6, 129, 245, 277, 193, 18, -182, -339, -399, -342, -184, 29, 230, 349, 344, 225, 52, -91, -146, -104, -15, 46, 32, -51, -147, -182, -116, 37, 224, 382, 462, 445, 330, 140, -77, -252, -315, -241, -70, 110, 219, 228, 174, 121, 109, 130, 148, 132, 82, 22, -17, -19, 12, 62, 107, 122, 98, 46, -4, -26, -14, 10, 21, 7, -18, -37, -40, -27, 8, 80, 191, 311, 380, 352, 242, 121, 68, 103, 178, 228, 231, 222, 245, 294, 312, 256, 143, 47, 25, 60, 77, 23, -72, -122, -68, 56, 150, 126, -10, -178, -295, -334, -323, -285, -210, -76, 99, 249, 300, 236, 111, 11, -10, 36, 112, 181, 227, 236, 184, 57, -113, -247, -259, -123, 104, 317, 431, 427, 351, 264, 193, 125, 28, -111, -270, -398, -443, -377, -215, -3, 208, 380, 495, 537, 483, 314, 51, -227, -405, -396, -209, 43, 210, 201, 34, -179, -314, -302, -157, 47, 225, 309, 264, 105, -108, -287, -352, -268, -67, 169, 353, 426, 382, 252, 89, -49, -126, -125, -65, 16, 81, 108, 98, 63, 13, -47, -113, -173, -213, -218, -184, -114, -23, 66, 128, 147, 127, 101, 108, 165, 247, 304, 294, 216, 110, 28, -6, -6, -1, -15, -45, -73, -83, -63, -9, 80, 201, 333, 433, 452, 364, 195, 17, -81, -46, 111, 314, 470, 517, 459, 346, 232, 143, 85, 57, 67, 111, 150, 122, -7, -202, -359, -377, -238, -30, 119, 132, 31, -92, -154, -120, -10, 140, 297, 423, 465, 390, 214, 16, -109, -119, -52, 16, 35, 11, -14, -13, 11, 43, 73, 100, 98, 29, -124, -310, -419, -359, -139, 131, 323, 376, 323, 233, 143, 44, -67, -159, -183, -143, -113, -177, -337, -497, -533, -404, -184, 3, 85, 78, 35, -22, -113, -239, -345, -353, -243, -91, -8, -46, -137, -159, -39, 184, 405, 530, 534, 431, 240, -22, -306, -512, -547, -400, -176, -21, -14, -110, -189, -162, -34, 116, 216, 236, 186, 78, -78, -251, -380, -401, -303, -141, 5, 97, 153, 218, 303, 374, 387, 328, 221, 90, -57, -220, -367, -428, -345, -127, 134, 315, 337, 213, 24, -132, -190, -133, 15, 196, 334, 366, 278, 123, -13, -76, -80, -90, -157, -260, -317, -253, -63, 174, 345, 374, 256, 52, -151, -284, -307, -218, -47, 153, 324, 414, 390, 243, -2, -287, -513, -574, -408, -43, 395, 733, 832, 661, 311, -54, -286, -319, -191, -16, 95, 87, -12, -134, -212, -230, -217, -210, -216, -207, -152, -53, 46, 97, 79, 18, -36, -53, -41, -43, -96, -201, -312, -359, -292, -126, 60, 160, 109, -65, -252, -322, -220, -2, 193, 226, 38, -317, -701, -949, -944, -667, -221, 210, 442, 387, 107, -226, -444, -476, -373, -238, -143, -90, -57, -50, -109, -255, -446, -581, -576, -427, -214, -40, 29, -2, -99, -218, -316, -352, -296, -158, 2, 106, 111, 46, -23, -57, -80, -152, -292, -450, -532, -480, -310, -97, 83, 193, 225, 180, 67, -84, -219, -279, -254, -186, -134, -117, -100, -34, 78, 183, 211, 153, 72, 54, 144, 303, 441, 469, 351, 120, -139, -323, -349, -203, 34, 231, 272, 143, -63, -219, -249, -177, -74, 14, 106, 239, 394, 477, 385, 94, -298, -630, -773, -703, -484, -210, 45, 242, 352, 364, 300, 228, 227, 322, 447, 484, 350, 67, -240, -434, -455, -351, -223, -139, -95, -44, 38, 125, 159, 114, 24, -40, -35, 23, 75, 68, 1, -79, -116, -74, 46, 227, 430, 594, 645, 531, 271, -28, -223, -228, -81, 74, 87, -90, -379, -628, -725, -655, -479, -275, -87, 63, 161, 171, 76, -90, -242, -284, -173, 45, 263, 377, 351, 230, 109, 61, 100, 174, 206, 143, -10, -209, -391, -513, -555, -518, -414, -269, -122, -19, 19, 10, 0, 30, 104, 188, 239, 234, 178, 88, -19, -126, -193, -176, -51, 142, 316, 375, 288, 109, -61, -144, -131, -75, -49, -96, -207, -330, -394, -340, -155, 108, 348, 457, 396, 218, 42, -22, 48, 197, 321, 343, 243, 54, -165, -361, -495, -551, -534, -461, -350, -214, -73, 30, 43, -63, -250, -418, -471, -390, -249, -156, -160, -210, -207, -94, 91, 251, 305, 244, 119, -7, -93, -116, -69, 22, 96, 73, -72, -266, -359, -246, 35, 306, 378, 199, -103, -328, -352, -213, -53, 8, -40, -120, -146, -90, 7, 82, 85, 6, -126, -257, -321, -289, -184, -81, -49, -92, -142, -117, 4, 154, 222, 142, -60, -291, -445, -468, -374, -223, -89, -31, -71, -180, -295, -358, -355, -318, -290, -285, -279, -249, -207, -193, -222, -260, -234, -109, 73, 216, 233, 109, -97, -304, -443, -471, -373, -184, -2, 40, -130, -440, -679, -635, -262, 264, 654, 701, 406, -48, -447, -665, -697, -602, -450, -302, -210, -193, -206, -149, 51, 362, 634, 703, 509, 146, -217, -456, -545, -529, -446, -306, -124, 48, 139, 117, 23, -61, -86, -62, -29, -12, -7, -22, -91, -249, -468, -637, -629, -406, -52, 271, 438, 419, 276, 102, -12, -5, 146, 390, 588, 581, 308, -123, -484, -592, -435, -171, 20, 83, 91, 139, 230, 274, 199, 44, -62, -22, 148, 336, 429, 396, 284, 151, 35, -35, -21, 110, 330, 532, 594, 477, 264, 101, 80, 173, 269, 271, 160, -1, -127, -160, -89, 53, 203, 282, 244, 112, -27, -88, -51, 22, 60, 57, 83, 208, 418, 604, 654, 548, 364, 203, 103, 35, -33, -87, -84, -19, 48, 54, 1, -30, 34, 180, 302, 291, 132, -89, -267, -353, -364, -323, -226, -69, 107, 214, 182, 29, -142, -233, -226, -182, -168, -194, -217, -211, -198, -221, -284, -339, -331, -252, -149, -64, 0, 70, 143, 174, 121, 0, -109, -130, -60, 34, 88, 94, 90, 99, 90, 23, -85, -157, -121, 0, 94, 60, -97, -275, -362, -341, -280, -241, -212, -129, 30, 203, 284, 223, 67, -80, -152, -147, -95, -18, 80, 174, 209, 148, 23, -66, -39, 95, 238, 292, 236, 129, 31, -47, -129, -208, -228, -141, 20, 154, 174, 96, 20, 30, 113, 174, 148, 67, 26, 80, 179, 220, 145, 0, -124, -166, -136, -79, -14, 62, 141, 178, 138, 42, -34, -34, 15, 26, -53, -172, -207, -85, 135, 309, 323, 198, 55, 6, 72, 189, 284, 316, 281, 180, 27, -128, -207, -157, -16, 108, 118, 14, -109, -160, -124, -69, -57, -85, -102, -86, -79, -133, -237, -304, -256, -112, 13, 16, -106, -252, -316, -274, -188, -123, -95, -78, -67, -87, -157, -238, -254, -162, -11, 92, 76, -38, -168, -246, -263, -245, -198, -102, 41, 181, 238, 178, 47, -73, -138, -170, -203, -224, -176, -31, 159, 287, 285, 180, 64, 0, -29, -66, -116, -123, -40, 110, 244, 295, 277, 250, 242, 208, 87, -95, -225, -184, 28, 289, 446, 444, 344, 244, 183, 140, 93, 63, 89, 165, 233, 242, 212, 214, 290, 395, 437, 370, 233, 104, 6, -94, -220, -305, -233, 34, 382, 612, 608, 445, 310, 327, 437, 464, 286, -48, -371, -529, -476, -250, 86, 473, 812, 961, 814, 406, -68, -374, -398, -212, 17, 169, 233, 252, 243, 184, 73, -24, -30, 75, 223, 324, 357, 369, 405, 438, 407, 304, 217, 247, 385, 494, 430, 188, -70, -161, -41, 165, 292, 277, 186, 109, 66, 17, -48, -69, 17, 174, 265, 179, -46, -239, -235, -24, 252, 444, 510, 514, 513, 483, 363, 151, -57, -148, -97, 14, 86, 91, 81, 107, 156, 172, 127, 50, -15, -74, -170, -313, -439, -452, -323, -127, 26, 97, 144, 238, 366, 433, 365, 204, 77, 69, 141, 181, 128, 47, 51, 176, 328, 370, 256, 65, -88, -163, -201, -250, -288, -245, -96, 92, 220, 250, 242, 275, 358, 416, 372, 232, 81, 5, 22, 95, 188, 290, 390, 444, 403, 277, 145, 96, 123, 123, -1, -226, -399, -369, -137, 136, 263, 178, -29, -215, -299, -289, -219, -96, 85, 284, 401, 359, 171, -52, -197, -223, -166, -78, 22, 134, 229, 259, 208, 134, 125, 202, 279, 230, 14, -269, -460, -469, -338, -179, -70, -14, 11, -5, -99, -254, -370, -327, -99, 208, 426, 443, 269, -7, -290, -499, -559, -411, -70, 319, 543, 460, 124, -241, -422, -368, -199, -67, -20, -2, 38, 60, -31, -267, -550, -727, -708, -523, -276, -44, 148, 292, 356, 311, 179, 39, -43, -80, -141, -250, -339, -296, -92, 161, 298, 234, 35, -167, -298, -376, -432, -439, -330, -117, 66, 54, -184, -492, -641, -531, -262, -26, 78, 103, 157, 263, 337, 293, 146, -5, -91, -128, -171, -218, -193, -49, 139, 213, 70, -226, -503, -624, -599, -521, -441, -314, -93, 158, 290, 207, -4, -137, -52, 176, 331, 260, 10, -223, -293, -214, -107, -62, -63, -53, -27, -39, -118, -222, -283, -302, -355, -485, -602, -534, -201, 253, 552, 512, 221, -41, -45, 193, 457, 531, 370, 89, -168, -333, -385, -300, -69, 234, 434, 369, 44, -331, -506, -386, -108, 95, 97, -45, -190, -239, -191, -75, 102, 335, 552, 631, 508, 257, 49, -10, 18, -5, -136, -251, -174, 118, 424, 469, 160, -322, -675, -719, -509, -238, -41, 72, 152, 199, 165, 43, -88, -131, -78, -34, -112, -311, -503, -557, -457, -298, -180, -112, -43, 51, 122, 105, 5, -100, -146, -149, -180, -270, -363, -378, -311, -245, -252, -293, -245, -33, 252, 408, 288, -57, -425, -623, -596, -423, -204, 8, 188, 281, 218, -1, -265, -414, -376, -230, -117, -108, -141, -104, 31, 179, 229, 161, 55, -1, -8, -33, -125, -243, -304, -258, -137, -2, 116, 224, 310, 315, 180, -66, -304, -397, -311, -135, 6, 59, 53, 43, 40, 18, -25, -55, -37, 1, -9, -104, -224, -267, -191, -68, -10, -56, -127, -112, 12, 156, 215, 177, 121, 116, 136, 90, -66, -262, -370, -321, -167, -16, 59, 57, -11, -155, -369, -569, -599, -364, 48, 387, 422, 141, -239, -449, -379, -132, 94, 185, 152, 65, -37, -149, -245, -272, -203, -93, -46, -111, -229, -289, -251, -190, -208, -310, -379, -293, -54, 207, 352, 360, 321, 319, 352, 351, 283, 178, 74, -49, -241, -473, -599, -464, -79, 336, 509, 348, 18, -215, -225, -108, -53, -137, -266, -296, -186, -13, 144, 291, 464, 620, 633, 419, 80, -137, -54, 256, 524, 515, 246, -46, -141, -15, 166, 243, 196, 124, 103, 115, 108, 82, 87, 141, 181, 134, 29, 8, 186, 509, 767, 785, 579, 333, 208, 186, 127, -49, -249, -298, -136, 108, 243, 200, 83, 34, 74, 106, 57, -6, 53, 273, 519, 600, 469, 274, 210, 324, 478, 511, 405, 286, 272, 347, 403, 381, 343, 375, 453, 441, 251, -25, -162, -21, 299, 537, 505, 253, 2, -79, -5, 98, 166, 253, 435, 673, 812, 731, 471, 196, 29, -52, -150, -303, -426, -409, -246, -52, 70, 133, 230, 393, 519, 459, 188, -136, -307, -249, -67, 82, 141, 174, 244, 313, 292, 174, 86, 175, 433, 671, 678, 411, 36, -236, -326, -306, -264, -196, -33, 226, 454, 497, 327, 82, -69, -86, -64, -107, -194, -213, -97, 80, 179, 150, 93, 144, 314, 462, 432, 231, 31, 8, 182, 409, 532, 510, 421, 342, 277, 192, 100, 78, 184, 370, 518, 542, 471, 395, 356, 299, 150, -76, -259, -276, -127, 73, 206, 252, 282, 352, 443, 494, 489, 476, 502, 538, 501, 353, 160, 27, -16, -54, -178, -355, -424, -246, 131, 480, 571, 368, 48, -169, -208, -155, -138, -180, -204, -138, 2, 139, 226, 282, 342, 385, 349, 216, 69, 32, 144, 310, 375, 266, 54, -122, -190, -183, -179, -203, -215, -175, -103, -61, -72, -88, -42, 73, 190, 237, 212, 178, 185, 209, 190, 104, 3, -48, -52, -68, -134, -187, -111, 134, 424, 544, 375, 9, -314, -408, -284, -125, -108, -264, -477, -605, -592, -478, -335, -206, -104, -30, 21, 70, 146, 236, 263, 123, -204, -595, -836, -770, -422, 11, 306, 356, 211, 2, -167, -265, -311, -326, -326, -325, -335, -352, -355, -335, -325, -378, -496, -588, -515, -211, 231, 593, 675, 450, 72, -241, -359, -297, -161, -49, -6, -23, -72, -100, -39, 162, 469, 738, 787, 528, 60, -379, -572, -458, -159, 128, 273, 262, 165, 65, 22, 70, 225, 456, 675, 765, 652, 359, -6, -320, -509, -556, -470, -265, 19, 299, 466, 460, 327, 188, 142, 178, 193, 95, -102, -286, -343, -254, -93, 53, 160, 247, 311, 292, 128, -158, -436, -542, -413, -143, 89, 150, 25, -193, -391, -493, -466, -316, -87, 118, 185, 55, -192, -374, -328, -37, 344, 613, 652, 498, 272, 81, -38, -88, -77, -17, 39, 16, -126, -321, -422, -321, -44, 255, 418, 396, 250, 57, -164, -423, -662, -736, -520, -59, 406, 589, 374, -84, -487, -603, -415, -78, 222, 386, 396, 280, 84, -110, -204, -146, 2, 92, 10, -214, -424, -467, -323, -94, 112, 287, 477, 673, 749, 561, 109, -406, -715, -677, -384, -74, 58, -21, -226, -437, -570, -562, -370, -18, 354, 531, 374, -45, -466, -628, -473, -184, -12, -78, -297, -490, -536, -429, -223, 31, 282, 432, 352, -4, -521, -945, -1058, -838, -463, -159, -34, -48, -103, -156, -217, -276, -268, -129, 108, 306, 328, 160, -69, -210, -211, -142, -99, -97, -74, 20, 151, 226, 183, 57, -62, -123, -153, -211, -314, -409, -427, -350, -227, -127, -88, -100, -143, -201, -261, -286, -236, -102, 64, 184, 204, 137, 51, 4, 8, 33, 43, 21, -26, -88, -146, -178, -161, -97, -25, -5, -75, -214, -357, -432, -401, -271, -79, 119, 259, 278, 147, -88, -320, -436, -389, -233, -81, -22, -71, -174, -253, -259, -187, -71, 34, 74, 17, -121, -280, -385, -400, -344, -268, -210, -168, -128, -86, -56, -35, 1, 78, 169, 196, 89, -137, -377, -499, -460, -327, -208, -156, -139, -95, -17, 34, 8, -73, -124, -82, 23, 87, 20, -172, -396, -549, -584, -516, -379, -197, 4, 172, 236, 156, -36, -249, -379, -367, -217, 10, 232, 351, 305, 99, -167, -351, -349, -178, 34, 145, 98, -41, -149, -130, 26, 263, 487, 608, 561, 334, 0, -298, -420, -321, -90, 113, 173, 94, -24, -83, -55, 16, 80, 117, 140, 157, 158, 129, 72, -2, -81, -151, -175, -97, 114, 411, 662, 736, 600, 351, 140, 50, 46, 34, -34, -121, -145, -77, 36, 117, 121, 62, -27, -121, -195, -201, -93, 115, 324, 397, 272, 27, -169, -192, -45, 154, 293, 333, 306, 247, 161, 40, -84, -157, -146, -82, -20, 18, 62, 137, 205, 176, 8, -230, -396, -390, -247, -97, -34, -33, 1, 121, 271, 348, 312, 221, 161, 154, 152, 110, 58, 65, 144, 209, 157, -22, -216, -284, -198, -60, 7, -21, -61, -28, 61, 117, 83, 10, 7, 116, 267, 349, 318, 227, 144, 66, -62, -248, -392, -359, -120, 192, 378, 337, 134, -84, -228, -311, -374, -405, -343, -189, -45, -43, -203, -383, -399, -200, 80, 248, 219, 74, -47, -93, -119, -186, -264, -258, -122, 64, 164, 119, 6, -42, 17, 112, 128, 43, -61, -88, -24, 66, 120, 135, 148, 164, 147, 80, 23, 73, 257, 473, 550, 400, 101, -168, -273, -212, -81, 33, 105, 135, 110, 9, -133, -214, -144, 54, 236, 253, 92, -109, -187, -97, 63, 164, 167, 139, 155, 219, 275, 280, 249, 209, 149, 25, -155, -301, -301, -147, 40, 114, 52, -20, 33, 194, 280, 118, -266, -646, -776, -598, -274, -21, 60, 21, -48, -123, -224, -333, -368, -261, -61, 81, 54, -106, -249, -256, -151, -53, -31, -30, 48, 203, 306, 229, -5, -219, -251, -115, 8, -40, -246, -434, -447, -292, -115, -42, -68, -99, -76, -39, -51, -109, -134, -75, 15, 31, -67, -180, -168, 0, 186, 215, 35, -214, -341, -269, -93, 29, 31, -38, -99, -141, -201, -278, -291, -150, 121, 370, 424, 238, -63, -297, -357, -267, -112, 42, 173, 253, 236, 110, -53, -131, -49, 129, 241, 160, -94, -381, -559, -596, -558, -510, -448, -320, -102, 156, 374, 489, 490, 397, 248, 106, 58, 156, 345, 451, 295, -140, -656, -955, -862, -454, 8, 290, 324, 198, 29, -129, -264, -341, -302, -147, 13, 24, -169, -447, -591, -464, -125, 230, 430, 431, 296, 99, -128, -358, -521, -536, -389, -175, -25, 5, -20, -11, 51, 98, 65, -16, -50, 2, 62, 6, -178, -344, -317, -83, 168, 211, -6, -304, -446, -363, -201, -153, -256, -367, -324, -128, 68, 126, 68, 41, 145, 317, 396, 294, 83, -79, -112, -66, -36, -44, -25, 68, 182, 194, 57, -127, -190, -64, 141, 228, 93, -186, -426, -496, -405, -261, -155, -115, -121, -154, -196, -198, -98, 111, 338, 427, 282, -32, -330, -435, -310, -70, 123, 182, 118, -1, -112, -177, -177, -114, -22, 43, 43, -12, -78, -130, -197, -321, -483, -580, -504, -250, 44, 207, 173, 34, -51, -11, 84, 99, -33, -258, -457, -542, -488, -308, -20, 322, 606, 692, 520, 193, -74, -121, 36, 219, 248, 80, -176, -386, -483, -480, -395, -219, 45, 326, 501, 499, 365, 217, 127, 58, -73, -273, -433, -432, -267, -72, 26, 41, 110, 325, 590, 677, 438, -20, -398, -442, -156, 217, 411, 339, 116, -87, -189, -216, -207, -154, -19, 190, 399, 518, 507, 386, 197, -23, -222, -316, -226, 24, 281, 346, 135, -234, -516, -525, -267, 94, 382, 522, 533, 452, 285, 48, -182, -292, -223, -38, 123, 168, 132, 139, 269, 477, 626, 616, 457, 247, 82, -7, -47, -70, -84, -82, -65, -38, -4, 39, 75, 72, 14, -58, -57, 77, 307, 503, 540, 396, 169, -5, -56, -7, 74, 132, 142, 89, -29, -176, -258, -175, 83, 396, 581, 537, 320, 90, -30, -56, -90, -209, -383, -517, -546, -471, -327, -120, 158, 465, 674, 653, 379, -10, -319, -439, -420, -387, -394, -375, -229, 51, 354, 545, 586, 556, 546, 565, 537, 394, 159, -75, -226, -280, -267, -203, -78, 97, 257, 315, 260, 192, 244, 453, 693, 765, 557, 142, -275, -504, -475, -251, 50, 321, 477, 467, 295, 44, -143, -155, 10, 244, 423, 502, 526, 544, 526, 393, 126, -155, -253, -46, 375, 755, 845, 592, 164, -176, -254, -70, 236, 499, 612, 557, 370, 125, -98, -227, -219, -85, 123, 339, 510, 601, 598, 498, 322, 123, -21, -49, 54, 244, 434, 546, 548, 465, 355, 267, 206, 134, 9, -171, -351, -439, -374, -165, 105, 339, 471, 501, 467, 412, 359, 315, 282, 257, 230, 189, 136, 94, 91, 130, 182, 205, 193, 184, 228, 323, 392, 325, 67, -316, -668, -828, -732, -438, -76, 234, 435, 524, 517, 420, 240, 4, -225, -373, -394, -303, -172, -84, -73, -108, -114, -33, 121, 269, 320, 237, 69, -82, -132, -70, 38, 100, 61, -56, -165, -169, -27, 219, 465, 602, 587, 463, 321, 233, 199, 161, 65, -86, -229, -283, -222, -91, 29, 77, 37, -63, -174, -243, -224, -92, 125, 352, 492, 490, 376, 239, 161, 147, 139, 68, -66, -197, -241, -164, 0, 183, 334, 427, 456, 408, 277, 82, -121, -268, -315, -263, -144, 2, 142, 243, 274, 225, 127, 41, 25, 82, 149, 139, 3, -232, -479, -637, -646, -509, -275, -20, 180, 276, 259, 159, 36, -60, -110, -123, -118, -107, -92, -79, -79, -105, -151, -187, -179, -118, -24, 60, 95, 66, -18, -125, -207, -219, -142, -14, 79, 56, -94, -278, -344, -197, 115, 415, 510, 333, 0, -268, -301, -85, 228, 441, 432, 218, -73, -295, -345, -209, 47, 319, 498, 529, 429, 280, 172, 141, 149, 129, 47, -65, -145, -158, -133, -130, -174, -218, -177, -7, 242, 450, 501, 358, 84, -203, -393, -422, -295, -70, 161, 322, 382, 377, 373, 412, 462, 446, 307, 71, -152, -255, -198, -42, 102, 149, 82, -53, -185, -243, -189, -38, 133, 217, 143, -53, -241, -280, -138, 74, 191, 134, -19, -103, -18, 180, 322, 269, 28, -261, -441, -443, -308, -131, 15, 101, 123, 81, -15, -142, -269, -368, -427, -431, -359, -208, -34, 51, -44, -316, -626, -790, -705, -422, -101, 110, 167, 120, 33, -70, -194, -322, -393, -356, -220, -69, -3, -61, -199, -330, -381, -328, -196, -33, 93, 124, 34, -139, -301, -358, -288, -151, -50, -49, -139, -257, -338, -343, -257, -78, 156, 356, 405, 232, -109, -457, -642, -605, -437, -293, -277, -369, -463, -462, -336, -127, 92, 253, 306, 231, 55, -141, -259, -236, -94, 80, 185, 161, 11, -212, -426, -527, -434, -143, 227, 477, 439, 106, -344, -673, -737, -580, -360, -212, -158, -133, -90, -47, -53, -117, -185, -189, -115, -14, 45, 33, -39, -153, -292, -426, -504, -482, -377, -279, -285, -394, -492, -438, -208, 55, 144, -51, -458, -874, -1119, -1134, -967, -693, -372, -65, 144, 177, 27, -203, -366, -380, -287, -205, -225, -341, -488, -606, -677, -695, -631, -451, -176, 87, 207, 130, -68, -241, -285, -214, -121, -91, -141, -242, -369, -511, -636, -658, -482, -97, 376, 728, 795, 571, 208, -87, -184, -88, 81, 185, 141, -30, -225, -311, -211, 46, 337, 522, 534, 420, 287, 219, 216, 220, 176, 83, -15, -74, -66, 14, 159, 339, 493, 546, 460, 281, 119, 61, 101, 143, 82, -99, -321, -472, -495, -408, -263, -97, 71, 218, 297, 277, 178, 69, 9, -3, -17, -57, -89, -49, 74, 204, 240, 155, 39, 21, 145, 322, 411, 346, 195, 88, 88, 139, 129, 10, -155, -255, -221, -79, 95, 247, 371, 473, 541, 541, 460, 328, 192, 80, -12, -91, -131, -91, 28, 159, 205, 119, -49, -196, -245, -207, -154, -156, -222, -317, -396, -433, -406, -296, -110, 83, 170, 81, -126, -264, -142, 270, 790, 1125, 1071, 647, 63, -429, -692, -730, -633, -488, -342, -211, -100, 1, 121, 281, 455, 564, 525, 321, 24, -245, -407, -448, -399, -278, -71, 221, 538, 770, 837, 755, 627, 555, 561, 575, 505, 316, 49, -212, -394, -443, -328, -56, 296, 589, 686, 559, 344, 252, 409, 736, 990, 938, 545, 3, -396, -463, -206, 202, 562, 753, 762, 646, 474, 292, 120, -40, -189, -306, -345, -257, -44, 205, 353, 297, 60, -207, -328, -215, 64, 347, 494, 467, 334, 198, 126, 122, 143, 132, 48, -107, -272, -348, -261, -27, 225, 332, 204, -89, -365, -446, -287, 0, 234, 290, 168, -30, -183, -219, -140, -11, 91, 118, 69, -6, -44, -13, 61, 118, 109, 41, -34, -76, -88, -100, -121, -115, -26, 146, 320, 380, 260, 8, -247, -383, -354, -201, -5, 159, 252, 251, 161, 18, -114, -181, -174, -145, -163, -255, -374, -437, -382, -208, 35, 280, 468, 547, 485, 293, 48, -130, -150, -8, 194, 329, 325, 211, 76, -11, -39, -39, -34, -26, -18, -31, -99, -227, -366, -440, -391, -219, 31, 309, 567, 752, 803, 676, 401, 99, -72, -27, 176, 379, 426, 284, 52, -121, -154, -69, 42, 101, 89, 42, 5, 5, 40, 91, 129, 127, 74, -7, -71, -72, 0, 105, 190, 215, 187, 131, 60, -37, -174, -332, -457, -499, -451, -360, -288, -264, -264, -241, -175, -85, -17, -1, -32, -83, -126, -143, -119, -44, 80, 218, 302, 274, 134, -45, -161, -145, -18, 126, 192, 136, -2, -145, -229, -242, -217, -193, -191, -201, -199, -165, -95, -6, 77, 141, 189, 237, 295, 346, 347, 260, 89, -99, -219, -211, -85, 89, 241, 336, 386, 407, 391, 313, 171, 10, -96, -103, -34, 34, 41, -12, -68, -62, 14, 116, 186, 207, 203, 206, 210, 175, 65, -110, -282, -367, -315, -139, 92, 291, 389, 359, 227, 57, -77, -133, -111, -53, 4, 50, 106, 185, 269, 311, 280, 194, 114, 90, 110, 110, 34, -105, -225, -239, -139, -5, 50, -13, -141, -224, -192, -74, 31, 41, -52, -179, -253, -238, -170, -116, -126, -200, -293, -350, -337, -258, -157, -92, -96, -150, -191, -153, -21, 145, 249, 226, 95, -49, -111, -63, 34, 81, 8, -174, -388, -534, -549, -430, -231, -33, 89, 110, 54, -21, -80, -133, -229, -401, -605, -726, -647, -342, 89, 470, 653, 601, 393, 157, -10, -84, -97, -102, -131, -186, -242, -267, -247, -199, -157, -145, -143, -99, 24, 198, 326, 311, 147, -60, -159, -71, 146, 338, 363, 198, -57, -261, -320, -242, -105, 0, 23, -28, -104, -140, -96, 10, 107, 110, -4, -174, -278, -228, -37, 186, 323, 329, 250, 174, 155, 183, 209, 191, 122, 18, -93, -179, -206, -156, -47, 64, 122, 118, 108, 159, 281, 395, 386, 200, -104, -376, -474, -347, -67, 224, 392, 364, 164, -102, -293, -299, -113, 150, 317, 263, -2, -344, -597, -666, -570, -397, -230, -99, 0, 76, 122, 124, 82, 2, -101, -214, -308, -337, -276, -149, -32, -8, -98, -238, -329, -311, -215, -127, -110, -158, -208, -200, -132, -55, -36, -112, -269, -450, -577, -585, -452, -228, -15, 90, 61, -30, -70, 17, 205, 392, 484, 461, 381, 310, 266, 213, 111, -32, -160, -204, -138, 5, 166, 289, 349, 348, 306, 249, 198, 159, 119, 56, -39, -147, -224, -231, -166, -71, -5, -4, -56, -109, -101, 7, 206, 444, 643, 732, 672, 474, 187, -118, -381, -563, -651, -644, -551, -384, -170, 49, 231, 349, 409, 437, 453, 436, 333, 98, -248, -599, -802, -761, -500, -162, 86, 161, 101, 19, 5, 61, 121, 113, 19, -121, -251, -329, -339, -285, -184, -75, -6, -6, -58, -95, -50, 89, 270, 403, 418, 293, 50, -259, -561, -761, -773, -577, -256, 39, 181, 151, 37, -48, -60, -40, -46, -83, -98, -48, 39, 83, 23, -110, -226, -245, -162, -38, 68, 138, 173, 164, 84, -69, -244, -359, -375, -336, -324, -377, -436, -393, -190, 117, 394, 517, 446, 232, -30, -247, -345, -294, -124, 58, 125, 9, -225, -420, -431, -242, 32, 249, 340, 344, 329, 307, 231, 63, -155, -314, -320, -168, 51, 221, 265, 180, 20, -133, -198, -122, 80, 309, 422, 328, 64, -215, -342, -250, -19, 192, 262, 171, -4, -165, -238, -206, -96, 32, 115, 107, 14, -103, -176, -173, -127, -98, -122, -187, -252, -296, -324, -342, -333, -262, -122, 43, 164, 205, 189, 175, 200, 245, 257, 201, 85, -51, -173, -269, -341, -389, -410, -404, -377, -330, -263, -188, -137, -149, -231, -334, -382, -341, -248, -184, -182, -193, -124, 56, 261, 347, 229, -32, -278, -365, -273, -92, 54, 96, 31, -103, -259, -374, -380, -240, -1, 214, 282, 172, -25, -164, -150, -3, 178, 297, 315, 245, 123, -6, -91, -88, 3, 118, 155, 64, -102, -224, -205, -63, 88, 135, 51, -96, -220, -273, -268, -229, -163, -72, 27, 93, 100, 62, 30, 42, 97, 158, 189, 173, 111, 12, -104, -193, -205, -124, -4, 68, 44, -48, -131, -157, -161, -206, -298, -353, -268, -44, 189, 266, 133, -99, -253, -236, -113, -35, -98, -266, -420, -453, -342, -132, 100, 285, 359, 283, 77, -171, -357, -423, -401, -357, -316, -233, -60, 178, 371, 415, 310, 161, 78, 73, 63, -35, -209, -358, -390, -295, -138, 10, 121, 184, 170, 53, -138, -294, -293, -118, 105, 200, 87, -149, -339, -369, -254, -89, 53, 166, 262, 312, 272, 145, 20, 3, 114, 264, 331, 279, 177, 119, 128, 155, 156, 154, 194, 258, 243, 57, -247, -475, -430, -97, 332, 612, 628, 459, 266, 146, 82, 30, 2, 57, 217, 410, 520, 487, 354, 204, 78, -44, -182, -295, -313, -218, -83, -6, -12, -30, 24, 145, 227, 186, 56, -18, 75, 305, 505, 516, 320, 44, -156, -206, -130, -1, 125, 210, 210, 90, -134, -371, -498, -451, -278, -100, -25, -75, -193, -292, -308, -218, -45, 148, 277, 278, 166, 36, -8, 57, 156, 185, 117, 37, 56, 189, 331, 343, 184, -52, -227, -274, -242, -227, -268, -318, -294, -155, 70, 307, 476, 517, 405, 174, -64, -169, -62, 203, 465, 572, 492, 319, 166, 65, -30, -154, -258, -251, -105, 92, 209, 195, 126, 121, 217, 326, 331, 209, 56, -8, 44, 150, 233, 289, 373, 510, 645, 687, 596, 421, 248, 125, 43, -15, -37, 0, 77, 114, 40, -120, -255, -256, -121, 47, 143, 151, 150, 217, 347, 465, 504, 463, 385, 296, 188, 55, -65, -114, -65, 34, 114, 147, 166, 210, 259, 246, 139, -3, -75, -27, 77, 124, 59, -59, -138, -152, -163, -229, -310, -291, -102, 182, 395, 418, 289, 150, 98, 100, 54, -80, -224, -259, -150, 23, 156, 225, 290, 392, 474, 418, 170, -186, -485, -611, -575, -475, -390, -329, -265, -194, -144, -126, -108, -44, 61, 152, 171, 133, 109, 156, 261, 373, 464, 562, 683, 769, 704, 426, 19, -325, -448, -337, -106, 121, 304, 437, 476, 330, -15, -406, -569, -331, 207, 715, 890, 698, 376, 205, 274, 438, 503, 413, 271, 201, 219, 246, 216, 159, 151, 205, 248, 202, 86, 8, 61, 214, 337, 322, 174, -1, -103, -115, -88, -71, -74, -85, -107, -149, -188, -168, -50, 127, 263, 276, 180, 72, 32, 50, 52, -2, -60, -24, 137, 345, 465, 423, 260, 76, -59, -142, -192, -200, -132, 17, 190, 307, 333, 292, 225, 124, -52, -311, -572, -704, -633, -404, -141, 59, 201, 347, 522, 666, 685, 547, 322, 125, 21, -5, -20, -67, -140, -212, -268, -305, -307, -248, -120, 36, 143, 142, 50, -56, -102, -80, -49, -82, -200, -351, -439, -394, -206, 54, 277, 365, 292, 119, -48, -141, -169, -210, -315, -450, -504, -393, -147, 88, 160, 33, -177, -289, -188, 86, 375, 506, 405, 139, -123, -210, -49, 279, 580, 647, 397, -62, -500, -691, -557, -190, 216, 487, 535, 366, 61, -255, -447, -419, -190, 96, 246, 148, -138, -420, -508, -351, -59, 199, 313, 285, 179, 50, -87, -231, -354, -407, -357, -215, -41, 89, 128, 67, -51, -154, -169, -71, 86, 185, 116, -144, -500, -787, -866, -710, -414, -139, -19, -97, -297, -470, -485, -318, -83, 42, -46, -293, -511, -527, -327, -61, 87, 59, -43, -71, 36, 204, 307, 287, 184, 75, -12, -101, -207, -300, -337, -317, -282, -260, -214, -80, 133, 304, 281, 34, -285, -451, -354, -93, 119, 146, 27, -87, -90, 9, 130, 216, 252, 210, 29, -308, -678, -840, -610, -61, 483, 679, 435, -27, -375, -437, -307, -187, -167, -163, -52, 155, 304, 259, 52, -133, -139, 26, 203, 229, 67, -193, -437, -583, -586, -411, -61, 353, 626, 577, 208, -259, -525, -432, -77, 277, 401, 235, -107, -439, -603, -516, -193, 236, 554, 556, 200, -325, -700, -698, -350, 84, 326, 276, 53, -142, -193, -111, 26, 160, 262, 301, 238, 73, -115, -211, -156, 0, 140, 199, 210, 240, 288, 265, 88, -202, -455, -534, -458, -383, -445, -605, -676, -502, -122, 242, 373, 236, -6, -161, -169, -126, -152, -254, -326, -266, -97, 53, 73, -22, -100, -40, 152, 335, 344, 122, -239, -566, -713, -642, -420, -168, 3, 32, -86, -289, -470, -513, -354, -29, 336, 587, 624, 461, 208, -9, -124, -154, -163, -205, -286, -377, -444, -477, -475, -438, -353, -213, -38, 124, 217, 201, 77, -103, -264, -322, -238, -38, 183, 315, 298, 169, 43, 32, 165, 359, 466, 369, 53, -380, -763, -941, -844, -523, -133, 141, 178, -20, -324, -556, -595, -435, -170, 73, 206, 206, 102, -49, -185, -250, -219, -114, 0, 59, 39, -33, -104, -133, -113, -67, -35, -60, -174, -374, -590, -698, -590, -266, 127, 382, 365, 114, -190, -366, -347, -202, -47, 44, 66, 30, -67, -226, -401, -504, -465, -305, -124, -32, -60, -162, -272, -367, -459, -536, -543, -419, -182, 64, 199, 179, 73, -8, -8, 50, 110, 141, 152, 136, 42, -185, -519, -811, -863, -573, -38, 486, 740, 619, 242, -141, -314, -225, -6, 134, 71, -152, -361, -390, -213, 39, 194, 179, 74, 16, 62, 136, 108, -64, -291, -418, -367, -193, -21, 63, 72, 69, 96, 155, 227, 298, 341, 289, 78, -271, -612, -749, -599, -280, -34, -23, -197, -351, -320, -118, 102, 213, 225, 230, 270, 275, 150, -89, -299, -334, -195, -25, 33, -14, -33, 98, 354, 582, 643, 528, 329, 137, -19, -139, -178, -74, 153, 352, 324, 6, -429, -675, -522, -12, 599, 1040, 1186, 1068, 777, 387, -33, -381, -535, -442, -171, 131, 354, 481, 548, 544, 400, 77, -331, -628, -659, -462, -249, -231, -434, -672, -702, -423, 50, 475, 644, 503, 154, -219, -454, -470, -306, -93, 28, -7, -146, -265, -275, -176, -36, 88, 202, 324, 417, 390, 171, -184, -506, -610, -437, -87, 266, 483, 516, 384, 141, -123, -284, -229, 41, 370, 529, 395, 57, -233, -270, -47, 254, 440, 445, 347, 248, 177, 101, 21, 6, 136, 394, 655, 773, 703, 516, 319, 161, 27, -95, -162, -109, 63, 265, 373, 334, 196, 48, -70, -177, -296, -400, -431, -368, -257, -167, -106, -13, 170, 412, 595, 613, 468, 275, 147, 103, 68, -28, -166, -255, -224, -101, 17, 57, 26, -12, -17, -7, -27, -87, -145, -146, -79, 17, 99, 157, 213, 284, 353, 389, 382, 350, 312, 261, 171, 40, -91, -158, -130, -36, 59, 117, 140, 150, 138, 63, -106, -332, -507, -527, -373, -129, 76, 171, 184, 212, 332, 532, 717, 771, 641, 362, 38, -213, -317, -258, -82, 131, 295, 348, 275, 116, -61, -200, -282, -335, -393, -459, -492, -434, -270, -47, 146, 238, 218, 138, 64, 36, 54, 101, 167, 246, 320, 352, 305, 183, 35, -71, -99, -63, -8, 35, 76, 133, 203, 240, 197, 77, -51, -91, 10, 219, 434, 556, 547, 442, 307, 190, 104, 37, -12, -34, -12, 51, 140, 227, 289, 301, 238, 88, -124, -327, -430, -379, -193, 45, 251, 389, 472, 521, 528, 464, 328, 175, 83, 82, 113, 80, -69, -287, -457, -489, -380, -196, -9, 150, 269, 316, 245, 55, -167, -288, -228, -43, 112, 118, -9, -132, -123, 26, 203, 281, 225, 101, 2, -33, -26, -12, -7, -11, -31, -66, -82, -25, 107, 225, 179, -93, -464, -671, -520, -73, 373, 524, 327, 10, -133, -8, 214, 281, 98, -190, -352, -292, -117, -5, -25, -71, 7, 255, 562, 765, 786, 657, 457, 231, -12, -242, -379, -337, -113, 192, 441, 539, 485, 328, 116, -117, -322, -423, -356, -137, 120, 281, 278, 166, 60, 47, 120, 207, 248, 243, 230, 232, 225, 166, 51, -62, -88, 14, 192, 323, 296, 98, -167, -345, -334, -157, 62, 191, 180, 85, 13, 42, 172, 346, 489, 548, 489, 309, 44, -215, -353, -294, -73, 176, 300, 227, 13, -211, -335, -329, -235, -119, -34, -12, -66, -183, -304, -348, -264, -75, 130, 271, 327, 344, 363, 373, 318, 172, -10, -126, -106, 19, 150, 193, 134, 27, -63, -110, -114, -87, -32, 29, 64, 49, 13, 32, 152, 330, 438, 363, 114, -170, -321, -274, -106, 40, 84, 53, 30, 63, 121, 138, 80, -36, -167, -280, -355, -367, -281, -96, 125, 280, 299, 207, 111, 106, 195, 286, 287, 186, 64, 8, 27, 47, -11, -139, -228, -149, 129, 499, 768, 775, 497, 59, -339, -538, -496, -304, -120, -63, -147, -271, -295, -138, 166, 488, 680, 665, 463, 162, -129, -327, -377, -263, -21, 251, 418, 382, 154, -130, -291, -220, 39, 327, 473, 412, 207, -17, -164, -218, -218, -208, -212, -230, -254, -274, -268, -208, -89, 55, 166, 194, 131, 10, -131, -267, -369, -396, -310, -122, 77, 169, 98, -64, -165, -97, 102, 274, 268, 71, -197, -395, -465, -449, -399, -310, -143, 89, 278, 289, 91, -185, -334, -226, 83, 404, 548, 453, 195, -87, -273, -308, -204, -31, 108, 124, -3, -201, -345, -353, -243, -109, -44, -72, -150, -220, -240, -181, -22, 225, 479, 602, 500, 217, -71, -186, -89, 102, 246, 322, 435, 665, 923, 982, 685, 123, -396, -586, -401, -57, 165, 140, -49, -239, -323, -299, -204, -65, 77, 144, 59, -149, -315, -256, 43, 372, 442, 143, -342, -683, -669, -368, -21, 175, 210, 187, 173, 136, 18, -159, -307, -373, -404, -489, -629, -707, -605, -357, -147, -137, -289, -385, -236, 118, 421, 420, 86, -362, -659, -690, -539, -376, -310, -351, -451, -546, -560, -418, -106, 258, 463, 348, -44, -478, -697, -625, -397, -202, -116, -82, -45, -52, -200, -478, -692, -605, -165, 405, 762, 700, 302, -167, -487, -609, -603, -528, -380, -167, 25, 69, -85, -353, -570, -630, -539, -363, -137, 122, 348, 390, 128, -372, -836, -961, -662, -154, 224, 297, 159, 43, 82, 204, 251, 166, 49, 28, 107, 163, 87, -86, -212, -177, -18, 118, 126, 19, -113, -210, -282, -354, -391, -323, -135, 78, 177, 110, -33, -102, -27, 117, 197, 143, 0, -143, -247, -327, -391, -393, -273, -51, 149, 195, 61, -135, -248, -230, -167, -178, -291, -422, -470, -416, -325, -255, -190, -70, 110, 253, 225, -16, -361, -615, -641, -450, -180, 10, 33, -100, -315, -506, -588, -519, -331, -119, 15, 39, 8, 16, 98, 186, 162, -30, -318, -539, -550, -330, 14, 323, 475, 433, 246, 10, -166, -208, -107, 63, 185, 171, 26, -158, -277, -292, -252, -231, -249, -253, -165, 29, 263, 429, 469, 411, 333, 282, 232, 118, -82, -297, -396, -285, 0, 300, 440, 344, 97, -113, -135, 48, 315, 498, 501, 356, 181, 87, 110, 193, 243, 198, 64, -91, -184, -170, -68, 56, 142, 179, 212, 290, 411, 501, 464, 264, -29, -278, -367, -283, -111, 30, 75, 43, 7, 45, 188, 412, 637, 756, 680, 395, -5, -353, -492, -382, -126, 97, 158, 44, -146, -291, -319, -228, -53, 166, 391, 575, 660, 608, 420, 147, -139, -368, -482, -447, -272, -30, 161, 208, 108, -21, -26, 165, 472, 695, 662, 351, -86, -421, -491, -286, 60, 356, 458, 329, 49, -232, -376, -321, -107, 153, 344, 402, 333, 187, 26, -92, -119, -37, 125, 282, 340, 259, 93, -48, -84, -14, 99, 201, 290, 396, 514, 573, 493, 268, 2, -159, -139, 19, 201, 313, 333, 286, 191, 52, -102, -190, -115, 146, 502, 790, 900, 837, 685, 512, 317, 76, -173, -318, -258, -18, 234, 312, 154, -126, -338, -358, -204, 11, 189, 293, 327, 289, 175, 18, -93, -81, 56, 222, 308, 296, 277, 353, 528, 687, 700, 549, 338, 193, 133, 69, -87, -305, -440, -357, -60, 303, 562, 646, 604, 520, 424, 286, 97, -87, -185, -175, -125, -134, -214, -266, -158, 140, 512, 764, 774, 567, 284, 62, -40, -52, -27, 3, 38, 72, 84, 50, -26, -98, -86, 85, 434, 873, 1221, 1276, 943, 321, -317, -681, -640, -295, 111, 379, 465, 447, 405, 343, 225, 76, -1, 83, 287, 422, 303, -85, -546, -812, -746, -420, -27, 277, 456, 542, 547, 445, 241, 28, -61, 37, 246, 404, 394, 233, 36, -91, -127, -103, -36, 97, 313, 559, 731, 751, 630, 442, 251, 65, -131, -301, -354, -213, 94, 436, 675, 770, 787, 807, 831, 779, 579, 248, -106, -365, -465, -410, -241, -14, 196, 308, 282, 168, 90, 148, 315, 447, 408, 207, 0, -53, 61, 197, 191, 25, -152, -167, 18, 279, 439, 414, 249, 45, -122, -218, -218, -104, 95, 269, 282, 90, -194, -382, -358, -174, 18, 122, 174, 260, 382, 411, 221, -152, -488, -550, -290, 95, 337, 304, 89, -101, -137, -33, 109, 214, 261, 247, 146, -57, -313, -504, -528, -388, -186, -23, 81, 178, 302, 410, 433, 373, 321, 377, 521, 610, 496, 179, -176, -366, -303, -70, 164, 277, 235, 63, -204, -516, -778, -872, -735, -418, -61, 215, 385, 488, 545, 515, 360, 130, -35, -32, 90, 148, -11, -355, -671, -741, -521, -155, 174, 391, 524, 594, 535, 262, -193, -632, -816, -664, -318, -18, 108, 132, 210, 402, 603, 644, 462, 157, -93, -183, -131, -30, 51, 101, 125, 108, 21, -135, -310, -434, -463, -404, -293, -172, -66, 12, 75, 147, 242, 318, 274, 26, -388, -779, -911, -691, -270, 61, 107, -79, -256, -214, 40, 322, 451, 400, 264, 102, -141, -556, -1092, -1517, -1569, -1174, -527, 59, 377, 432, 345, 199, -12, -299, -575, -686, -556, -284, -90, -129, -370, -630, -737, -654, -469, -286, -145, -36, 41, 64, 26, -25, -25, 32, 70, -11, -235, -512, -708, -747, -649, -475, -256, 2, 266, 431, 383, 109, -253, -483, -433, -135, 219, 402, 289, -88, -572, -959, -1087, -897, -468, 18, 368, 475, 371, 176, 2, -111, -182, -231, -243, -205, -138, -100, -118, -157, -149, -80, -23, -89, -302, -558, -683, -580, -306, -27, 100, 24, -204, -491, -742, -868, -796, -517, -134, 167, 230, 43, -240, -417, -383, -194, 2, 85, 19, -170, -437, -720, -914, -903, -648, -269, 3, -6, -261, -519, -515, -177, 310, 653, 650, 311, -195, -674, -992, -1087, -952, -634, -252, 43, 153, 102, 10, -9, 59, 144, 177, 157, 123, 77, -28, -230, -453, -539, -372, -8, 336, 450, 287, -10, -254, -349, -328, -259, -157, -11, 141, 187, 31, -280, -553, -594, -385, -102, 35, -37, -200, -287, -255, -211, -266, -393, -434, -270, 36, 275, 271, 55, -162, -175, 33, 284, 356, 163, -190, -509, -638, -534, -254, 86, 345, 387, 142, -318, -778, -965, -725, -135, 523, 923, 886, 472, -72, -466, -540, -310, 54, 333, 379, 187, -111, -340, -395, -292, -139, -37, -16, -43, -84, -147, -260, -406, -500, -444, -215, 86, 299, 318, 175, 4, -66, -18, 57, 38, -132, -409, -680, -833, -802, -585, -240, 129, 404, 499, 412, 239, 109, 97, 164, 184, 49, -241, -572, -790, -813, -670, -470, -311, -223, -178, -139, -98, -74, -82, -110, -138, -166, -207, -266, -305, -260, -99, 129, 313, 349, 237, 81, 9, 63, 166, 190, 79, -94, -188, -106, 123, 375, 530, 547, 459, 317, 143, -48, -220, -311, -299, -245, -256, -377, -513, -482, -167, 346, 795, 918, 651, 174, -231, -378, -276, -71, 99, 181, 183, 110, -49, -274, -469, -513, -357, -88, 125, 171, 79, -17, -9, 99, 216, 262, 250, 254, 309, 366, 336, 195, 31, -12, 134, 395, 588, 553, 256, -182, -569, -741, -657, -407, -169, -101, -246, -495, -641, -518, -119, 379, 729, 769, 508, 101, -280, -540, -675, -720, -692, -587, -411, -199, -1, 138, 196, 154, 2, -231, -467, -601, -583, -461, -353, -341, -390, -363, -154, 192, 487, 532, 281, -106, -380, -377, -125, 197, 406, 425, 315, 197, 163, 225, 329, 403, 395, 302, 165, 44, -22, -48, -84, -175, -311, -417, -401, -223, 73, 382, 590, 635, 518, 290, 34, -154, -197, -66, 191, 466, 645, 672, 568, 398, 214, 31, -137, -245, -217, -17, 295, 562, 621, 417, 56, -266, -388, -283, -63, 118, 173, 133, 103, 175, 356, 552, 631, 510, 224, -82, -230, -121, 181, 483, 565, 338, -92, -485, -616, -412, 13, 450, 715, 737, 548, 219, -171, -543, -798, -837, -624, -226, 204, 514, 644, 645, 619, 631, 658, 625, 478, 229, -48, -279, -412, -433, -349, -182, 20, 183, 232, 145, -15, -138, -138, -30, 66, 9, -257, -651, -981, -1064, -837, -405, 19, 238, 166, -124, -456, -646, -593, -318, 64, 409, 601, 590, 391, 72, -267, -520, -599, -471, -193, 98, 237, 124, -195, -530, -637, -378, 169, 730, 980, 743, 112, -596, -1025, -969, -477, 200, 751, 956, 782, 373, -30, -219, -118, 168, 419, 425, 125, -350, -761, -897, -687, -223, 319, 763, 991, 966, 737, 436, 237, 269, 526, 847, 1013, 883, 493, 26, -311, -397, -227, 119, 519, 818, 853, 536, -55, -661, -962, -791, -263, 287, 542, 396, -1, -394, -600, -580, -387, -76, 296, 641, 819, 717, 359, -82, -392, -461, -350, -218, -184, -252, -345, -390, -378, -348, -337, -361, -414, -483, -544, -557, -495, -379, -281, -270, -342, -410, -371, -198, 32, 213, 297, 326, 374, 464, 530, 476, 251, -103, -477, -738, -792, -625, -329, -74, -23, -220, -531, -727, -647, -330, 14, 177, 110, -59, -169, -156, -71, 16, 106, 236, 392, 488, 456, 360, 359, 543, 787, 807, 418, -260, -843, -973, -602, -14, 424, 547, 440, 275, 122, -66, -307, -486, -429, -94, 342, 608, 557, 285, 29, -29, 110, 331, 514, 606, 590, 451, 201, -66, -204, -118, 129, 352, 399, 280, 144, 111, 160, 173, 94, 20, 101, 365, 642, 691, 412, -66, -496, -696, -648, -431, -123, 229, 553, 729, 675, 453, 264, 283, 489, 663, 582, 226, -200, -452, -446, -280, -87, 111, 380, 716, 965, 918, 522, -29, -429, -500, -323, -129, -85, -174, -270, -300, -313, -387, -496, -494, -244, 233, 737, 1015, 933, 551, 59, -344, -548, -551, -432, -291, -195, -143, -73, 72, 281, 454, 470, 285, -29, -346, -570, -684, -702, -614, -381, -9, 401, 689, 746, 591, 345, 122, -43, -161, -200, -79, 247, 670, 940, 831, 325, -330, -784, -817, -479, -35, 235, 213, -19, -263, -336, -180, 121, 397, 491, 364, 120, -57, -41, 155, 396, 547, 578, 567, 606, 697, 757, 710, 568, 417, 331, 298, 249, 135, -16, -133, -171, -145, -95, -32, 61, 183, 263, 211, 5, -248, -374, -256, 60, 388, 517, 357, -5, -365, -513, -355, 42, 478, 723, 635, 243, -275, -701, -889, -814, -543, -162, 246, 591, 766, 696, 416, 79, -143, -191, -162, -210, -398, -617, -687, -522, -208, 69, 185, 158, 92, 55, 8, -131, -386, -673, -860, -852, -646, -306, 74, 396, 566, 527, 300, -6, -264, -402, -438, -420, -347, -169, 125, 440, 597, 493, 204, -53, -112, -2, 75, -52, -368, -662, -718, -487, -107, 229, 417, 464, 407, 259, 28, -218, -349, -273, -33, 202, 272, 163, 16, 6, 189, 458, 633, 600, 379, 91, -132, -230, -231, -224, -285, -412, -505, -432, -147, 223, 446, 351, -1, -341, -392, -109, 272, 429, 225, -185, -502, -530, -304, -3, 242, 429, 595, 702, 633, 333, -75, -349, -319, -41, 247, 335, 207, 22, -49, 16, 114, 130, 38, -109, -270, -432, -571, -601, -423, -38, 394, 640, 557, 212, -181, -423, -460, -371, -263, -183, -125, -80, -57, -55, -35, 48, 176, 245, 151, -109, -410, -578, -526, -303, -33, 191, 349, 453, 498, 462, 357, 236, 147, 61, -113, -432, -811, -1056, -999, -645, -172, 205, 393, 438, 428, 373, 210, -89, -432, -636, -577, -290, 55, 301, 405, 436, 470, 508, 489, 374, 194, 24, -81, -127, -142, -136, -94, -9, 98, 197, 291, 421, 612, 814, 915, 823, 564, 276, 113, 124, 229, 297, 262, 167, 106, 124, 189, 235, 236, 218, 215, 215, 167, 45, -97, -159, -80, 87, 208, 170, -8, -182, -197, -27, 191, 263, 93, -218, -435, -359, 20, 504, 815, 772, 397, -121, -553, -756, -717, -518, -267, -54, 62, 56, -58, -221, -340, -346, -239, -94, -10, -41, -159, -278, -315, -232, -46, 188, 397, 490, 385, 55, -413, -826, -973, -765, -312, 126, 312, 184, -115, -357, -388, -217, 35, 249, 373, 403, 333, 139, -164, -477, -642, -554, -251, 96, 305, 302, 154, -14, -132, -203, -261, -298, -271, -165, -33, 35, 4, -58, -39, 114, 327, 453, 393, 186, -26, -115, -55, 72, 164, 167, 93, -20, -149, -268, -340, -321, -205, -54, 23, -45, -232, -415, -463, -336, -111, 65, 89, -33, -203, -304, -283, -187, -112, -127, -225, -329, -353, -267, -108, 55, 170, 214, 182, 68, -115, -315, -448, -452, -355, -268, -300, -449, -591, -587, -421, -238, -223, -426, -692, -781, -575, -193, 98, 98, -176, -517, -687, -590, -306, -3, 182, 203, 94, -63, -183, -208, -149, -87, -126, -300, -534, -689, -668, -493, -274, -122, -74, -105, -184, -296, -407, -445, -342, -119, 86, 123, -29, -223, -270, -127, 52, 67, -113, -309, -303, -75, 156, 146, -129, -438, -501, -263, 68, 236, 164, -8, -119, -149, -206, -342, -451, -373, -104, 152, 176, -28, -213, -130, 181, 411, 282, -138, -461, -331, 218, 773, 905, 543, 0, -333, -320, -107, 106, 276, 465, 636, 572, 70, -770, -1496, -1599, -928, 163, 1084, 1429, 1213, 733, 261, -130, -493, -815, -973, -853, -491, -61, 270, 460, 559, 588, 487, 204, -182, -473, -504, -312, -117, -131, -360, -602, -632, -395, -35, 251, 379, 400, 398, 387, 312, 144, -50, -150, -89, 83, 241, 275, 175, 10, -130, -199, -194, -140, -69, -7, 27, 23, -8, -45, -52, -2, 110, 248, 327, 248, -21, -395, -681, -704, -447, -90, 115, 43, -199, -366, -284, -1, 257, 301, 135, -51, -67, 111, 343, 462, 432, 349, 326, 376, 412, 353, 222, 126, 160, 313, 484, 579, 587, 577, 621, 713, 766, 680, 421, 52, -301, -535, -609, -554, -416, -207, 97, 496, 896, 1117, 1012, 611, 158, -33, 179, 627, 941, 846, 402, -38, -127, 179, 584, 731, 523, 201, 92, 279, 518, 483, 102, -334, -427, -46, 531, 851, 675, 187, -181, -130, 268, 667, 755, 500, 111, -181, -304, -325, -305, -226, -54, 135, 154, -141, -667, -1107, -1126, -632, 133, 768, 989, 801, 436, 152, 63, 128, 246, 345, 392, 362, 231, 12, -217, -354, -348, -249, -153, -112, -83, 25, 241, 484, 621, 581, 414, 232, 106, 14, -89, -182, -153, 89, 486, 806, 802, 413, -154, -557, -566, -222, 213, 477, 497, 409, 402, 551, 758, 853, 724, 395, -13, -359, -539, -500, -251, 131, 514, 758, 791, 650, 436, 237, 70, -75, -176, -150, 60, 389, 642, 626, 318, -79, -278, -113, 328, 782, 1003, 922, 651, 358, 161, 87, 117, 215, 326, 374, 297, 107, -97, -196, -149, -22, 71, 71, 8, -51, -72, -60, -14, 86, 236, 333, 225, -145, -594, -759, -357, 537, 1475, 1921, 1646, 879, 103, -326, -398, -337, -317, -300, -142, 172, 434, 369, -88, -680, -1001, -814, -227, 396, 708, 596, 198, -244, -536, -583, -373, 45, 553, 953, 1049, 762, 223, -278, -462, -244, 193, 516, 452, -25, -675, -1149, -1220, -917, -473, -134, 8, 45, 113, 258, 401, 443, 385, 323, 333, 365, 276, -8, -382, -600, -475, -44, 447, 734, 712, 477, 203, 11, -75, -74, -2, 115, 205, 176, 6, -206, -298, -186, 44, 206, 177, 9, -116, -49, 206, 520, 752, 839, 789, 630, 385, 106, -105, -158, -46, 129, 252, 303, 364, 512, 699, 768, 597, 231, -140, -342, -352, -275, -203, -104, 117, 474, 813, 920, 704, 293, -69, -222, -189, -111, -92, -115, -111, -69, -63, -172, -375, -548, -559, -378, -99, 129, 187, 39, -269, -622, -867, -876, -625, -241, 62, 124, -47, -284, -401, -342, -215, -174, -284, -459, -545, -445, -193, 73, 217, 176, -10, -235, -391, -433, -398, -358, -349, -328, -215, 4, 221, 260, 37, -333, -603, -586, -316, -20, 85, -11, -131, -92, 115, 330, 398, 325, 259, 331, 513, 650, 604, 371, 55, -237, -455, -568, -524, -277, 122, 497, 644, 496, 185, -72, -151, -97, -32, -22, -31, -17, -19, -117, -305, -447, -380, -99, 198, 289, 139, -52, -48, 169, 366, 282, -105, -542, -732, -605, -356, -227, -271, -331, -241, -15, 168, 160, -4, -148, -137, -1, 129, 171, 158, 162, 171, 88, -146, -450, -633, -566, -296, -8, 136, 123, 48, 0, -25, -89, -226, -399, -544, -622, -633, -580, -466, -320, -209, -194, -252, -267, -139, 91, 234, 111, -265, -666, -827, -670, -356, -118, -54, -83, -92, -98, -228, -546, -922, -1109, -975, -643, -388, -388, -567, -695, -630, -446, -325, -347, -403, -332, -122, 46, -16, -301, -576, -591, -304, 82, 316, 310, 177, 81, 69, 53, -63, -265, -431, -462, -370, -261, -232, -299, -412, -506, -533, -458, -261, 31, 321, 485, 472, 358, 287, 327, 401, 365, 166, -67, -136, 36, 307, 422, 243, -124, -421, -448, -205, 156, 482, 694, 752, 592, 157, -468, -1016, -1169, -828, -245, 133, 15, -486, -939, -946, -461, 199, 632, 647, 356, 8, -219, -296, -261, -140, 45, 212, 207, -66, -525, -910, -968, -650, -166, 168, 159, -158, -579, -884, -963, -829, -566, -280, -63, 27, -5, -108, -211, -273, -293, -269, -172, 14, 219, 281, 57, -417, -891, -1050, -768, -230, 205, 287, 60, -211, -300, -197, -78, -102, -266, -436, -493, -435, -322, -191, -32, 132, 200, 59, -264, -562, -587, -266, 208, 541, 585, 441, 323, 320, 315, 135, -228, -566, -634, -382, -9, 208, 143, -104, -339, -428, -373, -237, -76, 73, 160, 132, 4, -106, -63, 150, 381, 422, 210, -94, -234, -77, 272, 581, 680, 568, 349, 106, -138, -358, -476, -421, -237, -98, -163, -404, -599, -528, -196, 140, 202, -60, -423, -595, -477, -220, -57, -79, -183, -211, -101, 72, 208, 265, 261, 206, 90, -58, -144, -61, 168, 361, 299, -63, -529, -801, -726, -420, -143, -73, -176, -287, -295, -228, -188, -222, -277, -278, -217, -151, -120, -99, -36, 64, 130, 97, 0, -53, 2, 104, 101, -90, -389, -598, -576, -361, -138, -75, -192, -361, -425, -314, -81, 160, 314, 344, 279, 187, 135, 135, 131, 38, -175, -421, -544, -438, -158, 118, 240, 217, 204, 338, 592, 784, 738, 448, 82, -167, -239, -217, -210, -228, -181, 11, 312, 570, 625, 427, 80, -218, -274, -10, 474, 919, 1023, 640, -78, -730, -902, -461, 338, 1035, 1277, 1045, 596, 219, 28, -41, -85, -105, -34, 141, 325, 382, 266, 56, -128, -235, -288, -306, -238, -17, 324, 619, 679, 468, 159, -10, 48, 202, 229, 52, -168, -186, 95, 505, 734, 588, 146, -312, -527, -421, -109, 233, 491, 623, 615, 450, 147, -196, -421, -409, -175, 124, 308, 294, 151, 21, 0, 64, 119, 99, 25, -19, 13, 90, 126, 80, -1, -29, 27, 93, 43, -161, -399, -456, -204, 264, 670, 734, 380, -206, -707, -879, -673, -223, 268, 638, 793, 704, 405, 5, -327, -433, -254, 122, 520, 778, 832, 714, 492, 232, -8, -155, -130, 87, 410, 666, 707, 510, 206, -12, -26, 146, 375, 512, 479, 285, 11, -237, -353, -264, 24, 408, 708, 765, 545, 167, -169, -312, -242, -48, 157, 319, 436, 505, 485, 340, 110, -73, -74, 131, 398, 505, 313, -120, -579, -828, -757, -413, 52, 463, 672, 591, 234, -246, -588, -563, -140, 445, 840, 811, 419, -40, -280, -221, -17, 127, 130, 58, 12, 5, -22, -100, -159, -101, 78, 246, 242, 41, -203, -288, -145, 104, 263, 245, 126, 40, 26, -3, -144, -368, -526, -482, -253, -1, 97, 3, -184, -332, -378, -340, -247, -88, 147, 399, 532, 438, 147, -161, -292, -191, 1, 77, -70, -348, -543, -495, -221, 102, 279, 237, 59, -93, -113, -13, 104, 156, 137, 106, 101, 78, -49, -307, -581, -666, -435, 29, 452, 552, 256, -239, -620, -672, -408, -5, 352, 573, 635, 518, 204, -239, -613, -670, -311, 286, 745, 748, 277, -368, -790, -761, -351, 167, 527, 607, 446, 172, -81, -209, -162, 31, 254, 356, 245, -41, -364, -568, -584, -440, -218, 15, 214, 345, 373, 293, 159, 66, 83, 198, 324, 355, 241, 3, -278, -500, -557, -378, 21, 502, 832, 815, 428, -134, -586, -733, -587, -317, -94, 29, 107, 191, 262, 266, 204, 172, 280, 534, 803, 904, 753, 427, 93, -122, -219, -275, -354, -443, -478, -424, -302, -159, -8, 186, 447, 723, 880, 787, 431, -37, -378, -409, -142, 217, 410, 298, -30, -326, -355, -67, 377, 734, 849, 737, 528, 334, 175, 7, -184, -342, -386, -301, -172, -118, -185, -304, -355, -266, -76, 105, 178, 115, -39, -207, -303, -257, -40, 300, 631, 785, 667, 336, -11, -169, -60, 221, 472, 524, 340, 15, -296, -481, -512, -447, -382, -384, -442, -464, -342, -37, 366, 704, 835, 739, 517, 307, 183, 131, 101, 77, 81, 122, 151, 92, -84, -315, -478, -480, -328, -109, 71, 150, 115, -8, -162, -261, -218, -14, 259, 443, 424, 247, 88, 113, 322, 538, 556, 321, -16, -223, -173, 67, 310, 399, 298, 73, -194, -452, -656, -736, -609, -254, 243, 705, 949, 885, 551, 87, -327, -534, -454, -116, 338, 718, 866, 738, 419, 68, -160, -179, 6, 303, 554, 597, 352, -114, -596, -851, -744, -325, 204, 612, 758, 642, 378, 127, 25, 119, 326, 463, 374, 64, -248, -276, 90, 642, 963, 747, 67, -653, -959, -712, -175, 227, 245, -49, -397, -602, -643, -592, -468, -211, 197, 623, 815, 606, 60, -541, -892, -859, -525, -102, 206, 284, 112, -221, -536, -618, -348, 174, 642, 729, 323, -361, -916, -1009, -596, 71, 623, 786, 511, -47, -621, -952, -904, -526, -35, 301, 319, 54, -302, -551, -612, -534, -403, -251, -64, 152, 323, 354, 231, 56, -20, 66, 242, 344, 253, -19, -353, -603, -681, -575, -327, -5, 313, 561, 700, 735, 698, 605, 439, 177, -155, -460, -597, -465, -75, 440, 874, 1033, 835, 368, -130, -389, -264, 148, 553, 649, 334, -224, -724, -940, -845, -551, -166, 281, 771, 1180, 1298, 983, 329, -337, -681, -591, -251, 36, 90, -51, -223, -303, -279, -182, -13, 231, 464, 482, 120, -538, -1116, -1162, -515, 510, 1285, 1294, 500, -635, -1466, -1574, -980, -54, 747, 1121, 995, 491, -166, -730, -1011, -943, -625, -265, -58, -69, -216, -350, -368, -263, -88, 109, 305, 446, 426, 143, -376, -915, -1143, -851, -146, 560, 802, 383, -453, -1174, -1307, -755, 155, 902, 1092, 657, -174, -1062, -1721, -1990, -1819, -1255, -449, 354, 890, 1000, 733, 312, -5, -86, 22, 178, 267, 280, 277, 299, 318, 267, 123, -64, -211, -271, -268, -257, -250, -203, -76, 81, 128, -63, -468, -861, -930, -497, 326, 1177, 1643, 1509, 883, 112, -416, -480, -115, 439, 875, 978, 721, 264, -149, -346, -310, -165, -59, -47, -74, -48, 59, 195, 287, 318, 321, 315, 264, 116, -101, -256, -203, 89, 488, 788, 854, 685, 359, -51, -481, -806, -833, -441, 233, 776, 756, 88, -827, -1362, -1141, -343, 451, 729, 411, -165, -611, -789, -809, -780, -631, -211, 445, 1015, 1105, 606, -147, -631, -547, -60, 368, 387, 20, -412, -609, -525, -349, -269, -295, -287, -129, 119, 269, 154, -211, -639, -903, -896, -697, -495, -434, -504, -551, -415, -71, 313, 492, 322, -122, -598, -852, -778, -437, 9, 395, 572, 423, -76, -771, -1313, -1324, -667, 393, 1323, 1664, 1338, 655, 41, -282, -375, -412, -463, -451, -295, -61, 67, -32, -291, -488, -441, -160, 184, 405, 440, 346, 224, 142, 123, 170, 253, 306, 242, 43, -193, -294, -144, 199, 534, 644, 436, -22, -566, -1012, -1190, -977, -365, 461, 1150, 1362, 1001, 308, -297, -515, -346, -32, 186, 241, 202, 123, -33, -305, -595, -675, -387, 155, 593, 594, 144, -416, -669, -474, -74, 120, -99, -563, -860, -682, -64, 653, 1090, 1080, 735, 303, -38, -262, -422, -527, -497, -256, 136, 463, 488, 158, -308, -568, -425, -2, 337, 293, -125, -605, -795, -580, -152, 185, 262, 123, -91, -298, -497, -661, -655, -319, 339, 1048, 1405, 1163, 425, -420, -968, -1051, -780, -402, -114, 5, -34, -191, -390, -514, -447, -167, 194, 419, 356, 45, -300, -445, -300, 32, 355, 519, 513, 428, 369, 373, 409, 422, 385, 306, 192, 21, -245, -598, -935, -1085, -908, -414, 215, 717, 902, 754, 410, 56, -167, -206, -70, 169, 382, 404, 146, -302, -672, -681, -260, 361, 803, 815, 450, -5, -280, -306, -198, -81, 37, 222, 469, 648, 603, 318, -36, -247, -239, -136, -130, -293, -508, -579, -397, -17, 396, 689, 782, 670, 399, 61, -214, -312, -205, 12, 177, 170, -2, -232, -392, -405, -265, -6, 301, 551, 620, 444, 85, -280, -474, -442, -277, -126, -58, -37, 3, 57, 32, -140, -396, -546, -416, -4, 485, 769, 684, 304, -115, -316, -217, 48, 245, 232, 69, -44, 41, 284, 479, 452, 240, 65, 113, 330, 437, 202, -302, -712, -688, -239, 261, 397, 92, -331, -457, -160, 304, 567, 489, 244, 89, 88, 76, -106, -393, -535, -359, 26, 319, 315, 104, -22, 135, 492, 758, 704, 350, -81, -360, -406, -286, -101, 82, 246, 367, 424, 402, 305, 133, -113, -392, -599, -616, -425, -157, -4, -38, -126, -58, 229, 551, 625, 359, -30, -190, 48, 504, 804, 708, 294, -138, -338, -266, -39, 228, 505, 781, 991, 1038, 901, 672, 467, 306, 102, -210, -558, -764, -723, -520, -345, -294, -264, -76, 298, 653, 708, 371, -158, -549, -598, -349, 3, 291, 446, 453, 292, -21, -369, -551, -445, -134, 139, 199, 101, 81, 294, 633, 802, 594, 105, -334, -445, -217, 118, 318, 321, 241, 216, 268, 318, 284, 160, 5, -109, -125, 0, 269, 609, 864, 880, 625, 240, -49, -88, 118, 440, 719, 841, 740, 386, -170, -768, -1169, -1203, -893, -434, -31, 250, 473, 669, 740, 538, 64, -439, -639, -393, 117, 535, 623, 438, 216, 125, 138, 132, 68, 37, 115, 230, 203, -44, -357, -462, -241, 113, 245, -29, -519, -796, -561, 78, 679, 811, 388, -278, -731, -693, -234, 315, 613, 519, 152, -218, -376, -294, -125, -55, -143, -275, -273, -66, 237, 442, 425, 222, -25, -209, -312, -367, -371, -280, -86, 120, 187, 31, -273, -537, -597, -426, -129, 150, 318, 348, 256, 89, -80, -181, -182, -130, -100, -121, -138, -82, 40, 131, 99, -36, -165, -214, -239, -352, -534, -580, -269, 351, 900, 938, 375, -383, -739, -419, 284, 774, 670, 116, -400, -504, -221, 148, 372, 473, 597, 748, 735, 377, -248, -805, -960, -652, -121, 314, 485, 415, 213, -26, -219, -258, -66, 285, 554, 482, 35, -524, -827, -679, -205, 285, 555, 567, 427, 236, 37, -131, -194, -104, 70, 168, 81, -120, -245, -185, -35, 0, -145, -305, -240, 80, 395, 385, 3, -431, -506, -99, 498, 822, 625, 46, -554, -890, -904, -698, -389, -56, 210, 301, 169, -84, -260, -224, -32, 132, 154, 92, 98, 226, 362, 355, 189, 10, -54, -51, -162, -473, -825, -922, -619, -98, 257, 189, -218, -627, -735, -485, -36, 414, 764, 973, 993, 769, 336, -134, -416, -392, -130, 191, 426, 524, 484, 306, 10, -300, -462, -374, -117, 86, 74, -104, -240, -180, 11, 129, 51, -113, -151, 18, 229, 231, -40, -369, -454, -194, 211, 456, 400, 165, -12, -3, 135, 271, 325, 298, 197, 0, -284, -554, -647, -484, -161, 106, 163, 23, -185, -356, -473, -546, -523, -327, 22, 339, 389, 100, -337, -610, -541, -219, 115, 293, 324, 291, 201, -24, -399, -758, -842, -539, -25, 356, 382, 137, -85, -62, 172, 390, 398, 197, -59, -221, -249, -193, -115, -51, -31, -75, -154, -165, 1, 343, 686, 799, 581, 162, -198, -318, -212, -15, 169, 353, 578, 795, 854, 623, 133, -420, -807, -898, -706, -337, 70, 369, 424, 190, -213, -532, -528, -162, 377, 831, 1066, 1127, 1104, 994, 700, 193, -362, -686, -599, -179, 294, 565, 560, 363, 83, -226, -522, -703, -648, -362, -36, 65, -172, -603, -932, -954, -679, -261, 170, 569, 886, 992, 759, 235, -290, -471, -191, 311, 642, 588, 271, -3, -59, 27, 72, 13, -40, 13, 87, -34, -442, -917, -1065, -672, 62, 647, 692, 209, -439, -857, -901, -699, -446, -238, -73, 18, -70, -426, -973, -1443, -1538, -1128, -355, 449, 941, 930, 451, -250, -825, -982, -643, -5, 565, 755, 489, -44, -549, -807, -783, -576, -324, -132, -53, -89, -190, -271, -272, -209, -169, -224, -346, -423, -359, -186, -30, 11, -16, 22, 213, 478, 636, 576, 360, 156, 77, 86, 70, -15, -95, -56, 109, 281, 319, 191, 4, -101, -56, 104, 301, 460, 521, 441, 224, -45, -229, -246, -163, -143, -281, -479, -520, -291, 62, 242, 77, -287, -499, -326, 117, 471, 460, 138, -183, -241, -41, 199, 300, 275, 263, 328, 369, 245, -43, -319, -381, -189, 98, 277, 250, 63, -186, -438, -664, -823, -846, -714, -499, -326, -269, -292, -302, -244, -133, -5, 136, 299, 424, 386, 109, -313, -644, -690, -487, -273, -260, -430, -562, -477, -231, -57, -112, -310, -417, -316, -141, -122, -315, -508, -431, -49, 370, 502, 270, -99, -323, -334, -301, -391, -554, -575, -328, 41, 236, 95, -239, -458, -377, -90, 156, 216, 158, 147, 246, 355, 332, 155, -57, -159, -87, 123, 399, 654, 772, 621, 155, -480, -966, -1007, -555, 141, 725, 976, 919, 720, 529, 420, 439, 628, 954, 1236, 1211, 744, -5, -679, -1005, -1002, -901, -842, -698, -226, 589, 1376, 1625, 1131, 212, -537, -733, -465, -123, 9, -21, 4, 177, 344, 310, 104, -12, 189, 648, 1039, 1059, 688, 166, -243, -450, -498, -414, -159, 250, 623, 680, 317, -236, -560, -402, 90, 499, 521, 223, -54, -52, 172, 340, 251, -11, -188, -111, 130, 292, 208, -62, -309, -356, -177, 133, 452, 677, 726, 556, 200, -197, -445, -428, -199, 64, 195, 150, 6, -128, -191, -164, -42, 157, 370, 470, 353, 40, -300, -467, -372, -105, 154, 296, 339, 368, 410, 398, 249, -13, -253, -311, -146, 129, 331, 341, 175, -43, -167, -116, 99, 392, 645, 765, 726, 569, 377, 223, 141, 126, 156, 210, 270, 321, 362, 412, 503, 645, 795, 877, 834, 666, 430, 197, 28, -30, 61, 303, 597, 745, 561, 32, -614, -1032, -1000, -593, -115, 159, 198, 194, 336, 595, 736, 540, 37, -481, -673, -408, 150, 677, 918, 826, 538, 253, 115, 159, 317, 455, 438, 201, -194, -583, -786, -727, -472, -187, -21, -31, -152, -260, -242, -73, 161, 295, 176, -221, -749, -1151, -1212, -889, -330, 226, 586, 656, 449, 52, -371, -607, -471, 48, 726, 1200, 1216, 815, 298, -1, 37, 282, 505, 568, 470, 258, -58, -469, -853, -976, -659, 15, 688, 943, 632, -3, -506, -538, -101, 505, 920, 943, 612, 125, -273, -402, -197, 248, 724, 1003, 962, 631, 142, -368, -795, -1030, -959, -537, 90, 596, 671, 285, -243, -479, -235, 271, 619, 564, 235, -25, -22, 121, 128, -106, -385, -402, -65, 403, 680, 637, 420, 251, 184, 86, -149, -425, -521, -343, -74, -19, -289, -653, -736, -389, 147, 442, 258, -238, -638, -636, -252, 261, 667, 904, 1020, 1022, 832, 398, -170, -623, -737, -492, -78, 242, 310, 128, -191, -522, -766, -858, -772, -536, -238, 16, 163, 201, 172, 106, 4, -133, -268, -322, -222, 45, 397, 669, 680, 333, -292, -935, -1261, -1077, -491, 129, 399, 194, -263, -605, -626, -435, -302, -370, -502, -430, -53, 419, 633, 425, 0, -233, -29, 468, 836, 732, 173, -482, -820, -660, -136, 446, 818, 851, 552, 20, -574, -1008, -1082, -743, -158, 361, 543, 340, -42, -293, -211, 159, 575, 771, 631, 239, -204, -536, -710, -766, -735, -573, -220, 276, 705, 781, 368, -370, -1025, -1209, -844, -226, 190, 121, -352, -877, -1103, -911, -438, 69, 432, 586, 534, 302, -50, -401, -590, -526, -271, -15, 68, -43, -217, -290, -221, -111, -91, -176, -244, -157, 82, 295, 249, -139, -708, -1124, -1104, -608, 122, 707, 858, 551, 33, -332, -296, 122, 647, 917, 713, 95, -642, -1163, -1282, -1047, -660, -327, -140, -79, -82, -98, -103, -74, 7, 127, 205, 142, -98, -430, -689, -741, -582, -320, -71, 135, 338, 546, 663, 547, 166, -306, -572, -429, 46, 514, 612, 224, -433, -956, -1024, -594, 102, 719, 977, 786, 265, -319, -672, -616, -217, 230, 375, 51, -569, -1077, -1113, -637, 31, 465, 447, 110, -203, -241, -28, 190, 204, 27, -123, -37, 279, 597, 654, 382, -16, -234, -107, 236, 473, 335, -167, -741, -1020, -830, -316, 181, 372, 209, -117, -356, -383, -255, -99, 24, 171, 415, 716, 894, 744, 226, -446, -909, -876, -345, 393, 942, 1057, 778, 364, 75, 1, 42, 46, -47, -172, -235, -225, -221, -291, -393, -404, -251, -26, 65, -112, -472, -746, -702, -349, 60, 247, 139, -82, -165, -7, 274, 477, 491, 361, 170, -67, -407, -838, -1210, -1307, -1053, -615, -307, -332, -611, -846, -782, -420, -3, 205, 149, 1, -4, 207, 495, 623, 459, 78, -300, -468, -361, -75, 212, 355, 296, 72, -226, -490, -616, -539, -271, 63, 259, 143, -282, -782, -1008, -752, -134, 439, 572, 198, -343, -566, -238, 417, 905, 859, 332, -280, -587, -508, -267, -133, -155, -165, 4, 302, 481, 319, -151, -647, -845, -619, -129, 310, 437, 185, -297, -732, -851, -535, 101, 738, 1010, 741, 77, -580, -840, -578, -12, 477, 611, 377, -23, -347, -467, -391, -197, 38, 249, 356, 272, -50, -556, -1071, -1359, -1233, -658, 201, 1025, 1475, 1362, 763, -6, -561, -667, -358, 111, 460, 542, 393, 142, -104, -311, -479, -582, -569, -408, -145, 105, 225, 174, 11, -146, -210, -177, -124, -140, -240, -340, -320, -133, 127, 280, 189, -110, -436, -597, -551, -413, -318, -270, -148, 138, 489, 645, 416, -103, -559, -617, -255, 192, 321, 7, -490, -751, -557, -58, 390, 534, 387, 137, -71, -250, -468, -693, -760, -526, -49, 412, 588, 418, 78, -180, -250, -216, -234, -356, -497, -532, -426, -251, -96, 22, 148, 300, 413, 394, 217, -27, -206, -248, -194, -130, -86, -12, 147, 363, 521, 509, 322, 61, -150, -248, -234, -125, 65, 286, 399, 236, -249, -869, -1271, -1183, -646, -15, 303, 152, -292, -705, -890, -883, -813, -706, -456, 3, 530, 830, 708, 282, -86, -110, 160, 386, 262, -189, -647, -798, -626, -395, -362, -514, -597, -395, 20, 337, 268, -197, -793, -1192, -1216, -880, -301, 365, 917, 1107, 759, -38, -872, -1217, -808, 132, 1031, 1368, 1038, 364, -194, -386, -266, -47, 115, 192, 202, 127, -57, -292, -410, -260, 139, 568, 758, 606, 263, 1, -13, 139, 218, 44, -320, -597, -512, -36, 563, 907, 803, 411, 128, 267, 777, 1263, 1291, 742, -80, -696, -802, -453, 62, 505, 819, 1033, 1088, 823, 171, -647, -1191, -1114, -465, 329, 796, 760, 400, 15, -245, -412, -516, -460, -115, 443, 874, 795, 137, -736, -1277, -1174, -564, 127, 534, 564, 337, 0, -367, -684, -816, -647, -231, 176, 313, 144, -101, -154, 39, 283, 370, 322, 368, 665, 1071, 1233, 918, 271, -286, -403, -69, 434, 809, 934, 829, 507, -39, -691, -1101, -894, -39, 966, 1383, 829, -332, -1206, -1059, 82, 1446, 2117, 1719, 621, -427, -868, -647, -98, 392, 627, 617, 474, 315, 219, 211, 243, 218, 82, -87, -116, 101, 451, 633, 401, -191, -776, -931, -514, 232, 880, 1138, 1019, 751, 533, 398, 269, 109, -5, 22, 183, 328, 287, 38, -245, -315, -44, 448, 864, 924, 563, -18, -481, -561, -238, 268, 647, 698, 449, 117, -46, 90, 473, 907, 1162, 1078, 636, -21, -631, -922, -764, -271, 238, 432, 182, -342, -799, -916, -670, -261, 83, 282, 417, 592, 785, 851, 660, 236, -232, -548, -656, -679, -788, -1042, -1328, -1461, -1332, -992, -592, -268, -64, 57, 142, 206, 252, 301, 383, 489, 546, 475, 283, 107, 124, 385, 726, 857, 592, 19, -538, -774, -646, -395, -323, -508, -725, -643, -135, 586, 1118, 1186, 849, 433, 242, 322, 461, 401, 90, -270, -386, -118, 385, 786, 789, 354, -262, -677, -633, -164, 427, 789, 749, 415, 63, -80, 0, 141, 175, 82, -13, 21, 181, 319, 289, 90, -109, -114, 133, 497, 749, 735, 469, 101, -190, -295, -190, 75, 408, 675, 742, 546, 158, -222, -379, -227, 110, 373, 339, -6, -459, -732, -646, -247, 242, 578, 627, 425, 115, -127, -164, 62, 496, 941, 1110, 774, -58, -1063, -1743, -1727, -1038, -96, 547, 581, 120, -421, -638, -400, 131, 667, 992, 1032, 813, 393, -143, -664, -1002, -1036, -778, -374, -17, 163, 153, 24, -117, -180, -111, 76, 294, 407, 330, 114, -69, -51, 183, 466, 568, 385, 21, -303, -420, -304, -38, 281, 588, 802, 800, 488, -64, -591, -761, -429, 223, 811, 1027, 872, 621, 559, 736, 940, 917, 604, 192, -52, -22, 153, 252, 178, 59, 127, 474, 906, 1053, 664, -135, -851, -943, -225, 944, 1859, 1928, 1079, -204, -1211, -1464, -962, -100, 647, 993, 910, 538, 60, -348, -540, -429, -64, 356, 572, 433, 6, -456, -691, -582, -214, 207, 498, 585, 516, 401, 328, 304, 265, 136, -85, -303, -402, -374, -356, -514, -842, -1093, -938, -291, 529, 971, 673, -210, -1096, -1409, -1014, -270, 309, 466, 318, 142, 84, 83, 38, -16, 68, 356, 704, 863, 708, 352, 21, -182, -350, -608, -903, -988, -660, -4, 607, 801, 507, 12, -293, -221, 110, 431, 560, 501, 375, 281, 246, 252, 276, 289, 256, 170, 94, 129, 308, 511, 522, 221, -269, -653, -697, -431, -104, 64, 99, 215, 554, 974, 1126, 779, 70, -549, -654, -175, 561, 1099, 1178, 867, 440, 154, 111, 262, 489, 668, 702, 547, 250, -52, -205, -135, 81, 274, 291, 98, -212, -484, -581, -453, -151, 185, 384, 330, 32, -359, -653, -732, -620, -430, -245, -64, 153, 396, 564, 543, 320, 18, -184, -181, -4, 221, 375, 392, 262, 11, -293, -533, -582, -398, -94, 103, 14, -359, -820, -1117, -1111, -840, -455, -94, 173, 338, 414, 440, 481, 582, 690, 637, 252, -452, -1190, -1545, -1269, -501, 288, 631, 380, -211, -727, -900, -765, -542, -399, -328, -211, 20, 304, 503, 531, 414, 264, 198, 285, 522, 824, 1021, 923, 460, -193, -644, -527, 189, 1093, 1567, 1244, 311, -606, -923, -534, 132, 505, 343, -107, -394, -290, 18, 162, -25, -321, -325, 141, 832, 1258, 1087, 421, -288, -594, -353, 219, 745, 924, 660, 65, -600, -1012, -920, -320, 462, 917, 690, -115, -966, -1273, -857, -85, 442, 387, -67, -427, -350, 64, 406, 351, -60, -492, -633, -444, -125, 104, 163, 83, -109, -425, -806, -1062, -961, -449, 230, 683, 660, 243, -249, -558, -669, -750, -911, -1068, -1041, -763, -369, -69, 49, 69, 107, 161, 122, -49, -206, -122, 261, 728, 941, 763, 399, 208, 354, 640, 702, 381, -101, -337, -118, 373, 740, 730, 422, 95, -74, -144, -261, -409, -366, 75, 831, 1505, 1666, 1201, 406, -246, -465, -307, -44, 104, 117, 103, 133, 159, 89, -87, -260, -286, -119, 148, 353, 375, 203, -75, -326, -430, -320, -17, 360, 639, 672, 440, 77, -205, -260, -103, 87, 92, -200, -710, -1200, -1411, -1209, -661, -1, 491, 627, 383, -99, -595, -895, -897, -635, -269, -13, -25, -320, -743, -1056, -1082, -829, -481, -271, -305, -494, -636, -578, -347, -125, -102, -329, -666, -872, -762, -334, 213, 573, 496, -50, -835, -1463, -1593, -1153, -395, 258, 493, 300, -44, -212, -66, 255, 480, 428, 140, -179, -340, -308, -196, -153, -246, -424, -585, -649, -595, -449, -247, -25, 178, 311, 322, 187, -56, -312, -453, -380, -88, 321, 672, 804, 657, 313, -39, -212, -113, 194, 521, 648, 455, 2, -488, -763, -689, -351, 13, 194, 162, 79, 141, 389, 663, 750, 582, 310, 166, 240, 387, 359, 36, -440, -787, -818, -592, -344, -268, -364, -480, -477, -347, -185, -70, 4, 82, 162, 186, 111, -23, -126, -160, -193, -317, -516, -632, -502, -139, 238, 380, 235, 1, -72, 55, 163, -20, -526, -1068, -1265, -968, -387, 94, 223, 40, -226, -388, -419, -391, -346, -259, -116, 8, 2, -164, -385, -504, -463, -337, -231, -159, -59, 64, 53, -275, -897, -1472, -1553, -973, -46, 635, 672, 177, -339, -425, -43, 468, 751, 736, 614, 556, 526, 376, 113, -12, 259, 862, 1339, 1185, 322, -748, -1302, -965, -6, 896, 1209, 931, 505, 385, 647, 964, 930, 419, -308, -801, -735, -133, 656, 1179, 1140, 581, -141, -577, -470, 75, 680, 960, 793, 370, 6, -107, 0, 185, 355, 509, 635, 610, 273, -381, -1074, -1376, -1030, -206, 572, 805, 392, -297, -725, -606, -104, 322, 305, -162, -748, -1057, -923, -508, -150, -94, -316, -554, -513, -92, 523, 991, 1037, 655, 100, -295, -351, -141, 101, 180, 66, -111, -198, -133, 33, 206, 319, 346, 277, 104, -148, -393, -504, -390, -75, 299, 553, 578, 402, 165, 37, 119, 389, 700, 842, 656, 152, -455, -843, -786, -338, 174, 365, 87, -440, -780, -626, -60, 490, 596, 174, -431, -720, -425, 289, 971, 1233, 1012, 550, 164, 16, 71, 217, 377, 527, 633, 626, 469, 229, 48, 20, 89, 86, -94, -372, -529, -402, -58, 252, 294, 52, -270, -430, -331, -73, 162, 253, 197, 75, 0, 80, 388, 880, 1351, 1500, 1136, 367, -411, -783, -625, -202, 55, -80, -467, -738, -638, -221, 238, 499, 510, 361, 134, -142, -419, -545, -353, 156, 715, 944, 648, -20, -673, -992, -931, -660, -345, -22, 332, 641, 710, 412, -128, -571, -622, -284, 150, 368, 300, 133, 68, 91, -6, -381, -925, -1303, -1229, -712, -51, 405, 510, 380, 251, 283, 463, 653, 695, 508, 133, -261, -449, -273, 219, 762, 1015, 807, 267, -276, -552, -536, -408, -336, -306, -172, 143, 510, 683, 545, 250, 107, 305, 729, 1055, 1038, 717, 363, 218, 283, 335, 151, -284, -768, -1043, -991, -681, -281, 65, 293, 387, 352, 217, 47, -73, -105, -89, -112, -225, -393, -524, -559, -511, -434, -353, -244, -95, 26, -6, -261, -631, -872, -780, -377, 69, 247, 43, -355, -617, -528, -160, 204, 306, 91, -285, -605, -738, -677, -474, -178, 164, 470, 615, 508, 173, -243, -551, -633, -504, -279, -103, -80, -250, -566, -899, -1077, -965, -564, -41, 359, 473, 307, 18, -215, -307, -270, -156, -3, 149, 221, 124, -139, -409, -443, -123, 383, 697, 507, -137, -806, -993, -523, 285, 845, 750, 89, -633, -887, -508, 217, 805, 936, 647, 222, -58, -109, -32, 33, 39, 35, 79, 160, 206, 145, -33, -282, -540, -747, -842, -745, -405, 124, 649, 902, 703, 112, -569, -963, -852, -318, 316, 709, 723, 496, 312, 358, 578, 709, 507, -23, -589, -800, -484, 126, 531, 334, -421, -1226, -1472, -927, 41, 721, 582, -310, -1338, -1796, -1404, -482, 327, 540, 133, -511, -935, -903, -492, 44, 466, 661, 643, 499, 331, 211, 155, 108, -3, -210, -442, -564, -478, -218, 46, 116, -86, -435, -655, -495, 86, 850, 1381, 1334, 690, -190, -764, -673, -12, 713, 957, 550, -165, -595, -374, 341, 1000, 1094, 546, -258, -806, -850, -550, -268, -241, -409, -517, -383, -64, 186, 145, -191, -586, -724, -443, 146, 729, 1001, 879, 543, 294, 307, 495, 587, 363, -144, -665, -892, -724, -330, 0, 69, -108, -389, -630, -774, -811, -719, -471, -101, 253, 421, 339, 122, -11, 75, 315, 485, 392, 35, -388, -640, -612, -392, -158, -24, 39, 141, 328, 505, 498, 223, -192, -468, -387, -12, 303, 176, -469, -1262, -1608, -1143, -59, 988, 1340, 828, -105, -751, -667, 14, 734, 950, 530, -193, -714, -705, -210, 436, 858, 856, 486, -8, -362, -422, -215, 94, 317, 343, 189, -44, -233, -293, -197, 12, 224, 294, 130, -212, -522, -561, -265, 164, 389, 205, -262, -642, -609, -164, 372, 628, 489, 164, -22, 90, 389, 622, 628, 434, 188, 0, -122, -227, -330, -396, -388, -321, -252, -232, -266, -331, -392, -415, -363, -206, 37, 285, 430, 422, 316, 228, 243, 348, 454, 478, 392, 212, -27, -272, -424, -370, -87, 277, 453, 249, -249, -694, -724, -261, 410, 863, 857, 484, 39, -250, -358, -368, -308, -102, 286, 733, 984, 878, 500, 127, -26, 8, 4, -217, -597, -880, -841, -478, 7, 380, 523, 451, 222, -112, -492, -800, -907, -772, -492, -244, -154, -220, -353, -464, -496, -394, -100, 378, 874, 1102, 867, 268, -314, -507, -266, 79, 135, -163, -458, -301, 396, 1206, 1514, 1042, 96, -681, -833, -410, 138, 384, 242, -50, -205, -124, 75, 218, 205, 37, -230, -518, -709, -686, -427, -78, 111, -24, -409, -768, -842, -590, -191, 144, 344, 473, 575, 578, 366, -38, -416, -495, -202, 255, 556, 528, 267, 7, -116, -162, -265, -460, -637, -669, -553, -404, -312, -236, -56, 257, 564, 646, 397, -75, -540, -812, -841, -664, -331, 103, 513, 702, 539, 111, -282, -356, -91, 221, 240, -104, -520, -600, -194, 442, 841, 698, 93, -599, -998, -953, -567, -70, 322, 478, 367, 70, -229, -321, -100, 346, 779, 964, 814, 415, -65, -483, -746, -804, -651, -374, -153, -164, -429, -755, -870, -647, -237, 44, -30, -416, -838, -1008, -824, -383, 131, 579, 880, 976, 833, 489, 84, -191, -227, -69, 121, 203, 144, 20, -71, -78, -2, 128, 270, 373, 393, 333, 243, 168, 74, -135, -501, -888, -1021, -702, -23, 625, 827, 469, -166, -623, -629, -270, 135, 327, 274, 123, 32, 60, 198, 424, 693, 889, 845, 471, -114, -624, -803, -626, -293, -45, 42, 64, 136, 258, 341, 336, 300, 301, 291, 127, -253, -687, -866, -614, -87, 326, 336, 13, -291, -270, 61, 414, 505, 303, 0, -205, -281, -311, -335, -284, -79, 231, 473, 497, 318, 75, -132, -338, -627, -969, -1175, -1063, -655, -202, 16, -58, -234, -260, -65, 203, 342, 292, 166, 116, 196, 346, 478, 543, 501, 292, -111, -598, -907, -794, -272, 321, 543, 202, -449, -894, -745, -58, 726, 1121, 940, 375, -194, -489, -471, -287, -112, -22, 23, 124, 339, 627, 843, 831, 541, 111, -187, -128, 290, 813, 1094, 935, 427, -131, -451, -435, -189, 102, 317, 436, 482, 451, 310, 64, -199, -338, -253, 32, 381, 627, 692, 631, 569, 591, 674, 699, 555, 226, -198, -586, -841, -924, -833, -579, -206, 168, 373, 272, -124, -632, -999, -1066, -854, -510, -177, 97, 332, 531, 634, 566, 321, -1, -250, -304, -130, 204, 559, 757, 651, 229, -331, -726, -707, -275, 296, 651, 613, 297, -10, -103, 2, 137, 152, 29, -159, -358, -546, -679, -646, -346, 175, 689, 926, 783, 406, 61, -75, -14, 141, 338, 590, 859, 974, 718, 56, -756, -1296, -1278, -757, -70, 425, 582, 479, 285, 129, 72, 137, 316, 531, 644, 558, 318, 100, 59, 197, 378, 477, 503, 542, 608, 580, 335, -56, -312, -167, 348, 886, 1059, 772, 293, -5, 18, 202, 293, 205, 61, 5, 24, -13, -168, -322, -283, 0, 381, 656, 780, 885, 1102, 1377, 1501, 1315, 877, 405, 80, -75, -138, -136, -19, 226, 474, 535, 356, 109, 41, 216, 421, 368, -6, -461, -670, -523, -199, 58, 178, 283, 466, 632, 586, 286, -49, -143, 39, 221, 80, -404, -892, -1000, -662, -174, 108, 112, 58, 165, 394, 496, 306, -54, -296, -259, -62, 52, -5, -79, 44, 392, 765, 937, 863, 677, 512, 373, 202, 11, -96, -72, -30, -146, -426, -612, -403, 196, 786, 887, 395, -271, -528, -154, 508, 888, 721, 267, 33, 280, 816, 1225, 1267, 1037, 788, 641, 524, 338, 128, 40, 110, 163, -51, -583, -1171, -1413, -1068, -237, 723, 1431, 1662, 1408, 838, 238, -92, 16, 459, 886, 926, 489, -142, -521, -401, 81, 554, 743, 671, 556, 543, 551, 401, 41, -350, -530, -418, -175, -42, -107, -226, -163, 176, 662, 1028, 1073, 778, 293, -178, -491, -606, -573, -483, -408, -367, -338, -298, -258, -242, -237, -175, 3, 248, 375, 192, -313, -892, -1157, -870, -143, 603, 917, 615, -103, -806, -1093, -816, -131, 617, 1082, 1091, 720, 236, -66, -54, 183, 430, 527, 489, 456, 527, 641, 613, 308, -218, -738, -998, -892, -513, -83, 194, 251, 181, 172, 365, 722, 1003, 918, 376, -360, -802, -588, 187, 989, 1229, 740, -74, -571, -393, 234, 712, 576, -128, -905, -1225, -922, -275, 246, 322, -45, -590, -976, -975, -558, 104, 738, 1100, 1114, 898, 648, 478, 341, 120, -199, -458, -431, -35, 557, 1015, 1069, 682, 43, -573, -960, -1039, -861, -569, -355, -372, -630, -950, -1064, -821, -326, 112, 224, 0, -303, -381, -141, 219, 388, 208, -190, -486, -422, 1, 524, 809, 671, 194, -343, -663, -679, -537, -482, -654, -960, -1145, -988, -479, 164, 639, 736, 426, -160, -805, -1257, -1290, -810, 36, 858, 1202, 851, -4, -870, -1285, -1130, -665, -284, -197, -322, -411, -288, 21, 336, 457, 280, -143, -619, -872, -684, -61, 692, 1113, 884, 87, -814, -1297, -1142, -545, 90, 483, 620, 633, 586, 400, -11, -575, -1049, -1194, -945, -437, 112, 528, 695, 538, 59, -583, -1065, -1065, -517, 281, 842, 833, 297, -412, -929, -1101, -998, -751, -434, -83, 208, 298, 113, -238, -526, -583, -434, -251, -174, -198, -215, -157, -45, 80, 244, 487, 728, 717, 222, -694, -1569, -1809, -1149, 75, 1102, 1269, 468, -804, -1822, -2086, -1565, -602, 346, 936, 1024, 677, 124, -326, -436, -176, 247, 546, 564, 382, 219, 220, 322, 322, 80, -327, -655, -670, -343, 133, 480, 501, 199, -250, -605, -689, -491, -178, -11, -171, -600, -975, -895, -184, 905, 1768, 1817, 910, -496, -1606, -1798, -1044, 99, 901, 949, 379, -312, -681, -624, -360, -180, -217, -400, -577, -642, -561, -337, -3, 337, 515, 395, 27, -322, -342, 64, 663, 1036, 902, 329, -329, -711, -676, -341, 69, 371, 467, 320, -37, -461, -705, -566, -66, 503, 774, 583, 93, -353, -516, -415, -230, -92, 20, 179, 338, 319, -21, -614, -1178, -1424, -1254, -797, -270, 165, 434, 495, 332, 27, -208, -137, 297, 865, 1190, 1046, 544, 31, -211, -181, -104, -181, -405, -611, -662, -577, -465, -379, -265, -63, 185, 350, 346, 229, 134, 131, 147, 69, -102, -221, -145, 91, 272, 191, -139, -480, -568, -344, 12, 253, 270, 138, 6, -40, -12, 58, 171, 325, 467, 493, 350, 117, -42, -29, 77, 72, -198, -663, -1045, -1048, -582, 158, 811, 1065, 831, 273, -305, -631, -590, -258, 157, 446, 491, 312, 21, -251, -423, -480, -448, -355, -218, -44, 144, 285, 284, 66, -347, -812, -1109, -1078, -737, -281, 51, 147, 68, -29, -48, -8, 19, 20, 53, 170, 343, 488, 572, 663, 835, 1019, 995, 570, -176, -864, -1062, -643, 94, 648, 701, 333, -112, -344, -330, -234, -205, -256, -309, -316, -278, -184, 24, 365, 701, 776, 429, -205, -737, -820, -451, 8, 143, -166, -667, -960, -827, -348, 233, 728, 1071, 1254, 1252, 1053, 730, 428, 243, 114, -116, -517, -945, -1134, -927, -431, 65, 322, 304, 146, -8, -111, -179, -216, -219, -231, -350, -600, -809, -676, -31, 915, 1632, 1636, 885, -154, -862, -922, -513, -90, 13, -198, -475, -581, -450, -167, 155, 429, 581, 557, 371, 147, 49, 140, 309, 380, 289, 169, 214, 468, 752, 839, 697, 547, 658, 1057, 1449, 1442, 877, -25, -827, -1184, -1056, -679, -356, -267, -393, -563, -561, -250, 326, 928, 1230, 1041, 473, -102, -306, -26, 507, 918, 965, 696, 342, 104, 23, 19, 24, 39, 88, 146, 139, 9, -222, -477, -672, -756, -696, -475, -123, 235, 426, 347, 83, -120, -46, 313, 728, 909, 725, 288, -144, -354, -268, 41, 413, 659, 625, 280, -220, -589, -611, -339, -96, -207, -684, -1158, -1160, -538, 374, 1022, 1068, 643, 179, 9, 120, 264, 252, 135, 91, 197, 341, 372, 290, 255, 390, 616, 694, 461, 7, -405, -576, -527, -427, -384, -344, -194, 62, 273, 296, 161, 51, 99, 232, 244, 32, -266, -383, -190, 163, 388, 339, 155, 110, 335, 682, 865, 716, 315, -122, -431, -588, -634, -579, -398, -109, 193, 421, 579, 740, 911, 955, 704, 182, -315, -414, 3, 648, 1034, 881, 333, -201, -427, -384, -305, -315, -286, -33, 397, 679, 487, -154, -820, -1009, -573, 163, 675, 647, 176, -364, -632, -508, -99, 375, 690, 680, 318, -225, -635, -651, -280, 175, 356, 147, -238, -485, -478, -376, -391, -505, -470, -107, 410, 623, 195, -719, -1538, -1705, -1161, -366, 103, 31, -341, -603, -553, -301, -81, -7, -21, -39, -68, -176, -371, -557, -633, -604, -558, -537, -455, -198, 204, 527, 520, 148, -346, -631, -563, -286, -69, -49, -144, -176, -60, 135, 299, 398, 480, 576, 638, 598, 476, 386, 416, 516, 513, 278, -128, -487, -598, -445, -195, -41, -51, -141, -173, -63, 179, 484, 746, 854, 729, 387, -36, -350, -424, -271, -3, 279, 558, 827, 1000, 914, 470, -208, -807, -1030, -834, -451, -178, -128, -177, -133, 61, 295, 425, 423, 360, 281, 138, -125, -427, -543, -298, 216, 643, 619, 86, -624, -1038, -885, -280, 393, 774, 735, 393, -28, -356, -526, -583, -614, -685, -775, -774, -561, -133, 336, 596, 505, 156, -195, -356, -341, -325, -430, -579, -575, -329, 17, 203, 101, -132, -203, 50, 480, 749, 619, 147, -377, -683, -709, -594, -509, -502, -485, -338, -22, 382, 716, 819, 609, 151, -352, -646, -579, -216, 192, 394, 310, 74, -104, -131, -73, -58, -130, -213, -210, -113, 2, 68, 104, 168, 253, 262, 114, -123, -230, -13, 492, 997, 1162, 867, 313, -143, -255, -43, 280, 501, 534, 399, 135, -233, -647, -948, -931, -499, 222, 917, 1272, 1178, 792, 404, 229, 272, 356, 283, -19, -441, -772, -832, -580, -128, 323, 591, 588, 356, 29, -239, -352, -314, -206, -126, -109, -111, -57, 66, 176, 139, -93, -407, -561, -357, 188, 826, 1213, 1138, 658, 43, -404, -525, -352, -46, 216, 320, 233, -11, -338, -659, -903, -1037, -1051, -930, -654, -237, 220, 547, 596, 360, -4, -280, -351, -274, -202, -222, -265, -180, 95, 440, 610, 435, -24, -501, -688, -445, 106, 660, 900, 680, 103, -532, -894, -795, -303, 305, 724, 798, 587, 285, 52, -71, -134, -162, -112, 75, 381, 688, 870, 911, 906, 958, 1046, 1022, 748, 260, -208, -385, -168, 283, 657, 702, 391, -88, -485, -629, -505, -207, 139, 428, 593, 607, 495, 329, 208, 203, 306, 432, 466, 351, 136, -44, -62, 107, 359, 522, 482, 271, 40, -56, 14, 135, 137, -55, -356, -580, -589, -406, -196, -131, -260, -472, -598, -541, -343, -143, -65, -141, -307, -460, -524, -469, -299, -44, 239, 471, 565, 485, 261, -32, -330, -611, -879, -1103, -1200, -1085, -762, -365, -81, -23, -133, -233, -172, 47, 272, 330, 178, -64, -223, -200, -35, 145, 246, 262, 258, 296, 367, 394, 285, 12, -345, -611, -612, -297, 177, 513, 460, 22, -492, -648, -195, 728, 1654, 2083, 1797, 978, 66, -511, -550, -118, 520, 1046, 1215, 958, 424, -85, -293, -124, 247, 539, 571, 398, 223, 205, 334, 478, 538, 536, 553, 600, 580, 395, 78, -182, -189, 95, 504, 779, 752, 433, -24, -420, -588, -453, -69, 368, 593, 420, -99, -681, -991, -891, -525, -181, -46, -87, -135, -78, 50, 149, 170, 144, 88, -58, -363, -756, -994, -845, -323, 243, 447, 132, -472, -951, -1050, -851, -634, -586, -649, -635, -456, -214, -74, -84, -136, -113, -34, -38, -214, -468, -591, -459, -174, 5, -119, -528, -1000, -1272, -1207, -831, -291, 209, 480, 409, 53, -345, -480, -198, 344, 759, 707, 175, -488, -824, -611, -28, 490, 575, 180, -403, -770, -675, -176, 421, 775, 719, 353, -31, -152, 88, 550, 953, 1060, 803, 299, -228, -562, -563, -217, 345, 878, 1117, 930, 408, -171, -534, -582, -432, -263, -139, 39, 403, 928, 1385, 1484, 1094, 356, -398, -842, -826, -421, 137, 573, 681, 427, -32, -429, -545, -367, -95, 6, -179, -531, -777, -710, -326, 177, 540, 568, 209, -433, -1116, -1510, -1350, -610, 395, 1142, 1200, 521, -515, -1355, -1622, -1321, -761, -309, -170, -326, -611, -831, -857, -688, -455, -354, -507, -841, -1120, -1121, -846, -550, -522, -817, -1162, -1166, -655, 166, 855, 1054, 723, 115, -427, -677, -582, -247, 119, 255, -37, -729, -1495, -1861, -1533, -658, 236, 624, 371, -170, -465, -219, 416, 1000, 1140, 733, -27, -816, -1324, -1348, -849, -5, 799, 1142, 833, 80, -614, -801, -416, 191, 551, 443, 46, -248, -168, 240, 693, 910, 825, 596, 458, 532, 754, 930, 875, 541, 54, -354, -495, -334, -20, 228, 273, 145, 7, 25, 242, 544, 729, 636, 246, -279, -672, -706, -351, 171, 516, 450, 42, -382, -490, -210, 212, 440, 332, 44, -150, -134, -44, -108, -385, -691, -777, -580, -284, -139, -209, -327, -278, -40, 179, 141, -198, -641, -916, -906, -709, -512, -432, -447, -475, -469, -445, -440, -463, -500, -538, -565, -553, -457, -270, -65, 21, -106, -404, -690, -750, -494, -23, 425, 612, 426, -44, -542, -757, -495, 179, 939, 1370, 1240, 673, 66, -194, -9, 395, 673, 647, 409, 200, 182, 319, 437, 381, 120, -256, -610, -807, -762, -477, -65, 275, 375, 219, -34, -172, -91, 149, 435, 726, 1077, 1498, 1819, 1745, 1105, 68, -884, -1264, -921, -150, 532, 738, 432, -119, -571, -710, -522, -145, 229, 443, 445, 320, 247, 372, 680, 970, 994, 648, 81, -389, -482, -118, 550, 1247, 1701, 1748, 1363, 671, -101, -714, -1034, -1098, -1050, -998, -926, -739, -423, -126, -72, -346, -766, -968, -672, 102, 1021, 1654, 1724, 1244, 466, -263, -655, -585, -146, 376, 650, 493, 15, -447, -576, -327, 24, 105, -250, -864, -1349, -1412, -1052, -521, -113, 31, -31, -148, -183, -71, 181, 506, 777, 838, 578, 24, -639, -1138, -1251, -925, -306, 321, 674, 593, 140, -429, -799, -778, -426, 1, 247, 243, 148, 186, 437, 757, 902, 738, 351, -31, -225, -203, -66, 79, 201, 328, 476, 609, 670, 629, 498, 306, 83, -132, -276, -285, -145, 69, 243, 296, 247, 186, 198, 297, 438, 563, 628, 591, 408, 70, -340, -653, -704, -453, -25, 384, 657, 808, 905, 942, 816, 442, -101, -567, -706, -469, -53, 255, 320, 239, 224, 373, 571, 591, 320, -114, -418, -355, 61, 563, 803, 593, 25, -573, -848, -622, -3, 690, 1131, 1184, 972, 765, 770, 974, 1164, 1097, 692, 102, -383, -516, -241, 282, 766, 938, 678, 89, -532, -844, -656, -71, 546, 799, 530, -45, -485, -436, 100, 762, 1097, 885, 298, -244, -381, -49, 496, 882, 872, 491, -21, -387, -454, -255, 40, 250, 271, 132, -39, -89, 81, 449, 867, 1120, 1036, 594, -30, -541, -679, -384, 150, 590, 669, 361, -114, -451, -471, -237, 5, 14, -282, -734, -1067, -1056, -657, -45, 484, 684, 502, 119, -161, -124, 208, 594, 749, 572, 220, -28, -18, 150, 207, -48, -549, -986, -1032, -586, 146, 788, 1020, 757, 152, -501, -917, -923, -518, 131, 756, 1101, 1052, 700, 275, 3, -33, 69, 131, 21, -238, -489, -539, -295, 161, 610, 814, 667, 273, -115, -260, -108, 176, 341, 239, -53, -302, -312, -88, 165, 207, -47, -448, -710, -615, -177, 345, 603, 380, -231, -859, -1061, -627, 257, 1107, 1463, 1200, 597, 83, -106, -86, -150, -442, -790, -863, -516, 18, 287, 5, -676, -1254, -1261, -619, 317, 1044, 1245, 946, 410, -75, -351, -392, -261, -57, 112, 177, 144, 94, 108, 184, 236, 189, 78, 33, 149, 358, 444, 213, -337, -1004, -1516, -1715, -1598, -1249, -754, -202, 255, 436, 256, -134, -403, -275, 207, 682, 744, 294, -354, -724, -575, -85, 309, 299, -62, -438, -523, -298, -13, 38, -243, -726, -1176, -1422, -1417, -1192, -806, -328, 144, 506, 701, 764, 779, 786, 721, 468, -17, -615, -1078, -1194, -926, -445, -11, 189, 135, -56, -232, -283, -176, 50, 313, 502, 517, 321, -17, -355, -550, -553, -422, -257, -122, -14, 83, 134, 55, -199, -550, -805, -773, -410, 133, 608, 820, 740, 484, 220, 66, 43, 90, 105, 2, -232, -500, -633, -506, -162, 167, 214, -121, -656, -1034, -994, -568, -58, 205, 95, -246, -564, -681, -585, -358, -83, 199, 448, 594, 591, 474, 360, 347, 420, 466, 381, 182, -8, -82, -39, 38, 95, 153, 250, 327, 231, -134, -657, -1023, -960, -482, 75, 315, 105, -316, -556, -407, -18, 260, 196, -120, -369, -282, 130, 591, 758, 463, -189, -898, -1318, -1228, -643, 169, 793, 875, 351, -476, -1108, -1171, -686, -69, 173, -184, -919, -1525, -1586, -1070, -328, 189, 267, 63, -49, 213, 815, 1419, 1631, 1277, 520, -250, -665, -582, -130, 409, 763, 787, 497, 28, -427, -707, -753, -641, -541, -586, -758, -889, -795, -455, -61, 118, -27, -325, -423, -66, 695, 1507, 1953, 1821, 1220, 466, -123, -401, -416, -322, -265, -298, -363, -338, -141, 178, 426, 378, -37, -637, -1067, -1051, -604, -30, 321, 320, 141, 75, 265, 585, 778, 699, 441, 234, 223, 343, 405, 278, 7, -241, -343, -306, -224, -163, -100, 19, 188, 300, 231, -45, -426, -739, -862, -790, -605, -394, -195, 3, 224, 463, 685, 839, 881, 795, 600, 360, 155, 44, 33, 72, 99, 84, 56, 72, 171, 335, 501, 589, 546, 367, 87, -219, -469, -596, -586, -489, -389, -345, -343, -302, -133, 174, 521, 749, 741, 512, 201, -15, -43, 71, 189, 190, 66, -71, -82, 86, 346, 514, 434, 85, -396, -797, -951, -838, -586, -382, -333, -393, -382, -124, 396, 988, 1351, 1282, 836, 297, -21, -18, 126, 105, -247, -806, -1236, -1261, -870, -320, 73, 161, 22, -154, -238, -220, -149, -43, 118, 335, 528, 588, 473, 260, 77, -2, -22, -70, -167, -221, -117, 160, 481, 661, 603, 364, 96, -79, -143, -143, -117, -50, 90, 302, 528, 690, 736, 648, 436, 136, -180, -403, -432, -247, 60, 333, 447, 376, 182, -54, -283, -479, -601, -590, -427, -180, 11, 32, -113, -306, -401, -340, -172, 8, 156, 283, 398, 449, 341, 29, -403, -770, -901, -765, -490, -262, -187, -217, -206, -32, 285, 588, 686, 499, 131, -183, -244, -33, 249, 318, 1, -616, -1222, -1468, -1197, -539, 161, 568, 540, 198, -183, -357, -254, 1, 212, 265, 208, 192, 310, 488, 532, 305, -105, -407, -317, 185, 796, 1088, 830, 169, -486, -773, -616, -228, 111, 267, 291, 294, 308, 268, 114, -125, -349, -472, -485, -426, -319, -149, 93, 365, 589, 712, 746, 721, 621, 380, -21, -467, -736, -659, -280, 148, 344, 205, -133, -434, -545, -480, -337, -167, 54, 342, 607, 687, 477, 47, -385, -604, -535, -262, 66, 358, 593, 771, 858, 807, 616, 356, 131, -1, -70, -153, -311, -531, -737, -834, -757, -500, -127, 234, 441, 400, 129, -238, -522, -601, -474, -235, 1, 176, 295, 397, 506, 619, 704, 723, 638, 445, 187, -44, -165, -165, -129, -172, -324, -472, -426, -90, 419, 822, 859, 500, -28, -404, -448, -229, 16, 85, -55, -280, -430, -429, -288, -51, 250, 577, 832, 882, 646, 193, -259, -478, -375, -73, 188, 234, 79, -105, -140, 25, 282, 467, 488, 392, 307, 328, 431, 483, 334, -59, -581, -1002, -1112, -857, -386, 25, 151, -44, -388, -622, -581, -298, 32, 209, 154, -44, -212, -205, 2, 315, 568, 607, 357, -125, -657, -1004, -998, -645, -135, 265, 373, 178, -178, -522, -751, -851, -837, -688, -368, 111, 630, 992, 1022, 677, 85, -514, -886, -903, -592, -103, 377, 720, 901, 983, 1035, 1047, 929, 590, 48, -516, -847, -773, -346, 168, 459, 369, -8, -401, -552, -385, -39, 258, 354, 266, 136, 104, 193, 299, 287, 113, -114, -194, 30, 542, 1114, 1423, 1238, 587, -223, -773, -799, -383, 87, 190, -209, -832, -1184, -944, -231, 469, 686, 326, -260, -591, -439, 6, 347, 325, 19, -281, -348, -185, 46, 212, 305, 380, 435, 409, 280, 142, 113, 175, 129, -241, -909, -1542, -1733, -1347, -660, -140, -62, -298, -466, -295, 155, 583, 721, 543, 213, -97, -326, -481, -539, -433, -138, 260, 617, 840, 940, 953, 849, 553, 86, -343, -447, -108, 458, 829, 703, 150, -448, -716, -570, -232, 1, 12, -97, -146, -35, 216, 525, 803, 951, 875, 548, 98, -222, -202, 149, 591, 846, 830, 697, 652, 732, 777, 618, 278, -21, -80, 83, 244, 192, -67, -320, -345, -112, 215, 429, 429, 245, -32, -318, -528, -569, -391, -67, 215, 271, 76, -204, -357, -294, -109, 40, 86, 80, 83, 68, -54, -308, -578, -705, -649, -542, -556, -704, -791, -594, -103, 419, 638, 427, -26, -381, -420, -206, 18, 54, -108, -325, -443, -412, -285, -138, -11, 102, 208, 289, 291, 151, -157, -585, -988, -1183, -1059, -668, -209, 103, 207, 223, 339, 629, 975, 1163, 1062, 728, 339, 50, -109, -193, -244, -240, -137, 44, 206, 230, 81, -163, -352, -352, -121, 275, 689, 950, 962, 756, 494, 362, 441, 645, 784, 714, 447, 120, -117, -208, -183, -92, 46, 217, 364, 404, 314, 188, 180, 366, 649, 828, 778, 572, 421, 473, 661, 744, 524, 34, -461, -647, -387, 188, 767, 1039, 874, 376, -177, -492, -430, -98, 220, 274, 40, -243, -272, 54, 512, 706, 385, -344, -1083, -1430, -1254, -749, -255, -11, -32, -166, -238, -163, 13, 166, 171, -12, -298, -507, -487, -233, 89, 256, 154, -134, -416, -533, -469, -310, -137, 15, 123, 124, -41, -325, -544, -513, -240, 22, -26, -436, -895, -947, -384, 548, 1316, 1507, 1129, 550, 177, 155, 338, 481, 443, 230, -84, -429, -724, -854, -731, -394, -40, 108, -1, -194, -206, 74, 501, 797, 792, 552, 291, 176, 220, 336, 458, 586, 717, 783, 686, 416, 114, -21, 99, 369, 571, 557, 367, 172, 114, 199, 318, 360, 289, 146, -19, -184, -329, -414, -384, -208, 77, 372, 551, 524, 276, -103, -444, -559, -362, 58, 463, 608, 412, 19, -309, -387, -223, 8, 120, 47, -137, -296, -317, -159, 149, 515, 807, 901, 764, 482, 204, 22, -100, -273, -529, -739, -707, -362, 117, 416, 340, -6, -292, -230, 187, 691, 960, 859, 498, 94, -197, -349, -399, -379, -308, -221, -173, -187, -215, -185, -89, -19, -87, -297, -513, -570, -435, -247, -183, -281, -393, -332, -69, 218, 299, 105, -202, -377, -308, -101, 26, -56, -295, -519, -580, -448, -191, 84, 278, 304, 115, -238, -586, -714, -534, -188, 13, -174, -723, -1315, -1564, -1288, -630, 71, 523, 638, 514, 301, 97, -50, -93, 30, 328, 694, 942, 933, 676, 325, 53, -64, -67, -21, 70, 252, 518, 740, 734, 425, -50, -407, -429, -161, 105, 59, -363, -900, -1144, -861, -182, 476, 705, 394, -207, -671, -690, -263, 331, 754, 822, 596, 291, 103, 70, 76, -31, -306, -647, -869, -838, -579, -262, -84, -134, -331, -490, -458, -224, 69, 234, 162, -97, -369, -465, -308, 16, 319, 419, 263, -41, -303, -368, -231, -38, 0, -250, -745, -1269, -1545, -1391, -832, -112, 419, 477, -4, -837, -1661, -2122, -2035, -1454, -627, 121, 532, 507, 151, -266, -440, -206, 325, 795, 810, 231, -678, -1389, -1471, -921, -186, 167, -119, -806, -1343, -1294, -652, 183, 717, 691, 197, -449, -942, -1128, -1017, -730, -432, -281, -355, -589, -786, -729, -356, 173, 574, 645, 407, 70, -145, -166, -87, -57, -153, -348, -556, -697, -712, -565, -274, 52, 229, 107, -285, -702, -821, -478, 182, 790, 990, 685, 78, -484, -750, -680, -422, -172, -55, -80, -177, -261, -285, -265, -258, -316, -424, -495, -427, -201, 72, 208, 84, -259, -623, -777, -608, -190, 267, 548, 539, 270, -108, -412, -509, -397, -206, -114, -216, -434, -556, -402, 15, 464, 646, 414, -113, -636, -876, -742, -351, 86, 394, 496, 414, 235, 89, 95, 288, 556, 685, 506, 41, -476, -768, -728, -508, -379, -480, -689, -722, -385, 252, 902, 1271, 1260, 965, 549, 103, -357, -811, -1163, -1293, -1153, -811, -388, 38, 459, 859, 1156, 1227, 1020, 645, 309, 164, 185, 207, 88, -159, -394, -484, -420, -298, -194, -90, 82, 324, 515, 490, 181, -326, -853, -1247, -1439, -1403, -1108, -542, 202, 906, 1321, 1343, 1088, 795, 624, 541, 396, 110, -222, -418, -389, -239, -176, -323, -602, -809, -779, -493, -57, 392, 739, 870, 675, 122, -633, -1253, -1387, -922, -102, 619, 879, 641, 185, -123, -96, 188, 485, 562, 329, -138, -652, -999, -1024, -721, -248, 140, 253, 98, -123, -174, 24, 320, 452, 273, -115, -428, -403, 0, 572, 990, 1027, 688, 181, -217, -340, -212, -21, 25, -157, -470, -697, -662, -355, 67, 398, 521, 481, 427, 488, 677, 886, 965, 824, 491, 107, -149, -159, 60, 355, 514, 403, 60, -314, -483, -313, 142, 684, 1073, 1144, 869, 361, -168, -484, -439, -73, 378, 607, 439, -28, -468, -551, -198, 346, 687, 570, 70, -459, -646, -370, 156, 544, 503, 46, -527, -845, -725, -295, 111, 210, -29, -379, -549, -423, -135, 57, -7, -284, -576, -713, -656, -480, -283, -133, -85, -197, -473, -787, -889, -588, 47, 655, 813, 384, -332, -807, -691, -95, 506, 665, 303, -265, -615, -541, -157, 254, 474, 464, 317, 154, 70, 124, 324, 594, 774, 708, 385, -16, -237, -145, 142, 348, 261, -69, -359, -305, 169, 847, 1365, 1460, 1131, 605, 165, -37, -34, 44, 118, 216, 413, 705, 947, 945, 621, 116, -303, -448, -353, -222, -218, -315, -342, -198, 15, 86, -66, -257, -170, 320, 966, 1303, 1040, 333, -314, -457, -43, 576, 970, 956, 681, 422, 346, 415, 490, 462, 304, 47, -244, -466, -492, -271, 79, 301, 174, -280, -777, -965, -698, -143, 371, 615, 586, 435, 289, 161, 18, -116, -145, 4, 287, 560, 689, 652, 525, 402, 329, 312, 341, 395, 421, 336, 98, -237, -546, -727, -787, -803, -810, -745, -534, -222, -5, -63, -352, -579, -424, 158, 850, 1204, 1009, 465, 7, -43, 292, 722, 945, 854, 545, 183, -110, -265, -234, -11, 315, 565, 571, 308, -77, -384, -504, -467, -354, -189, 76, 469, 896, 1175, 1157, 838, 369, -38, -218, -96, 291, 797, 1186, 1221, 811, 112, -524, -764, -513, 25, 510, 696, 568, 296, 82, 37, 147, 310, 379, 235, -134, -571, -795, -588, 13, 684, 1028, 870, 384, -71, -271, -271, -287, -410, -489, -287, 227, 778, 991, 720, 194, -178, -148, 189, 525, 609, 431, 153, -77, -236, -380, -527, -628, -629, -543, -439, -365, -309, -232, -136, -59, -14, 52, 213, 456, 657, 677, 504, 284, 198, 295, 445, 466, 301, 66, -58, -4, 142, 231, 188, 48, -116, -288, -494, -726, -887, -846, -554, -112, 291, 507, 524, 451, 427, 517, 679, 789, 700, 328, -285, -955, -1419, -1481, -1130, -554, -20, 283, 326, 185, -55, -351, -656, -878, -899, -669, -272, 106, 313, 322, 234, 178, 212, 301, 366, 331, 160, -120, -408, -531, -343, 133, 647, 843, 524, -142, -697, -714, -130, 683, 1203, 1139, 617, 25, -318, -384, -370, -469, -688, -864, -820, -511, -41, 414, 706, 764, 610, 356, 161, 141, 288, 451, 453, 236, -69, -257, -214, -30, 87, 2, -234, -437, -452, -282, -61, 74, 82, 8, -102, -253, -455, -673, -815, -801, -642, -441, -316, -311, -376, -410, -339, -153, 90, 294, 345, 169, -206, -624, -857, -746, -337, 109, 270, -9, -590, -1094, -1144, -605, 318, 1204, 1657, 1537, 1020, 464, 178, 237, 475, 636, 583, 377, 196, 154, 190, 141, -85, -394, -552, -389, 43, 502, 738, 684, 492, 380, 456, 652, 801, 773, 552, 206, -164, -463, -591, -465, -80, 438, 854, 940, 627, 60, -477, -730, -610, -228, 189, 431, 399, 139, -205, -476, -577, -512, -365, -239, -199, -242, -318, -368, -358, -291, -199, -124, -88, -67, 9, 211, 529, 836, 941, 719, 227, -305, -626, -635, -447, -275, -242, -282, -220, 46, 429, 695, 647, 265, -274, -726, -916, -814, -514, -177, 25, -28, -363, -860, -1284, -1397, -1101, -504, 152, 645, 898, 974, 975, 935, 807, 560, 264, 78, 130, 374, 573, 441, -139, -987, -1691, -1859, -1382, -519, 262, 584, 381, -96, -479, -526, -237, 213, 628, 889, 961, 844, 565, 199, -115, -246, -157, 39, 165, 120, -36, -132, -28, 257, 573, 754, 738, 589, 419, 298, 224, 161, 75, -37, -151, -210, -136, 122, 519, 873, 943, 599, -35, -609, -754, -357, 326, 826, 775, 173, -601, -1050, -877, -178, 643, 1144, 1114, 676, 173, -60, 98, 522, 931, 1068, 826, 279, -375, -915, -1182, -1135, -868, -566, -405, -439, -547, -514, -200, 307, 707, 690, 183, -551, -1065, -1033, -474, 264, 764, 809, 504, 143, -14, 78, 272, 359, 224, -83, -395, -538, -440, -167, 99, 172, -40, -469, -909, -1128, -992, -542, 33, 496, 681, 568, 275, -15, -166, -145, -16, 116, 191, 205, 181, 116, -16, -219, -420, -485, -310, 79, 525, 828, 868, 663, 329, -7, -252, -337, -217, 91, 467, 700, 629, 283, -104, -258, -85, 234, 385, 187, -246, -582, -539, -115, 401, 651, 469, -6, -435, -510, -158, 426, 911, 1038, 784, 364, 69, 51, 207, 279, 76, -332, -662, -639, -251, 226, 452, 304, -14, -145, 125, 697, 1222, 1368, 1031, 369, -318, -766, -839, -552, -35, 516, 912, 1032, 868, 512, 96, -263, -483, -517, -371, -131, 51, 50, -137, -362, -441, -311, -89, 39, -4, -122, -149, -31, 117, 133, -15, -193, -233, -114, 17, 4, -161, -341, -398, -338, -272, -260, -224, -44, 259, 476, 369, -89, -657, -1006, -986, -717, -436, -281, -223, -190, -185, -264, -384, -366, -57, 440, 747, 492, -320, -1226, -1624, -1258, -417, 315, 543, 337, 91, 119, 383, 571, 408, -93, -666, -1005, -978, -670, -281, -4, 53, -88, -280, -296, 23, 633, 1253, 1535, 1304, 695, 57, -303, -312, -115, 87, 198, 246, 301, 393, 513, 644, 762, 806, 691, 379, -49, -422, -598, -588, -525, -522, -548, -462, -185, 183, 419, 374, 106, -169, -273, -187, -18, 145, 306, 496, 670, 709, 550, 287, 118, 160, 322, 382, 212, -82, -246, -115, 225, 511, 517, 235, -156, -448, -525, -372, -29, 411, 775, 836, 467, -213, -871, -1186, -1113, -916, -914, -1161, -1378, -1214, -597, 154, 571, 416, -150, -728, -997, -909, -618, -293, -7, 219, 348, 329, 196, 89, 138, 315, 404, 191, -320, -855, -1067, -819, -297, 128, 189, -104, -512, -738, -618, -176, 405, 869, 989, 681, 87, -481, -724, -575, -249, -68, -180, -438, -540, -334, 31, 235, 103, -209, -345, -74, 462, 849, 729, 113, -607, -945, -659, 102, 920, 1379, 1300, 798, 175, -261, -366, -198, 51, 217, 278, 348, 535, 792, 917, 703, 144, -507, -894, -790, -250, 449, 1001, 1211, 1026, 487, -288, -1086, -1610, -1603, -1029, -147, 618, 950, 829, 498, 232, 122, 47, -157, -517, -887, -1077, -1005, -758, -500, -360, -364, -452, -536, -536, -410, -172, 94, 274, 287, 151, -33, -166, -239, -337, -545, -836, -1055, -1016, -642, -36, 571, 954, 1000, 743, 327, -66, -283, -252, -15, 284, 460, 372, -1, -548, -1068, -1359, -1296, -897, -344, 65, 53, -466, -1275, -1913, -1934, -1200, -11, 1051, 1498, 1238, 626, 185, 228, 662, 1094, 1158, 794, 271, -44, 18, 317, 520, 376, -82, -568, -750, -497, 22, 460, 527, 183, -349, -748, -806, -550, -198, 3, -47, -253, -388, -256, 192, 831, 1445, 1835, 1899, 1648, 1174, 619, 149, -81, -1, 323, 688, 856, 712, 348, 4, -118, -12, 129, 80, -213, -573, -714, -475, 42, 546, 754, 596, 244, -20, -6, 258, 550, 597, 276, -273, -705, -672, -82, 784, 1423, 1433, 806, -55, -636, -685, -378, -125, -209, -558, -843, -799, -466, -138, -99, -394, -816, -1111, -1156, -996, -735, -440, -151, 60, 97, -73, -341, -494, -385, -51, 313, 521, 525, 417, 295, 175, 18, -163, -267, -186, 65, 334, 454, 388, 245, 141, 60, -135, -532, -1011, -1275, -1074, -416, 401, 988, 1115, 829, 380, 42, -16, 230, 691, 1187, 1498, 1452, 1035, 435, -54, -230, -136, -28, -176, -641, -1212, -1568, -1515, -1122, -653, -348, -245, -178, 71, 574, 1172, 1583, 1595, 1217, 678, 279, 207, 449, 842, 1193, 1375, 1337, 1067, 580, -41, -624, -958, -924, -605, -250, -108, -254, -552, -781, -799, -623, -351, -59, 238, 533, 763, 844, 756, 589, 466, 419, 355, 160, -150, -404, -417, -179, 113, 239, 159, 67, 199, 591, 1020, 1184, 952, 458, -37, -356, -485, -493, -412, -237, -23, 70, -102, -498, -859, -908, -621, -286, -268, -666, -1198, -1437, -1178, -605, -97, 113, 119, 173, 396, 649, 700, 478, 151, -65, -139, -250, -560, -995, -1265, -1142, -700, -270, -137, -267, -347, -97, 444, 938, 1043, 717, 238, -48, -6, 234, 450, 534, 542, 584, 696, 828, 919, 950, 916, 779, 483, 52, -357, -526, -350, 52, 401, 458, 197, -194, -474, -510, -327, -28, 309, 647, 941, 1100, 1030, 713, 244, -208, -499, -579, -483, -291, -83, 78, 154, 141, 94, 109, 262, 545, 846, 1010, 942, 684, 400, 281, 424, 766, 1114, 1260, 1100, 698, 234, -110, -273, -330, -431, -668, -997, -1256, -1265, -955, -428, 76, 303, 129, -353, -856, -1054, -782, -159, 460, 712, 456, -138, -726, -1033, -1019, -842, -677, -568, -446, -268, -123, -174, -489, -925, -1203, -1109, -676, -168, 101, -15, -432, -903, -1183, -1137, -777, -233, 292, 595, 552, 196, -293, -700, -903, -939, -929, -942, -927, -769, -426, 3, 344, 479, 437, 353, 342, 410, 474, 452, 335, 174, 27, -72, -98, -4, 244, 607, 942, 1075, 923, 569, 211, 16, -2, 36, 0, -129, -244, -213, -1, 306, 565, 669, 585, 334, -36, -447, -774, -880, -701, -328, 12, 99, -128, -508, -766, -704, -335, 142, 485, 553, 366, 61, -204, -327, -298, -193, -137, -221, -418, -569, -466, -13, 663, 1263, 1494, 1267, 740, 177, -234, -481, -649, -801, -917, -944, -877, -759, -611, -387, -25, 438, 834, 959, 751, 355, 5, -184, -307, -548, -952, -1325, -1377, -983, -338, 163, 218, -171, -725, -1116, -1189, -1001, -699, -392, -122, 70, 110, -66, -412, -754, -891, -742, -408, -90, 66, 69, 41, 81, 163, 142, -129, -658, -1264, -1661, -1613, -1082, -265, 504, 929, 902, 551, 152, -44, 52, 344, 634, 756, 681, 509, 375, 349, 379, 347, 164, -142, -436, -569, -481, -245, -7, 124, 141, 116, 129, 221, 393, 640, 952, 1276, 1498, 1489, 1210, 763, 340, 74, -49, -150, -298, -409, -306, 92, 641, 1029, 1000, 556, -48, -492, -601, -421, -122, 141, 297, 314, 171, -122, -479, -737, -756, -528, -194, 63, 149, 103, 36, 23, 69, 146, 239, 353, 464, 514, 460, 335, 244, 273, 406, 529, 528, 387, 183, -5, -193, -456, -816, -1157, -1264, -990, -400, 232, 591, 507, 61, -482, -839, -855, -566, -155, 158, 224, 34, -280, -520, -523, -248, 196, 608, 790, 638, 200, -352, -800, -996, -948, -813, -786, -944, -1182, -1274, -1056, -561, -7, 358, 435, 313, 163, 90, 74, 36, -53, -137, -138, -35, 119, 267, 405, 574, 785, 974, 1025, 870, 554, 207, -46, -163, -192, -214, -289, -421, -566, -645, -575, -315, 93, 521, 810, 876, 771, 634, 571, 564, 490, 248, -122, -432, -465, -132, 451, 1030, 1345, 1272, 872, 343, -92, -310, -341, -330, -419, -619, -790, -742, -410, 67, 430, 483, 240, -108, -362, -467, -510, -585, -666, -618, -349, 62, 393, 444, 221, -52, -87, 266, 912, 1558, 1875, 1654, 891, -203, -1270, -1921, -1913, -1279, -339, 453, 755, 521, 7, -415, -494, -231, 157, 416, 417, 236, 81, 136, 431, 804, 1005, 866, 430, -65, -356, -335, -129, 3, -122, -490, -913, -1188, -1239, -1129, -966, -814, -687, -599, -588, -674, -799, -836, -685, -373, -49, 131, 125, 24, -51, -48, 5, 83, 223, 483, 827, 1072, 998, 535, -131, -657, -782, -514, -109, 132, 82, -168, -422, -547, -549, -510, -497, -503, -481, -398, -255, -71, 138, 359, 568, 728, 813, 825, 783, 696, 558, 369, 171, 43, 54, 202, 401, 533, 527, 396, 211, 37, -111, -261, -427, -590, -712, -780, -825, -887, -956, -940, -719, -245, 383, 958, 1280, 1285, 1081, 848, 706, 641, 561, 413, 244, 153, 191, 303, 381, 373, 316, 280, 268, 208, 33, -208, -377, -372, -264, -265, -541, -1022, -1408, -1385, -880, -136, 469, 691, 552, 274, 80, 74, 241, 526, 861, 1142, 1227, 1017, 566, 91, -168, -128, 68, 183, 90, -123, -260, -214, -83, -77, -314, -702, -993, -981, -655, -195, 169, 311, 268, 178, 169, 273, 417, 476, 361, 87, -220, -392, -327, -56, 269, 478, 486, 339, 158, 53, 60, 141, 217, 218, 113, -78, -288, -427, -435, -322, -164, -41, 11, 18, 12, 3, -13, -7, 96, 353, 715, 1016, 1082, 877, 545, 289, 199, 183, 78, -160, -387, -385, -74, 381, 685, 632, 249, -248, -628, -784, -747, -596, -394, -179, 3, 102, 106, 79, 128, 327, 663, 1037, 1327, 1436, 1319, 983, 505, 27, -278, -298, -44, 342, 668, 784, 635, 260, -229, -669, -876, -732, -275, 284, 682, 782, 670, 567, 617, 757, 782, 553, 146, -213, -345, -259, -93, 56, 233, 529, 915, 1188, 1121, 685, 116, -261, -282, -47, 201, 294, 253, 203, 230, 308, 363, 359, 307, 218, 72, -121, -268, -224, 63, 460, 710, 642, 337, 63, 45, 264, 479, 457, 177, -172, -389, -432, -414, -436, -454, -330, -2, 412, 690, 701, 526, 381, 418, 585, 677, 516, 100, -391, -738, -821, -681, -459, -296, -255, -302, -350, -317, -178, 20, 211, 350, 451, 544, 619, 600, 408, 49, -352, -620, -644, -447, -148, 137, 359, 501, 527, 374, 27, -414, -759, -840, -633, -258, 119, 406, 586, 644, 510, 107, -521, -1140, -1422, -1159, -430, 414, 969, 1012, 623, 101, -228, -195, 159, 636, 1007, 1124, 985, 705, 434, 262, 184, 139, 89, 66, 135, 320, 557, 722, 719, 544, 268, -16, -257, -438, -556, -598, -557, -447, -302, -162, -65, -52, -167, -415, -706, -859, -699, -200, 451, 951, 1067, 793, 325, -95, -344, -440, -454, -422, -343, -249, -234, -370, -610, -778, -710, -409, -76, 44, -127, -443, -648, -585, -290, 83, 419, 708, 993, 1259, 1405, 1321, 981, 475, -51, -456, -626, -492, -71, 471, 842, 776, 239, -493, -999, -1010, -618, -184, -41, -231, -509, -586, -380, -45, 204, 287, 283, 298, 343, 358, 322, 313, 410, 551, 523, 143, -516, -1093, -1169, -607, 301, 1021, 1141, 645, -115, -703, -879, -689, -342, -28, 174, 266, 253, 125, -101, -349, -517, -551, -492, -443, -462, -495, -408, -105, 360, 784, 937, 735, 315, -51, -152, 19, 268, 346, 149, -216, -520, -582, -388, -92, 104, 81, -137, -408, -550, -452, -141, 211, 385, 257, -90, -393, -384, -6, 520, 844, 745, 302, -180, -414, -341, -127, 16, 19, -29, -27, 9, -52, -316, -733, -1124, -1317, -1276, -1102, -920, -791, -702, -634, -604, -636, -708, -743, -666, -457, -159, 168, 473, 688, 714, 464, -29, -561, -826, -627, -34, 639, 1039, 1001, 629, 173, -150, -259, -171, 67, 399, 705, 813, 587, 51, -578, -1029, -1153, -1014, -796, -623, -479, -272, 8, 231, 226, -56, -463, -729, -659, -252, 318, 840, 1168, 1263, 1151, 878, 496, 66, -342, -669, -868, -906, -767, -480, -149, 57, -3, -348, -820, -1162, -1183, -884, -470, -210, -265, -573, -893, -948, -603, 46, 717, 1097, 1039, 655, 241, 75, 213, 448, 476, 148, -385, -773, -733, -284, 252, 481, 228, -352, -913, -1175, -1111, -908, -779, -788, -826, -733, -448, -62, 258, 393, 341, 199, 77, 26, 27, 33, 11, -36, -96, -165, -258, -370, -456, -439, -262, 54, 395, 613, 619, 449, 237, 121, 143, 221, 215, 38, -266, -543, -625, -448, -107, 194, 282, 123, -139, -294, -222, -3, 138, 23, -313, -616, -593, -167, 438, 848, 812, 391, -116, -429, -490, -464, -536, -717, -838, -714, -323, 167, 546, 708, 683, 549, 345, 86, -162, -267, -129, 188, 445, 393, -12, -531, -802, -611, -60, 503, 729, 488, -86, -719, -1168, -1331, -1248, -1038, -831, -707, -666, -630, -508, -266, 35, 290, 435, 499, 569, 683, 791, 803, 696, 559, 513, 587, 670, 601, 324, -50, -340, -448, -444, -484, -647, -859, -959, -857, -624, -430, -393, -471, -499, -321, 87, 601, 1025, 1197, 1054, 651, 139, -278, -426, -243, 155, 515, 580, 252, -327, -861, -1078, -901, -493, -133, -27, -177, -405, -491, -335, -17, 278, 407, 367, 283, 292, 420, 555, 528, 255, -186, -591, -743, -551, -99, 409, 772, 893, 810, 641, 489, 377, 261, 98, -100, -277, -386, -447, -523, -639, -720, -639, -339, 69, 352, 328, 25, -326, -481, -380, -184, -112, -214, -319, -192, 238, 793, 1182, 1236, 1019, 732, 525, 388, 220, -24, -278, -437, -475, -464, -484, -520, -469, -254, 70, 349, 451, 382, 253, 169, 132, 69, -58, -195, -236, -143, 9, 109, 116, 105, 181, 362, 541, 571, 379, 14, -418, -841, -1215, -1490, -1561, -1336, -843, -281, 78, 99, -107, -251, -90, 370, 885, 1165, 1093, 788, 474, 289, 210, 137, 31, -33, 48, 291, 584, 781, 809, 694, 490, 195, -224, -738, -1182, -1344, -1134, -705, -354, -290, -453, -568, -407, -14, 317, 315, -5, -333, -323, 82, 598, 840, 674, 337, 229, 561, 1170, 1660, 1727, 1383, 905, 570, 446, 391, 239, -38, -323, -493, -539, -565, -654, -768, -763, -519, -56, 456, 812, 900, 753, 491, 227, 8, -169, -300, -334, -197, 141, 627, 1126, 1484, 1609, 1499, 1227, 881, 523, 178, -139, -403, -575, -636, -610, -561, -550, -595, -648, -619, -442, -120, 255, 553, 661, 555, 316, 84, -8, 103, 394, 780, 1166, 1476, 1671, 1737, 1679, 1514, 1259, 929, 533, 84, -382, -799, -1084, -1177, -1069, -804, -456, -101, 185, 336, 299, 82, -220, -444, -428, -113, 408, 925, 1218, 1166, 803, 292, -157, -383, -327, -53, 285, 519, 533, 317, -44, -421, -707, -853, -873, -811, -718, -624, -530, -410, -233, 13, 290, 510, 574, 430, 106, -296, -650, -839, -799, -530, -110, 311, 572, 569, 340, 43, -143, -151, -69, -62, -226, -499, -711, -726, -555, -341, -229, -254, -323, -316, -192, -17, 106, 133, 98, 79, 121, 206, 274, 277, 203, 70, -83, -213, -255, -146, 140, 555, 958, 1179, 1108, 770, 317, -54, -224, -207, -113, -46, -16, 40, 178, 354, 436, 312, 8, -308, -436, -290, 37, 357, 520, 507, 398, 273, 141, -33, -267, -509, -669, -685, -564, -374, -198, -106, -132, -265, -449, -587, -594, -447, -202, 41, 219, 328, 396, 428, 390, 245, 17, -217, -385, -461, -461, -395, -243, -13, 193, 214, -46, -491, -847, -862, -512, -40, 230, 145, -197, -572, -840, -1011, -1135, -1164, -971, -518, 11, 302, 179, -196, -430, -224, 349, 892, 1018, 671, 154, -146, -79, 196, 404, 390, 197, -51, -297, -555, -810, -953, -863, -542, -172, 20, -43, -229, -307, -157, 136, 374, 414, 261, 13, -243, -492, -725, -893, -912, -744, -450, -144, 97, 293, 490, 676, 754, 625, 307, -43, -239, -212, -62, 37, -11, -152, -228, -109, 221, 671, 1111, 1437, 1584, 1518, 1239, 804, 328, -51, -242, -228, -71, 127, 265, 269, 114, -157, -440, -610, -601, -444, -245, -99, -24, 36, 135, 254, 313, 263, 147, 90, 189, 426, 660, 732, 561, 186, -285, -739, -1085, -1251, -1185, -882, -422, 48, 393, 548, 546, 459, 348, 248, 193, 236, 415, 696, 968, 1099, 1021, 764, 420, 83, -184, -333, -318, -134, 137, 337, 320, 64, -304, -609, -750, -751, -684, -558, -305, 121, 643, 1069, 1225, 1092, 810, 545, 362, 218, 59, -90, -154, -109, -45, -85, -261, -453, -488, -295, 19, 262, 306, 165, -55, -263, -423, -510, -463, -214, 213, 661, 906, 819, 467, 69, -169, -200, -137, -145, -301, -536, -689, -628, -346, 13, 241, 193, -92, -409, -519, -344, -33, 156, 79, -167, -329, -231, 59, 300, 310, 132, -28, -24, 79, 92, -79, -310, -362, -120, 288, 616, 731, 712, 723, 814, 868, 753, 498, 288, 267, 370, 382, 164, -169, -344, -188, 177, 433, 336, -76, -551, -847, -907, -833, -723, -584, -380, -137, 51, 119, 102, 97, 143, 180, 136, 49, 68, 298, 654, 891, 812, 453, 57, -135, -88, 36, 62, -51, -219, -360, -484, -615, -691, -565, -159, 398, 822, 892, 643, 327, 178, 214, 250, 115, -164, -389, -383, -158, 109, 238, 183, 27, -120, -205, -216, -138, 38, 274, 461, 478, 293, 0, -248, -356, -328, -226, -109, 5, 114, 210, 294, 389, 505, 588, 525, 243, -174, -499, -516, -216, 167, 344, 204, -104, -322, -308, -132, 49, 161, 260, 409, 554, 556, 340, 3, -259, -347, -342, -403, -567, -678, -538, -114, 390, 707, 725, 556, 398, 347, 359, 356, 342, 376, 453, 464, 312, 64, -51, 162, 652, 1106, 1197, 846, 268, -233, -521, -668, -793, -874, -768, -390, 132, 531, 613, 420, 163, 16, -26, -88, -238, -397, -416, -237, 36, 234, 252, 113, -94, -300, -466, -547, -481, -258, 18, 168, 62, -250, -575, -729, -683, -552, -445, -342, -128, 247, 669, 909, 821, 470, 88, -106, -64, 91, 198, 179, 69, -50, -120, -131, -96, -31, 42, 113, 186, 292, 446, 600, 639, 464, 83, -357, -677, -795, -786, -784, -829, -809, -570, -88, 455, 785, 735, 366, -83, -373, -396, -202, 73, 282, 302, 69, -374, -847, -1094, -934, -402, 248, 710, 817, 615, 263, -120, -507, -867, -1083, -981, -495, 202, 770, 928, 663, 219, -114, -231, -242, -323, -525, -737, -808, -684, -436, -179, 17, 126, 117, -40, -306, -524, -485, -106, 460, 895, 931, 548, -35, -541, -820, -882, -794, -579, -252, 83, 229, 29, -451, -911, -1006, -629, 1, 507, 645, 458, 182, 9, -54, -129, -291, -483, -580, -533, -428, -403, -505, -646, -687, -564, -341, -141, -46, -44, -73, -83, -73, -63, -61, -61, -65, -108, -244, -505, -857, -1199, -1397, -1350, -1045, -579, -111, 221, 375, 399, 367, 304, 168, -80, -406, -689, -798, -689, -427, -124, 139, 340, 459, 448, 265, -55, -373, -530, -479, -335, -282, -409, -618, -713, -569, -247, 59, 182, 91, -113, -310, -439, -497, -481, -365, -142, 122, 310, 323, 172, -20, -102, 8, 279, 591, 818, 885, 788, 576, 321, 81, -108, -235, -281, -225, -65, 143, 290, 270, 71, -200, -393, -423, -338, -263, -280, -360, -406, -351, -204, -6, 233, 527, 833, 1010, 915, 562, 183, 60, 257, 498, 353, -376, -1427, -2205, -2226, -1475, -415, 369, 581, 344, 19, -103, 31, 286, 482, 503, 321, -18, -394, -640, -633, -388, -67, 153, 242, 344, 616, 1024, 1301, 1149, 514, -314, -896, -960, -593, -144, 53, -97, -446, -751, -849, -716, -432, -124, 73, 58, -182, -535, -796, -791, -496, -51, 340, 555, 609, 604, 622, 669, 687, 619, 448, 207, -39, -216, -267, -191, -65, -14, -131, -403, -701, -859, -783, -497, -125, 196, 379, 419, 380, 370, 481, 728, 1008, 1151, 1038, 723, 423, 366, 604, 958, 1162, 1087, 843, 664, 677, 773, 707, 327, -268, -801, -1026, -915, -639, -398, -263, -158, 26, 320, 654, 926, 1079, 1106, 1021, 845, 621, 430, 363, 449, 625, 779, 839, 833, 844, 909, 957, 843, 475, -89, -662, -1034, -1107, -932, -640, -347, -107, 47, 70, -93, -427, -788, -957, -792, -352, 104, 294, 116, -273, -595, -663, -516, -347, -306, -361, -336, -88, 348, 778, 994, 919, 630, 267, -61, -303, -428, -421, -313, -212, -247, -459, -720, -804, -561, -63, 429, 656, 549, 268, 49, 11, 83, 97, -40, -265, -401, -307, 29, 491, 909, 1148, 1158, 961, 626, 240, -117, -390, -549, -599, -561, -473, -372, -279, -187, -64, 109, 304, 432, 400, 186, -116, -347, -373, -175, 154, 480, 706, 809, 809, 739, 634, 546, 535, 631, 794, 921, 908, 720, 418, 114, -91, -151, -64, 143, 431, 727, 942, 1003, 906, 719, 540, 425, 354, 268, 133, -12, -88, -39, 109, 264, 332, 297, 235, 257, 419, 675, 895, 950, 795, 491, 162, -76, -182, -182, -140, -116, -140, -218, -330, -447, -538, -590, -606, -591, -526, -368, -93, 250, 539, 649, 562, 397, 330, 441, 638, 727, 583, 270, -6, -78, 43, 191, 197, 38, -171, -308, -368, -460, -676, -987, -1253, -1343, -1224, -960, -623, -243, 159, 511, 687, 608, 343, 92, 58, 287, 642, 908, 964, 850, 695, 599, 559, 509, 404, 266, 139, 34, -93, -289, -532, -719, -725, -505, -144, 176, 301, 204, 22, -41, 139, 509, 861, 987, 830, 524, 291, 270, 427, 601, 639, 508, 294, 117, 46, 82, 187, 320, 455, 566, 637, 664, 654, 593, 442, 173, -169, -456, -544, -387, -91, 163, 261, 232, 203, 265, 393, 486, 481, 406, 324, 250, 143, -21, -183, -237, -155, -48, -76, -279, -502, -518, -238, 190, 514, 590, 491, 370, 262, 40, -414, -997, -1395, -1339, -879, -371, -170, -310, -479, -328, 176, 707, 874, 581, 100, -190, -149, 46, 119, -24, -231, -289, -156, 13, 48, -64, -180, -167, -53, 10, -90, -324, -555, -690, -750, -797, -828, -757, -520, -188, 64, 108, -20, -150, -148, -41, 22, -60, -233, -333, -266, -110, -47, -181, -429, -583, -503, -235, 25, 100, -52, -332, -605, -787, -849, -783, -580, -270, 56, 275, 311, 201, 61, -7, 6, 36, 13, -65, -150, -191, -197, -214, -278, -375, -453, -460, -378, -221, -21, 176, 313, 331, 217, 29, -118, -131, -6, 156, 224, 127, -93, -313, -402, -298, -23, 333, 658, 847, 835, 636, 334, 51, -130, -207, -244, -297, -362, -380, -307, -171, -67, -77, -202, -352, -413, -332, -148, 47, 172, 183, 81, -102, -317, -502, -606, -632, -652, -762, -1004, -1304, -1497, -1432, -1078, -549, -35, 312, 454, 440, 340, 194, 9, -194, -371, -462, -451, -384, -340, -375, -476, -577, -611, -569, -495, -451, -459, -493, -506, -473, -402, -321, -253, -222, -255, -378, -571, -735, -721, -442, 25, 444, 555, 280, -205, -588, -647, -416, -128, -4, -71, -172, -141, 33, 205, 224, 82, -93, -177, -174, -194, -306, -437, -412, -122, 354, 791, 967, 815, 431, -21, -413, -667, -734, -588, -269, 88, 309, 284, 56, -213, -372, -372, -278, -174, -86, 16, 155, 284, 321, 219, 11, -221, -412, -543, -628, -660, -597, -389, -40, 377, 738, 927, 897, 692, 419, 183, 14, -141, -361, -635, -825, -753, -361, 184, 578, 594, 269, -124, -301, -194, 21, 108, -18, -258, -468, -624, -807, -1053, -1239, -1168, -771, -224, 180, 295, 242, 275, 496, 732, 689, 259, -343, -755, -776, -524, -294, -264, -355, -357, -169, 94, 235, 171, 3, -117, -158, -214, -365, -552, -609, -436, -117, 143, 203, 93, -40, -91, -75, -64, -77, -60, 17, 75, -34, -371, -803, -1066, -967, -528, 36, 486, 696, 671, 465, 124, -291, -674, -894, -897, -766, -651, -627, -608, -415, 45, 672, 1218, 1465, 1362, 1026, 630, 283, 13, -197, -353, -438, -420, -261, 56, 501, 943, 1185, 1071, 620, 50, -358, -456, -322, -167, -141, -221, -281, -240, -126, -1, 155, 432, 844, 1215, 1269, 875, 222, -285, -359, -88, 132, -34, -545, -993, -962, -395, 352, 811, 767, 356, -129, -520, -846, -1176, -1448, -1473, -1130, -522, 73, 435, 558, 615, 741, 892, 896, 663, 299, 32, 0, 126, 206, 104, -111, -244, -157, 91, 301, 290, 40, -305, -573, -677, -668, -669, -788, -1043, -1351, -1565, -1552, -1271, -808, -337, -10, 140, 206, 302, 453, 555, 473, 160, -287, -692, -915, -923, -758, -473, -105, 288, 600, 700, 522, 133, -299, -605, -705, -616, -397, -94, 265, 647, 1006, 1307, 1544, 1699, 1697, 1411, 770, -105, -896, -1235, -952, -211, 588, 1062, 1069, 742, 338, 49, -73, -74, -5, 111, 256, 386, 446, 404, 264, 49, -202, -409, -442, -182, 357, 967, 1326, 1221, 721, 150, -156, -108, 101, 199, 95, -52, -12, 259, 549, 569, 220, -306, -689, -737, -509, -212, 2, 134, 255, 363, 362, 173, -138, -382, -400, -221, -47, -81, -343, -662, -821, -735, -506, -313, -273, -374, -531, -658, -711, -664, -499, -226, 78, 289, 301, 115, -147, -326, -321, -155, 56, 190, 187, 74, -64, -142, -120, -34, 34, 8, -117, -265, -331, -267, -127, -10, 25, 24, 74, 202, 320, 298, 98, -156, -260, -95, 265, 612, 768, 719, 587, 487, 421, 299, 66, -209, -376, -329, -95, 197, 436, 605, 753, 905, 1008, 976, 769, 445, 123, -93, -165, -112, 15, 159, 267, 319, 351, 449, 668, 956, 1154, 1116, 856, 582, 546, 824, 1217, 1391, 1146, 588, 54, -149, 22, 362, 591, 567, 330, 4, -319, -598, -783, -791, -569, -181, 193, 381, 359, 266, 273, 430, 615, 653, 475, 177, -59, -109, 24, 241, 432, 533, 528, 420, 230, -10, -258, -482, -666, -784, -779, -584, -199, 249, 548, 526, 190, -269, -600, -661, -485, -217, 21, 204, 361, 511, 623, 653, 591, 462, 290, 80, -147, -323, -347, -175, 108, 326, 333, 143, -78, -154, -36, 159, 270, 239, 156, 164, 320, 528, 614, 466, 120, -254, -468, -434, -210, 34, 139, 54, -121, -209, -94, 176, 410, 422, 200, -65, -125, 111, 492, 729, 641, 301, -17, -63, 213, 653, 1032, 1223, 1228, 1116, 935, 705, 457, 246, 120, 76, 78, 117, 242, 506, 865, 1144, 1141, 798, 286, -96, -135, 142, 493, 651, 515, 206, -46, -66, 162, 510, 797, 895, 793, 582, 393, 299, 253, 121, -190, -617, -932, -889, -428, 247, 790, 950, 742, 400, 173, 134, 177, 162, 61, -32, -36, 0, -86, -423, -961, -1458, -1620, -1283, -520, 396, 1128, 1413, 1173, 533, -229, -804, -988, -770, -316, 143, 445, 555, 539, 479, 409, 321, 207, 90, 19, 41, 160, 331, 467, 467, 272, -94, -519, -849, -978, -909, -732, -548, -402, -278, -158, -58, -4, 35, 154, 433, 825, 1144, 1182, 874, 362, -110, -391, -505, -582, -679, -708, -525, -114, 337, 555, 397, -27, -433, -580, -431, -146, 77, 156, 155, 197, 355, 590, 790, 834, 676, 377, 98, 15, 202, 550, 807, 744, 345, -152, -408, -235, 249, 712, 852, 629, 270, 79, 195, 501, 725, 647, 243, -313, -767, -930, -770, -415, -61, 146, 206, 248, 423, 763, 1124, 1281, 1093, 612, 45, -408, -686, -846, -950, -968, -806, -429, 54, 441, 579, 470, 234, -13, -249, -517, -819, -1066, -1133, -968, -652, -333, -106, 37, 170, 335, 493, 570, 522, 374, 174, -44, -283, -535, -753, -859, -797, -569, -241, 87, 328, 417, 342, 148, -66, -198, -206, -145, -130, -249, -486, -727, -841, -755, -486, -97, 337, 745, 1045, 1150, 1015, 680, 255, -136, -430, -613, -679, -587, -301, 134, 559, 763, 622, 196, -306, -661, -760, -645, -445, -280, -207, -225, -289, -338, -313, -191, -8, 166, 288, 390, 557, 830, 1125, 1256, 1071, 590, 17, -390, -499, -369, -168, -20, 74, 194, 379, 570, 650, 554, 325, 92, -26, 8, 145, 282, 324, 221, -14, -309, -553, -650, -565, -336, -48, 220, 426, 547, 567, 464, 244, -40, -300, -458, -484, -396, -228, -11, 210, 356, 323, 47, -421, -914, -1222, -1219, -931, -522, -186, -46, -97, -228, -296, -213, 0, 218, 299, 179, -69, -285, -349, -287, -246, -354, -572, -701, -540, -91, 392, 569, 276, -334, -896, -1090, -860, -420, -71, 9, -147, -383, -545, -567, -467, -292, -87, 105, 252, 334, 363, 359, 319, 200, -41, -385, -724, -910, -856, -605, -309, -126, -111, -184, -201, -66, 185, 417, 496, 399, 240, 181, 297, 509, 637, 553, 294, 51, 12, 207, 481, 617, 510, 236, -37, -207, -299, -400, -538, -642, -626, -496, -360, -317, -344, -302, -87, 234, 453, 394, 98, -187, -205, 91, 494, 708, 594, 264, -32, -120, -21, 87, 30, -244, -647, -1018, -1214, -1160, -875, -471, -117, 26, -102, -423, -755, -914, -814, -485, -41, 386, 682, 772, 641, 351, 30, -203, -311, -357, -433, -559, -656, -625, -461, -292, -272, -437, -657, -745, -627, -422, -342, -503, -806, -1011, -924, -554, -107, 164, 140, -106, -375, -477, -336, -16, 334, 578, 653, 606, 540, 541, 610, 669, 629, 473, 265, 94, 2, -38, -76, -119, -137, -117, -114, -217, -447, -696, -781, -589, -189, 194, 349, 234, 9, -96, 30, 301, 497, 438, 114, -301, -570, -534, -214, 208, 506, 548, 369, 144, 47, 129, 282, 338, 212, -29, -227, -265, -156, -15, 45, -4, -124, -260, -373, -411, -285, 77, 618, 1096, 1192, 747, -72, -837, -1124, -812, -148, 443, 657, 469, 65, -349, -691, -974, -1196, -1279, -1120, -700, -129, 404, 750, 857, 771, 584, 381, 224, 139, 113, 92, 9, -167, -419, -680, -885, -986, -953, -773, -473, -150, 46, 4, -256, -573, -755, -741, -654, -693, -931, -1221, -1296, -1001, -441, 88, 335, 286, 153, 162, 348, 519, 437, 18, -592, -1141, -1453, -1522, -1453, -1332, -1148, -841, -399, 95, 506, 742, 810, 781, 709, 583, 358, 31, -310, -523, -510, -298, -26, 161, 223, 237, 307, 445, 538, 439, 113, -299, -576, -583, -393, -228, -276, -526, -769, -756, -393, 183, 715, 986, 942, 680, 342, 33, -200, -331, -321, -130, 233, 680, 1050, 1174, 960, 452, -166, -652, -830, -693, -409, -207, -219, -395, -557, -552, -363, -95, 142, 338, 556, 825, 1044, 1025, 643, -31, -751, -1247, -1390, -1226, -885, -467, -11, 439, 769, 833, 569, 86, -380, -623, -602, -444, -317, -288, -288, -209, -25, 163, 215, 76, -165, -333, -291, -40, 281, 493, 484, 277, 6, -168, -150, 41, 287, 438, 395, 163, -172, -509, -787, -1006, -1177, -1275, -1236, -1008, -620, -192, 127, 271, 290, 302, 379, 477, 473, 285, -37, -328, -438, -352, -196, -108, -109, -81, 120, 504, 903, 1091, 954, 581, 172, -126, -323, -524, -780, -999, -1012, -737, -277, 133, 310, 253, 132, 134, 311, 553, 698, 672, 531, 393, 331, 335, 351, 362, 405, 529, 726, 911, 977, 861, 578, 197, -217, -632, -1017, -1302, -1371, -1122, -554, 181, 829, 1161, 1100, 761, 362, 86, -9, 16, 83, 173, 323, 551, 797, 936, 879, 648, 377, 209, 187, 234, 223, 83, -149, -379, -528, -584, -584, -560, -508, -414, -281, -146, -49, 6, 54, 134, 248, 358, 417, 421, 416, 464, 571, 657, 591, 289, -201, -695, -967, -906, -594, -261, -117, -192, -321, -288, -15, 357, 589, 534, 268, 19, -28, 108, 244, 191, -65, -358, -472, -317, 16, 343, 526, 549, 493, 457, 484, 550, 581, 498, 259, -96, -451, -666, -676, -554, -454, -488, -619, -695, -587, -329, -94, -42, -159, -266, -186, 86, 372, 475, 372, 227, 230, 408, 602, 624, 447, 232, 178, 330, 540, 600, 429, 138, -73, -77, 104, 355, 559, 665, 658, 527, 258, -124, -527, -822, -904, -753, -438, -80, 196, 311, 262, 136, 64, 145, 371, 623, 755, 711, 563, 448, 446, 518, 552, 477, 333, 228, 239, 346, 457, 478, 380, 200, 3, -143, -188, -110, 45, 167, 138, -66, -330, -465, -360, -81, 170, 232, 105, -49, -58, 122, 407, 682, 907, 1102, 1251, 1256, 1003, 494, -101, -540, -676, -553, -345, -210, -187, -213, -208, -132, 15, 230, 492, 730, 828, 694, 339, -109, -495, -727, -807, -760, -567, -179, 388, 991, 1411, 1487, 1226, 775, 300, -110, -442, -691, -833, -859, -820, -800, -818, -779, -553, -122, 339, 579, 465, 110, -224, -353, -317, -309, -473, -758, -968, -959, -752, -493, -286, -105, 145, 489, 813, 962, 876, 648, 434, 317, 258, 173, 42, -53, -8, 208, 515, 764, 833, 681, 360, -9, -293, -409, -367, -270, -226, -253, -259, -138, 106, 336, 405, 333, 330, 615, 1170, 1684, 1775, 1311, 550, -56, -240, -111, -13, -168, -457, -543, -218, 365, 792, 740, 247, -320, -576, -400, 18, 381, 515, 450, 317, 212, 146, 98, 77, 117, 235, 395, 532, 592, 544, 369, 51, -386, -853, -1197, -1278, -1052, -611, -134, 205, 318, 220, 11, -179, -249, -160, 51, 295, 483, 574, 569, 488, 341, 145, -48, -160, -139, -21, 78, 57, -76, -210, -232, -159, -141, -317, -661, -961, -972, -617, -54, 453, 732, 798, 771, 734, 664, 496, 228, -51, -255, -378, -502, -698, -941, -1123, -1147, -998, -730, -391, 0, 415, 764, 918, 815, 534, 252, 108, 86, 49, -110, -350, -507, -445, -191, 83, 213, 171, 79, 78, 207, 380, 473, 424, 261, 51, -157, -345, -491, -551, -474, -246, 77, 381, 530, 437, 113, -302, -595, -580, -230, 289, 708, 825, 630, 287, -13, -187, -266, -299, -272, -134, 98, 287, 263, -14, -387, -611, -565, -355, -197, -203, -262, -159, 190, 614, 807, 601, 120, -317, -465, -341, -171, -166, -336, -513, -529, -371, -161, -16, 62, 167, 344, 514, 520, 264, -186, -643, -928, -989, -903, -784, -689, -595, -454, -265, -86, 4, -21, -124, -219, -240, -179, -85, -15, 7, 14, 50, 140, 271, 411, 547, 681, 807, 885, 848, 654, 321, -73, -439, -710, -838, -780, -505, -60, 396, 654, 595, 324, 128, 271, 767, 1327, 1567, 1303, 681, 53, -317, -431, -480, -617, -795, -822, -578, -151, 222, 355, 257, 82, -27, -51, -62, -102, -128, -82, 13, 41, -101, -385, -654, -764, -718, -658, -712, -856, -912, -710, -255, 244, 531, 482, 193, -113, -251, -190, -52, -5, -145, -431, -710, -813, -665, -347, -58, 21, -139, -402, -570, -557, -456, -439, -579, -760, -779, -544, -186, 33, -41, -322, -542, -492, -189, 159, 357, 390, 392, 466, 555, 484, 150, -345, -756, -884, -732, -460, -215, -3, 265, 615, 909, 933, 598, 61, -365, -445, -203, 115, 250, 140, -60, -135, -4, 222, 361, 314, 143, -15, -79, -90, -153, -325, -547, -696, -687, -535, -321, -109, 108, 378, 694, 954, 1018, 830, 494, 224, 196, 404, 664, 746, 550, 181, -139, -245, -138, 35, 118, 58, -69, -152, -144, -90, -87, -198, -414, -656, -820, -816, -609, -248, 133, 363, 326, 52, -283, -475, -435, -267, -182, -321, -620, -844, -769, -362, 173, 546, 575, 302, -65, -319, -388, -350, -338, -425, -578, -697, -696, -572, -416, -361, -498, -810, -1160, -1359, -1283, -964, -574, -307, -237, -274, -244, -44, 265, 517, 574, 444, 262, 171, 212, 310, 360, 319, 214, 91, -36, -173, -292, -317, -188, 56, 275, 323, 176, -26, -94, 58, 332, 529, 525, 373, 247, 273, 416, 509, 416, 156, -101, -190, -81, 97, 188, 145, 72, 119, 334, 602, 722, 568, 192, -205, -417, -353, -84, 244, 521, 725, 876, 962, 903, 620, 132, -398, -752, -791, -557, -234, 12, 158, 309, 560, 850, 973, 751, 221, -340, -605, -442, -31, 284, 269, -28, -341, -425, -264, -69, -69, -304, -581, -631, -329, 205, 689, 865, 646, 141, -428, -852, -1020, -944, -735, -529, -402, -332, -222, 7, 327, 585, 602, 300, -223, -745, -1043, -1022, -737, -335, 35, 288, 405, 413, 350, 240, 81, -133, -384, -583, -605, -373, 39, 418, 526, 286, -133, -433, -397, -60, 340, 568, 579, 511, 514, 575, 519, 192, -354, -859, -1039, -809, -349, 40, 164, 38, -158, -242, -137, 114, 405, 626, 705, 624, 439, 255, 166, 171, 165, 32, -231, -466, -454, -102, 459, 958, 1179, 1106, 893, 701, 565, 408, 165, -111, -293, -296, -164, -20, 60, 123, 271, 530, 790, 889, 758, 494, 285, 243, 322, 370, 284, 113, 18, 136, 459, 838, 1089, 1114, 933, 642, 324, 16, -276, -539, -727, -795, -736, -600, -462, -369, -299, -183, 31, 320, 581, 698, 637, 462, 281, 148, 19, -188, -484, -754, -834, -646, -282, 46, 168, 70, -117, -259, -325, -388, -511, -644, -657, -457, -103, 214, 314, 156, -143, -410, -540, -535, -455, -343, -203, -36, 125, 235, 269, 258, 252, 275, 303, 304, 285, 301, 398, 562, 704, 726, 593, 364, 141, -3, -76, -139, -242, -368, -432, -336, -49, 354, 722, 902, 835, 580, 265, 7, -148, -213, -205, -112, 70, 289, 428, 393, 219, 76, 147, 452, 799, 912, 669, 216, -143, -208, -45, 84, -10, -267, -417, -246, 184, 574, 634, 340, -65, -289, -241, -72, -1, -101, -250, -274, -113, 147, 392, 600, 834, 1114, 1338, 1340, 1062, 644, 343, 310, 459, 534, 341, -72, -452, -556, -364, -99, -39, -293, -723, -1065, -1135, -951, -685, -512, -486, -528, -505, -336, -32, 320, 609, 745, 694, 495, 248, 71, 33, 96, 139, 28, -273, -670, -987, -1079, -917, -587, -207, 142, 430, 627, 665, 476, 72, -404, -733, -740, -418, 73, 512, 750, 769, 641, 444, 218, -9, -185, -228, -90, 183, 454, 560, 415, 62, -340, -606, -608, -343, 69, 445, 645, 647, 548, 468, 439, 364, 112, -337, -814, -1044, -846, -273, 415, 941, 1182, 1206, 1140, 1034, 830, 474, 20, -373, -566, -553, -447, -354, -261, -45, 384, 957, 1441, 1584, 1281, 646, -59, -585, -818, -810, -700, -599, -503, -300, 117, 724, 1299, 1527, 1215, 462, -354, -817, -753, -343, 30, 91, -135, -374, -340, 40, 596, 1070, 1285, 1212, 911, 454, -94, -642, -1074, -1300, -1315, -1183, -968, -681, -309, 107, 441, 540, 359, 5, -334, -529, -589, -604, -634, -649, -586, -442, -305, -275, -358, -443, -389, -149, 187, 456, 527, 367, 32, -383, -774, -1036, -1081, -874, -462, 30, 454, 694, 694, 463, 68, -371, -709, -830, -723, -492, -286, -185, -155, -100, 28, 180, 262, 244, 198, 211, 272, 253, 32, -361, -732, -864, -708, -423, -229, -200, -204, -48, 304, 659, 742, 433, -108, -567, -715, -571, -349, -259, -333, -429, -375, -121, 223, 492, 580, 501, 347, 191, 49, -108, -294, -462, -538, -478, -329, -204, -198, -293, -363, -279, -48, 156, 111, -242, -707, -932, -691, -78, 539, 805, 653, 336, 182, 284, 427, 301, -173, -738, -1004, -792, -308, 21, -87, -546, -974, -1004, -568, 89, 623, 829, 734, 506, 305, 202, 189, 218, 218, 122, -95, -376, -582, -586, -372, -51, 221, 365, 396, 360, 245, -3, -378, -736, -876, -706, -355, -76, -22, -112, -119, 66, 291, 249, -209, -879, -1317, -1200, -601, 70, 410, 329, 62, -104, -105, -111, -295, -602, -770, -573, -56, 481, 723, 582, 231, -86, -268, -387, -554, -766, -887, -781, -448, -38, 259, 349, 260, 89, -84, -235, -372, -503, -629, -744, -828, -835, -694, -365, 106, 570, 836, 785, 452, 14, -312, -407, -305, -164, -142, -285, -487, -578, -457, -177, 90, 195, 143, 96, 237, 593, 975, 1109, 849, 308, -215, -463, -390, -170, -19, -14, -54, 15, 242, 515, 661, 582, 319, -20, -357, -672, -952, -1125, -1088, -812, -415, -112, -55, -195, -313, -204, 136, 503, 649, 502, 217, 21, 7, 53, -30, -291, -567, -627, -394, -33, 201, 195, 68, 37, 185, 372, 375, 118, -239, -444, -377, -141, 63, 138, 155, 233, 364, 402, 215, -153, -498, -637, -581, -506, -555, -680, -697, -498, -190, -23, -143, -435, -610, -462, -58, 316, 396, 135, -290, -646, -805, -778, -642, -462, -290, -174, -150, -182, -166, -4, 279, 512, 495, 173, -288, -610, -600, -292, 82, 268, 165, -116, -346, -335, -66, 285, 472, 342, -27, -370, -421, -127, 286, 499, 367, 63, -71, 167, 636, 952, 820, 296, -260, -501, -364, -84, 59, -7, -142, -179, -116, -83, -159, -238, -117, 271, 743, 982, 816, 387, 40, 36, 332, 627, 608, 198, -406, -893, -1050, -867, -487, -82, 234, 423, 480, 415, 252, 53, -91, -109, -8, 102, 80, -134, -428, -566, -368, 109, 570, 670, 300, -311, -770, -829, -574, -306, -239, -300, -224, 142, 643, 922, 757, 290, -79, -55, 277, 524, 353, -190, -698, -749, -270, 402, 804, 711, 279, -126, -241, -62, 218, 420, 505, 550, 609, 638, 540, 274, -71, -330, -376, -210, 39, 215, 217, 53, -191, -407, -507, -445, -234, 46, 266, 304, 131, -146, -326, -238, 127, 592, 893, 852, 487, -1, -362, -435, -220, 141, 450, 540, 353, -26, -403, -567, -421, -51, 326, 513, 463, 285, 130, 61, 25, -48, -152, -188, -64, 202, 493, 673, 688, 564, 357, 106, -141, -307, -307, -147, 29, 37, -185, -491, -623, -453, -113, 102, 7, -293, -486, -330, 110, 510, 558, 214, -260, -520, -377, 102, 694, 1169, 1382, 1292, 950, 497, 126, -26, 11, 53, -81, -375, -586, -445, 68, 641, 841, 472, -246, -843, -970, -665, -283, -178, -412, -752, -905, -769, -473, -214, -73, 15, 150, 336, 491, 547, 525, 492, 454, 331, 49, -328, -595, -564, -260, 59, 106, -178, -549, -656, -361, 127, 421, 276, -205, -695, -915, -838, -618, -379, -96, 301, 748, 1005, 872, 425, 27, 33, 454, 914, 967, 503, -143, -504, -396, -82, -1, -343, -857, -1078, -736, 16, 748, 1108, 1049, 784, 542, 396, 278, 105, -125, -358, -528, -583, -461, -114, 424, 987, 1330, 1288, 908, 423, 81, -24, 7, 1, -140, -369, -542, -523, -261, 176, 624, 877, 784, 357, -215, -666, -829, -742, -587, -505, -470, -327, 26, 508, 896, 1007, 842, 559, 328, 215, 188, 208, 278, 398, 513, 529, 404, 199, 18, -93, -188, -323, -459, -487, -365, -227, -288, -613, -971, -987, -478, 320, 898, 893, 394, -135, -277, -17, 285, 278, -43, -346, -307, 62, 420, 422, 40, -430, -663, -589, -419, -399, -551, -664, -515, -107, 304, 425, 164, -296, -648, -683, -417, -38, 254, 378, 378, 351, 351, 354, 309, 204, 105, 113, 286, 569, 817, 880, 710, 392, 83, -97, -131, -87, -42, -33, -57, -111, -204, -318, -365, -198, 274, 970, 1597, 1795, 1381, 518, -346, -758, -573, -53, 342, 329, 30, -141, 132, 759, 1256, 1139, 353, -635, -1171, -923, -136, 575, 719, 292, -277, -513, -270, 211, 559, 591, 416, 274, 311, 479, 636, 693, 670, 627, 576, 477, 321, 187, 192, 365, 574, 605, 342, -98, -421, -384, -12, 388, 469, 133, -362, -613, -423, -3, 204, -69, -639, -982, -699, 97, 826, 916, 298, -527, -919, -648, -55, 299, 168, -188, -312, -32, 346, 354, -147, -783, -976, -478, 385, 984, 903, 259, -452, -793, -690, -394, -178, -102, -29, 168, 466, 695, 695, 415, -79, -664, -1193, -1523, -1537, -1215, -671, -112, 281, 453, 467, 428, 403, 422, 503, 656, 830, 898, 723, 299, -193, -492, -446, -138, 197, 387, 449, 529, 689, 803, 658, 196, -360, -647, -453, 85, 579, 690, 385, -63, -315, -217, 133, 521, 794, 933, 982, 957, 816, 528, 126, -295, -638, -852, -921, -841, -611, -273, 74, 309, 372, 304, 214, 172, 149, 57, -146, -383, -499, -395, -114, 177, 333, 343, 331, 441, 704, 1001, 1146, 1017, 627, 111, -357, -638, -672, -489, -192, 84, 246, 289, 288, 327, 412, 458, 364, 119, -183, -419, -547, -626, -725, -810, -739, -390, 187, 764, 1080, 1039, 763, 456, 223, 0, -325, -736, -1055, -1078, -770, -312, 58, 244, 353, 547, 832, 1006, 833, 285, -379, -796, -780, -465, -174, -124, -235, -220, 131, 729, 1228, 1299, 903, 313, -109, -206, -107, -74, -240, -498, -627, -509, -245, -36, 17, 27, 193, 579, 1003, 1153, 844, 183, -500, -897, -919, -704, -447, -240, -47, 192, 462, 671, 746, 704, 617, 521, 371, 94, -309, -729, -1011, -1071, -944, -742, -544, -356, -142, 93, 275, 308, 163, -76, -264, -283, -127, 84, 189, 75, -242, -624, -883, -897, -675, -343, -56, 107, 166, 188, 201, 168, 42, -152, -302, -282, -61, 262, 533, 663, 694, 728, 818, 901, 860, 638, 305, -9, -234, -412, -630, -890, -1051, -923, -435, 272, 931, 1317, 1386, 1242, 999, 684, 271, -197, -562, -639, -395, -15, 224, 182, -19, -103, 113, 540, 878, 862, 478, -35, -385, -430, -242, 6, 184, 260, 256, 184, 51, -109, -234, -279, -260, -230, -200, -110, 99, 386, 574, 480, 93, -367, -596, -436, -8, 381, 492, 317, 59, -68, -21, 54, -27, -333, -745, -1048, -1071, -805, -405, -92, -19, -166, -359, -407, -260, -64, -46, -292, -639, -771, -486, 108, 659, 842, 622, 254, 37, 47, 86, -100, -573, -1126, -1458, -1412, -1067, -635, -288, -80, 13, 25, -22, -82, -88, -23, 28, -70, -372, -744, -925, -733, -222, 333, 644, 621, 409, 231, 196, 233, 203, 68, -60, -29, 189, 424, 418, 35, -585, -1080, -1086, -499, 423, 1205, 1442, 1051, 302, -386, -722, -681, -442, -192, -1, 163, 330, 447, 432, 272, 54, -103, -155, -157, -187, -257, -309, -297, -263, -303, -458, -638, -682, -499, -167, 127, 252, 232, 217, 308, 444, 429, 114, -446, -998, -1251, -1094, -678, -301, -200, -376, -608, -628, -326, 154, 534, 596, 354, 21, -157, -99, 73, 157, 37, -226, -467, -545, -443, -243, -47, 74, 95, 15, -136, -294, -376, -349, -276, -277, -426, -662, -824, -778, -532, -226, -9, 83, 129, 203, 266, 180, -159, -693, -1194, -1409, -1228, -751, -209, 187, 361, 375, 345, 338, 327, 231, 4, -299, -540, -591, -449, -253, -189, -339, -604, -762, -636, -217, 325, 772, 984, 959, 797, 613, 476, 381, 268, 66, -238, -549, -681, -483, 13, 562, 844, 698, 251, -181, -350, -250, -85, -47, -147, -245, -219, -84, 55, 152, 276, 507, 790, 936, 801, 450, 139, 77, 221, 309, 114, -315, -678, -679, -289, 238, 595, 658, 536, 417, 389, 396, 344, 202, 16, -158, -307, -439, -541, -571, -499, -337, -147, -5, 16, -129, -433, -791, -1023, -992, -730, -452, -389, -558, -703, -505, 94, 776, 1063, 721, -32, -710, -958, -809, -576, -499, -526, -416, -28, 500, 875, 907, 647, 295, 6, -183, -276, -230, -13, 273, 375, 76, -568, -1211, -1463, -1217, -727, -364, -275, -289, -137, 229, 565, 552, 121, -454, -775, -630, -136, 387, 677, 681, 533, 401, 373, 433, 513, 545, 495, 369, 199, 24, -132, -260, -327, -276, -70, 237, 494, 547, 388, 195, 197, 453, 767, 832, 504, -46, -455, -458, -96, 341, 573, 536, 375, 276, 310, 414, 470, 385, 103, -374, -945, -1385, -1440, -1033, -393, 53, -4, -500, -1032, -1202, -955, -601, -489, -670, -848, -686, -144, 464, 770, 685, 462, 436, 702, 1044, 1160, 942, 540, 184, -40, -227, -488, -789, -978, -963, -849, -869, -1153, -1560, -1751, -1465, -758, 1, 394, 256, -224, -672, -812, -645, -384, -238, -232, -222, -45, 310, 689, 888, 811, 525, 189, -69, -218, -300, -359, -399, -401, -356, -295, -265, -290, -347, -361, -244, 48, 463, 844, 1009, 865, 499, 136, -11, 103, 347, 541, 634, 737, 978, 1320, 1534, 1373, 810, 101, -394, -488, -292, -105, -159, -455, -791, -933, -764, -314, 276, 816, 1108, 1027, 607, 62, -323, -388, -200, 25, 137, 189, 363, 741, 1169, 1352, 1122, 603, 132, -13, 186, 530, 791, 884, 856, 751, 541, 175, -311, -778, -1077, -1152, -1057, -876, -669, -496, -453, -628, -984, -1289, -1231, -673, 183, 889, 1066, 689, 95, -284, -237, 120, 494, 651, 550, 309, 87, -3, 74, 278, 486, 539, 341, -44, -406, -531, -378, -124, -9, -126, -338, -399, -186, 189, 483, 528, 373, 226, 266, 498, 757, 848, 687, 341, -28, -267, -292, -98, 258, 681, 1045, 1216, 1093, 667, 43, -584, -1035, -1238, -1262, -1239, -1233, -1167, -889, -336, 360, 939, 1193, 1129, 945, 847, 872, 889, 761, 506, 287, 259, 412, 595, 664, 614, 552, 548, 532, 359, -25, -485, -774, -731, -416, -70, 62, -95, -422, -688, -710, -453, -48, 286, 378, 220, -7, -67, 144, 487, 671, 495, 36, -400, -531, -329, -10, 183, 209, 210, 318, 488, 548, 419, 228, 184, 322, 408, 155, -466, -1122, -1379, -1101, -581, -276, -374, -625, -602, -127, 539, 933, 812, 356, -26, -83, 85, 170, -29, -441, -820, -980, -911, -707, -432, -107, 205, 358, 215, -174, -553, -642, -388, -44, 54, -200, -566, -653, -275, 367, 857, 915, 629, 348, 349, 580, 721, 481, -116, -742, -1025, -848, -405, -7, 186, 260, 392, 642, 870, 878, 610, 217, -75, -174, -167, -200, -294, -314, -127, 206, 438, 348, -9, -301, -181, 380, 1027, 1273, 901, 140, -513, -691, -408, 0, 188, 64, -183, -291, -147, 137, 345, 317, 59, -283, -517, -507, -245, 143, 469, 585, 480, 279, 147, 158, 250, 293, 207, 12, -219, -453, -702, -957, -1124, -1066, -740, -280, 72, 151, 3, -170, -218, -187, -260, -542, -897, -1020, -699, -31, 612, 859, 609, 94, -318, -388, -132, 247, 533, 603, 443, 116, -260, -532, -566, -342, -8, 206, 162, -62, -231, -153, 130, 375, 357, 101, -111, 12, 503, 1053, 1256, 947, 356, -61, 1, 455, 884, 880, 364, -352, -808, -722, -176, 477, 866, 821, 431, -56, -395, -446, -216, 172, 532, 683, 535, 150, -276, -516, -440, -104, 288, 522, 522, 397, 356, 539, 887, 1165, 1128, 715, 135, -259, -216, 221, 726, 927, 679, 173, -214, -221, 111, 497, 649, 503, 235, 69, 79, 140, 65, -215, -601, -912, -1027, -943, -729, -465, -217, -62, -67, -231, -435, -495, -306, 43, 311, 286, -32, -430, -651, -599, -390, -204, -109, -10, 218, 566, 850, 853, 505, -48, -559, -846, -898, -824, -740, -691, -656, -604, -531, -451, -374, -302, -238, -192, -171, -167, -159, -136, -99, -52, 12, 110, 243, 382, 488, 542, 564, 577, 571, 491, 279, -65, -461, -784, -928, -846, -553, -109, 385, 797, 994, 902, 564, 120, -266, -492, -537, -420, -164, 193, 550, 739, 612, 171, -385, -763, -749, -349, 236, 750, 1013, 969, 645, 114, -489, -961, -1094, -811, -266, 217, 353, 99, -300, -503, -317, 164, 652, 878, 768, 439, 72, -232, -474, -684, -836, -839, -607, -145, 419, 894, 1105, 972, 527, -111, -791, -1374, -1768, -1948, -1944, -1819, -1631, -1402, -1124, -786, -407, -49, 202, 296, 236, 85, -90, -258, -427, -609, -777, -855, -764, -485, -92, 280, 507, 546, 449, 319, 236, 194, 109, -118, -516, -989, -1352, -1438, -1194, -719, -202, 188, 372, 363, 213, -16, -253, -406, -389, -191, 91, 305, 347, 244, 133, 141, 291, 499, 669, 765, 807, 793, 673, 391, -21, -436, -707, -783, -733, -650, -560, -414, -184, 51, 150, 44, -172, -305, -227, 5, 205, 222, 65, -122, -200, -132, 19, 176, 282, 301, 206, 14, -171, -208, -38, 211, 301, 90, -301, -576, -532, -268, -111, -280, -641, -789, -429, 314, 967, 1108, 737, 251, 66, 243, 464, 361, -123, -694, -970, -799, -332, 155, 487, 647, 673, 562, 281, -134, -553, -824, -891, -808, -662, -494, -302, -120, -45, -165, -453, -746, -849, -669, -276, 161, 486, 612, 533, 306, 46, -112, -98, 12, 29, -195, -605, -933, -911, -505, 33, 379, 400, 242, 137, 159, 162, -24, -369, -631, -590, -272, 71, 223, 217, 282, 559, 896, 956, 557, -96, -528, -382, 278, 1024, 1406, 1271, 794, 261, -162, -451, -598, -526, -160, 436, 1028, 1359, 1339, 1094, 823, 624, 453, 241, 28, -49, 79, 297, 354, 69, -485, -1011, -1197, -950, -441, 31, 263, 241, 95, -29, -64, -11, 100, 247, 398, 511, 548, 512, 457, 444, 498, 578, 614, 557, 413, 220, 24, -143, -265, -337, -376, -425, -516, -624, -647, -468, -62, 434, 782, 796, 487, 54, -260, -346, -265, -154, -87, -32, 62, 182, 246, 214, 168, 252, 530, 886, 1099, 1025, 719, 367, 99, -122, -441, -905, -1357, -1530, -1270, -685, -79, 260, 247, 4, -272, -456, -523, -513, -466, -407, -353, -296, -197, -8, 266, 532, 662, 594, 405, 246, 208, 237, 187, -31, -367, -672, -835, -856, -793, -670, -454, -136, 190, 374, 349, 232, 243, 501, 879, 1088, 917, 436, -38, -188, 77, 571, 994, 1147, 1017, 723, 394, 104, -120, -265, -313, -267, -166, -78, -72, -179, -372, -558, -622, -498, -229, 40, 171, 142, 76, 132, 358, 644, 832, 862, 816, 805, 826, 761, 517, 174, -64, -75, 19, -69, -524, -1198, -1685, -1645, -1083, -341, 184, 335, 227, 55, -97, -273, -481, -601, -468, -63, 428, 754, 814, 731, 707, 803, 882, 756, 396, -16, -239, -178, 47, 208, 142, -151, -547, -885, -1048, -995, -765, -481, -287, -269, -377, -464, -401, -193, 29, 141, 142, 157, 301, 548, 732, 686, 394, 15, -219, -167, 143, 538, 792, 741, 356, -234, -800, -1120, -1096, -791, -360, 61, 413, 660, 734, 557, 150, -287, -478, -280, 167, 539, 600, 416, 275, 395, 691, 862, 700, 318, 51, 128, 442, 659, 558, 246, 38, 118, 345, 407, 149, -246, -443, -278, 76, 289, 190, -92, -291, -262, -83, 88, 211, 370, 617, 844, 863, 626, 333, 278, 566, 988, 1199, 1020, 589, 196, 4, -82, -265, -607, -945, -1051, -854, -501, -192, -1, 147, 324, 472, 455, 235, -33, -117, 71, 381, 554, 472, 251, 104, 103, 115, -32, -324, -522, -386, 59, 499, 589, 284, -104, -175, 191, 711, 934, 640, 25, -471, -538, -231, 112, 183, -67, -435, -679, -709, -594, -434, -251, -13, 271, 520, 650, 675, 711, 852, 1053, 1148, 1000, 641, 251, -7, -140, -287, -540, -789, -786, -377, 302, 884, 1038, 735, 253, -69, -115, -40, -71, -259, -436, -408, -191, -25, -156, -562, -921, -857, -278, 506, 1005, 918, 360, -246, -509, -344, 18, 278, 325, 289, 375, 636, 907, 943, 637, 120, -337, -520, -412, -169, 11, 25, -115, -325, -503, -564, -449, -156, 214, 487, 503, 245, -123, -375, -393, -269, -221, -384, -679, -866, -760, -403, -47, 49, -157, -461, -564, -307, 219, 755, 1043, 988, 668, 254, -97, -315, -400, -381, -264, -27, 345, 798, 1170, 1253, 921, 239, -535, -1097, -1273, -1107, -789, -499, -305, -186, -115, -100, -145, -194, -159, -22, 116, 107, -76, -272, -241, 100, 558, 792, 614, 181, -120, 0, 478, 942, 1027, 668, 127, -255, -334, -223, -131, -152, -218, -215, -122, -32, -30, -100, -132, -35, 167, 361, 435, 364, 209, 53, -52, -97, -86, -23, 79, 203, 331, 453, 549, 569, 447, 160, -216, -529, -634, -505, -252, -34, 62, 58, 22, -26, -133, -338, -588, -748, -714, -521, -339, -344, -559, -825, -912, -692, -239, 230, 513, 548, 434, 328, 317, 363, 342, 163, -142, -434, -558, -463, -249, -92, -120, -318, -550, -652, -541, -238, 166, 565, 855, 948, 794, 432, 2, -309, -385, -249, -35, 129, 212, 254, 280, 233, 36, -295, -601, -676, -447, -55, 243, 277, 84, -154, -288, -308, -310, -348, -361, -241, 27, 315, 443, 338, 95, -119, -215, -235, -270, -327, -296, -67, 325, 669, 707, 339, -280, -826, -1006, -737, -172, 416, 789, 847, 629, 264, -99, -341, -399, -293, -116, 14, 30, -37, -92, -75, -40, -130, -427, -820, -1027, -801, -174, 499, 774, 466, -187, -706, -755, -406, -32, 49, -139, -303, -201, 97, 283, 134, -249, -536, -500, -238, -53, -115, -276, -227, 136, 549, 596, 115, -601, -1042, -922, -421, -8, -15, -379, -760, -881, -746, -568, -522, -593, -649, -605, -504, -442, -456, -501, -519, -494, -432, -324, -160, 9, 70, -41, -229, -269, -19, 395, 645, 504, 107, -114, 169, 838, 1365, 1260, 514, -374, -832, -690, -271, -39, -162, -409, -427, -105, 352, 650, 669, 510, 312, 112, -121, -343, -393, -158, 254, 545, 477, 110, -229, -236, 95, 480, 614, 438, 142, -45, -73, -45, -52, -45, 84, 341, 556, 529, 218, -203, -505, -596, -566, -547, -546, -447, -156, 257, 586, 656, 466, 175, -43, -130, -120, -48, 99, 334, 577, 660, 459, 19, -430, -647, -554, -299, -120, -140, -268, -300, -113, 197, 386, 248, -192, -685, -922, -765, -336, 88, 283, 221, 41, -102, -167, -215, -303, -390, -367, -176, 107, 324, 364, 256, 121, 28, -73, -284, -607, -869, -831, -392, 290, 879, 1093, 895, 501, 196, 125, 215, 264, 121, -205, -553, -720, -596, -233, 191, 473, 495, 286, -5, -210, -233, -94, 94, 210, 182, 34, -127, -156, 61, 523, 1074, 1460, 1468, 1069, 454, -82, -336, -307, -167, -89, -119, -172, -139, -9, 128, 172]])
save('data.npy', data)

In [4]:
import numpy as np
data = np.load('classifier-Y.npy')
print(data)

[[0.17667873 0.18153837 0.17667873 ... 0.19054075 0.18213062 0.1822717 ]
 [0.14888826 0.1609937  0.14888826 ... 0.15050088 0.16027902 0.1604362 ]
 [0.17610013 0.19009434 0.17610013 ... 0.18814002 0.18915027 0.18949841]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [18]:
from scipy.io import wavfile
import soundfile as sf
from numpy import asarray
from numpy import save

data = []

for i in range (1, 60):
    path = r'C:\Users\Alexe\Audioclassify\data\44.1K\{}_01.wav'.format(i)
    data.append(sf.read(path)[0])
    #print(data)
#_, output = wavfile.read(path)

#print(data)

save('x.npy', data)

#import numpy as np
#data = np.load('x.npy')
print(data)

[array([[-0.00134277, -0.00134277],
       [-0.00201416, -0.00201416],
       [-0.00177002, -0.00177002],
       ...,
       [-0.00018311, -0.00018311],
       [ 0.00036621,  0.00036621],
       [ 0.00024414,  0.00024414]]), array([[0.        , 0.        ],
       [0.00018311, 0.00018311],
       [0.00021362, 0.00021362],
       ...,
       [0.00415039, 0.00415039],
       [0.00332642, 0.00332642],
       [0.00164795, 0.00164795]]), array([[0.00112915, 0.00112915],
       [0.00158691, 0.00158691],
       [0.00186157, 0.00186157],
       ...,
       [0.00057983, 0.00057983],
       [0.00045776, 0.00045776],
       [0.00021362, 0.00021362]]), array([[-0.00521851, -0.00521851],
       [-0.00750732, -0.00750732],
       [-0.00799561, -0.00799561],
       ...,
       [ 0.00027466,  0.00027466],
       [ 0.00027466,  0.00027466],
       [ 0.00012207,  0.00012207]]), array([[-9.88769531e-03, -9.88769531e-03],
       [-1.04064941e-02, -1.04064941e-02],
       [-8.14819336e-03, -8.14819336e-03]

In [9]:
from scipy.io import wavfile
import soundfile as sf
from numpy import asarray
from numpy import save

path = r'C:\Users\Alexe\Audioclassify\data\44.1K\1_01.wav'
data, samplerate = sf.read(path)

In [19]:
from scipy.io import wavfile
import soundfile as sf
from numpy import asarray
from numpy import save

path = r'C:\Users\Alexe\OneDrive\Рабочий стол\44.1K\1_01.wav'
data, samplerate = sf.read(path)
#_, output = wavfile.read(path)

print(data)

save('test.npy', data)

import numpy as np
data = np.load('test.npy')
print(data)

RuntimeError: Error opening 'C:\\Users\\Alexe\\OneDrive\\Рабочий стол\\44.1K\\1_01.wav': System error.

In [21]:
import numpy as np
data = np.load('x_train.npy')
print(data)

[[0.17667873 0.18153837 0.17667873 ... 0.19054075 0.18213062 0.1822717 ]
 [0.14888826 0.1609937  0.14888826 ... 0.15050088 0.16027902 0.1604362 ]
 [0.17610013 0.19009434 0.17610013 ... 0.18814002 0.18915027 0.18949841]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
